# Heart transplant recipient clinical and protein markers predict post-surgical primary graft dysfunction

### Setup

In [ ]:
import os
import pickle
import itertools
import numpy as np

seed=0
np.random.seed(seed)

import pandas as pd
import matplotlib
from matplotlib.pylab import plt
from matplotlib.lines import Line2D

from matplotlib.ticker import AutoMinorLocator

from scipy.stats import ttest_rel,ks_2samp, pearsonr, ttest_ind, mannwhitneyu, levene, sem, t, variation
from statsmodels.stats.multitest import multipletests

from sklearn.metrics import confusion_matrix, roc_curve, precision_score, recall_score, roc_auc_score,precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score,precision_recall_curve, average_precision_score, f1_score

from joblib import Parallel, delayed

from functools import reduce

dpi = 400
matplotlib.rcParams['figure.dpi'] = dpi
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['axes.labelweight'] = 'bold'
matplotlib.rcParams['font.weight'] = 'bold'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['figure.titleweight'] = 'bold'

import seaborn as sns

sns.set_style("white")

prospective_color = 'blue'
retrospective_color = 'red'
integrated_color = 'green'

dropbox_figures = '/Users/nickgiangreco/gmail_dropbox/Dropbox/PGD Paper/figures/'
dropbox_data = '/Users/nickgiangreco/gmail_dropbox/Dropbox/PGD Paper/data/'

def mean_and_std(data):

    return np.mean(data), np.std(data)

### Clinical characteristics

see src/r/tableone.R for table 1

In [ ]:
uni_agg = pd.read_csv('../../data/bootstrap_clinical_logit/integrated_logit_bootstrap_pgd_~_clinical_features_lwr_mean_median_upr.csv')
print(uni_agg.variable.unique())
fs = uni_agg.variable.str.replace('_Y','')
fs = fs.str.replace('_',' ')
uni_agg.variable = fs
tmp = uni_agg.set_index('variable').round(4)[['lwr','mean','upr']]
tmp.to_csv(dropbox_data+'clinical_population_associations.csv')
tmp

In [ ]:
uni_agg = pd.read_csv('../../data/bootstrap_clinical_logit/integrated_logit_bootstrap_pgd_~_all_clinical_features_lwr_mean_median_upr.csv')
print(uni_agg.variable.unique())
fs = uni_agg.variable.str.replace('_Y','')
fs = fs.str.replace('_',' ')
uni_agg.variable = fs
tmp = uni_agg.set_index('variable').round(4)[['lwr','mean','upr']]
tmp.to_csv(dropbox_data+'all_clinical_population_associations.csv')
tmp

### Exosome Protein characteristics

#### Identified proteins

In [ ]:
cumc = pd.read_csv('../../data/df_samples_cumc_allsets.csv',index_col=0)
cedar = pd.read_csv('../../data/df_samples_cedar_allsets.csv',index_col=0)
paris = pd.read_csv('../../data/df_samples_paris_allsets.csv',index_col=0)

In [ ]:
from matplotlib_venn import venn3

In [ ]:
co_p100 = cumc.index.values
ce_p010 = cedar.index.values
co_ce_p110 = np.intersect1d(co_p100,ce_p010)
pa_p001 = paris.index.values
co_pa_p101 = np.intersect1d(co_p100,pa_p001)
ce_pa_p011 = np.intersect1d(ce_p010,pa_p001)
co_ce_pa_p111 = np.intersect1d(co_ce_p110,pa_p001)

In [ ]:
fig,ax = plt.subplots(dpi=dpi)
vd = venn3((len(co_p100),len(ce_p010),len(co_ce_p110),
            len(pa_p001),len(co_pa_p101),
            len(ce_pa_p011),len(co_ce_pa_p111)),
           set_labels=['Columbia','Cedars-Sinai','Pitíe Salpetriere'])
ax.set_title('Identified protein markers',pad=8,size=18)

s=15
for text in vd.subset_labels:
    text.set_fontsize(s)
s=16
for text in vd.set_labels:
    text.set_fontsize(s)

fig.savefig(dropbox_figures+'ProteinDescription_venn_diagram.png')

In [ ]:
index = pd.Index(np.union1d(np.union1d(co_p100,ce_p010),pa_p001))
integrated = pd.DataFrame(index=index)
integrated = integrated.join(cumc).join(cedar).join(paris)

In [ ]:
cohort = pd.read_csv('../../data/integrated_sample_groups_imputed_data_raw.csv',index_col=0).set_index('Sample')[['Cohort','PGD']]
tmp = integrated.T
integrated_melted_full = tmp.join(cohort['Cohort']).rename_axis('Sample').reset_index().melt(id_vars=['Cohort','Sample'],var_name='Protein')

In [ ]:
integrated_melted_full.Protein.nunique()

In [ ]:
#integrated_melted_full.groupby(['Cohort','Sample']).agg(sum).sort_values('value').T

In [ ]:
tmp = integrated_melted_full[~integrated_melted_full.value.notna()].groupby('Cohort')['Protein'].unique()
nonidentified_proteins = np.union1d(np.union1d(tmp.iloc[0],tmp.iloc[1]),tmp.iloc[2])
len(nonidentified_proteins)

In [ ]:
print(len(co_ce_pa_p111))
print(len(np.intersect1d(nonidentified_proteins,co_ce_pa_p111)))
print(len(co_ce_pa_p111) - len(np.intersect1d(nonidentified_proteins,co_ce_pa_p111)))

In [ ]:
common_proteins = integrated_melted_full[~integrated_melted_full.Protein.isin(nonidentified_proteins)].Protein.drop_duplicates().values

idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

common_proteins_to_genes = idmap_sub[idmap_sub.Protein.isin(common_proteins)]

display(np.setdiff1d(common_proteins,common_proteins_to_genes.Protein.values))

display(common_proteins_to_genes.shape)

common_proteins_to_genes_immunos = common_proteins_to_genes[common_proteins_to_genes.Gene_name.str.startswith('IG')]

display(common_proteins_to_genes_immunos.shape)

common_proteins_to_genes_no_immunos = common_proteins_to_genes[~common_proteins_to_genes.Gene_name.str.startswith('IG')]

display(common_proteins_to_genes_no_immunos.shape)

pickle.dump(common_proteins_to_genes_no_immunos.Protein.values,open('../../data/proteins_no_immunoglobulins.pkl','wb'))

In [ ]:
common_proteins_to_genes_immunos.to_csv('../../data/IGS_to_genes.csv',index=False)

In [ ]:
common_proteins_to_genes_immunos.to_csv('../../data/identified_IG_uniprot_to_genes.csv')

In [ ]:
tmp = pd.concat([common_proteins_to_genes_no_immunos,
           common_proteins_to_genes_immunos]).Protein.unique()
pickle.dump(tmp,open('../../data/proteins_immunoglobulins.pkl','wb'))

In [ ]:
integrated_melted_full[~integrated_melted_full.Protein.isin(nonidentified_proteins)].Protein.drop_duplicates().to_csv('../../data/integrated_cohort_identified_proteins.csv',index=False) 

integrated_melted_full[~integrated_melted_full.Protein.isin(nonidentified_proteins)].Protein.drop_duplicates().str.split('-').apply(lambda x : x[0]).to_csv('../../data/integrated_cohort_identified_proteins_chopped_isoforms.csv',index=False)

integrated_melted_full[~integrated_melted_full.Protein.isin(nonidentified_proteins)].Protein.nunique() 

#### Protein value distributions

In [ ]:
cumc_df = (cumc.
          rename_axis('Protein').
          loc[common_proteins].
          apply(lambda x : (x - np.mean(x)) / np.std(x),axis=1).
          reset_index().
          melt(id_vars='Protein'))
cumc_df['Cohort'] = 'Columbia'

cedar_df = (cedar.
          rename_axis('Protein').
          loc[common_proteins].
          apply(lambda x : (x - np.mean(x)) / np.std(x),axis=1).
          reset_index().
          melt(id_vars='Protein'))
cedar_df['Cohort'] = 'Cedar-Sinai'

paris_df = (paris.
          rename_axis('Protein').
          loc[common_proteins].
          apply(lambda x : (x - np.mean(x)) / np.std(x),axis=1).
          reset_index().
          melt(id_vars='Protein'))
paris_df['Cohort'] = 'Pitíe Salpetriere'

In [ ]:
matplotlib.rcParams['axes.titlepad'] = 8
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['axes.labelsize'] = 16
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16

fig,ax = plt.subplots(nrows=3,ncols=1,sharex=True,sharey=True,dpi=dpi,figsize=(6,4))

cohorts=['Columbia','Cedar-Sinai','Pitíe Salpetriere']

for i,grp in cumc_df.groupby('variable'):
    sns.distplot(grp['value'],
                 color='Blue',
                 label=cohorts[0],
                 kde=False,
                 ax=ax[0])
    ax[0].set_alpha(0.8)

for i,grp in cedar_df.groupby('variable'):
    sns.distplot(grp['value'],
                 color='Green',
                 label=cohorts[1],
                 kde=False,
                 ax=ax[1])
    ax[1].set_alpha(0.8)

for i,grp in paris_df.groupby('variable'):
    sns.distplot(grp['value'],
                 color='red',
                 label=cohorts[2],
                 kde=False,
                 ax=ax[2])
    ax[2].set_alpha(0.8)
sns.despine()
ax[0].set_xlabel('')
ax[1].set_xlabel('')

for i,a in enumerate(ax):
    a.text(2.5,50,cohorts[i])
    a.set_xlim(-5,5)
ax[1].set_ylabel('Density')
ax[1].yaxis.set_label_coords(-0.1,0)
ax[0].set_title('Exosome protein expression distribution')
ax[2].set_xlabel('Standardized protein expression')

fig.tight_layout()
fig.savefig(dropbox_figures+'ProteinDescription_distributions.pdf')

##### Distribution deviation from normal

In [ ]:
from scipy.stats import normaltest
normaltest(cumc_df['value'].values)

In [ ]:
normaltest(cedar_df['value'].values)

In [ ]:
normaltest(paris_df['value'].values)

##### Distribution significance

In [ ]:
import scipy.stats as sc
print(sc.ks_2samp(cumc_df['value'].values,cedar_df['value'].values))
print(sc.ks_2samp(cedar_df['value'].values,paris_df['value'].values))
print(sc.ks_2samp(cumc_df['value'].values,paris_df['value'].values))

In [ ]:
display(cumc_df['value'].describe())
display(cedar_df['value'].describe())
display(paris_df['value'].describe())

#### Protein correlations

In [ ]:
plt.figure(dpi=200)
integrated.dropna().T.corr('spearman').rename_axis('P1').reset_index().melt(id_vars='P1',var_name='P2')['value'].hist()
plt.ylabel('Number of correlated proteins')
plt.xlabel('Correlation')
plt.tight_layout()
plt.savefig(dropbox_figures+'protein_correlations.pdf')

#### Enrichment of identified proteins in GO categories (via StringDB)

In [ ]:
pd.read_csv('../../data/integrated_cohort_identified_proteins_enrichment.Component.tsv',sep='\t').head()

### Sample x Protein Heatmap

In [ ]:
integrated_melted_full[['Cohort','Sample']].drop_duplicates().set_index('Sample')

In [ ]:
proteins = pickle.load(open('../../data/proteins_immunoglobulins.pkl','rb'))
idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()
dat = (integrated.
 loc[proteins].
 join(idmap_sub.set_index('Protein')).
 set_index('Gene_name').
 T.
 join(integrated_melted_full.
      loc[:,['Cohort','Sample']].
      drop_duplicates().
      set_index('Sample')
     )
)
lut = dict(zip(dat.loc[:,'Cohort'].unique(),'rbg'))
col_colors = dat.loc[:,'Cohort'].map(lut)

In [ ]:
sig_proteins = (pd.read_csv('../../data/individual_clinical_and_protein_01_'+
                            'within_notwithcohorts_marker_performance_statistics.csv').
                feature.
                unique()
               )
dict(zip(dat.columns.isin(sig_proteins),['black','gray']))

In [ ]:
g = sns.clustermap(dat.drop('Cohort',1).T,
               row_cluster=True,col_cluster=True,
               standard_scale=1,col_colors=col_colors,
               cmap='viridis',figsize=(20,80))
g.fig.tight_layout()
g.fig.savefig('../../docs/imgs/samplexgeneheeatmap.png')

### Individual clinical and protein prediction processing/analysis

In [ ]:
from functools import reduce
data_dir='../../data/integrated_pgd_predictions/'
scores = ['roc_auc']
scorers = { 'roc_auc' : roc_auc_score}


#### clinical predictions

In [ ]:
type_='clinical_01_within_notwithcohorts_features_pgd_prediction_'

##### test

files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' in x)
                                            )
        ]

n=50
lsts=[]
scorers = { 'roc_auc' : m.roc_auc_score, 'precision' : m.precision_score, 'recall' : m.recall_score }
feature_mccv_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_protein_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',''))
        dat = pd.read_csv(data_dir+file,index_col=0)
        vals = []
        for b in range(n):
            x = (dat.
                 sample(n=dat.shape[0],replace=True)
                )
            vals.append([feature,b,scorer(x.y_true,x.y_pred)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Feature','Bootstrap',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Feature'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}).
                                   set_index('Feature'))

    display(feature_mccv_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_score_means_dfs.append(feature_mccv_score_means_df)
feature_mccv_score_means_df = pd.concat(feature_mccv_score_means_dfs,
                                        axis=1,sort=True).reset_index()
feature_mccv_score_means_df.head()

##### performances

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
print(len(files))
files[:5]

feature_mccv_score_means_dfs = []
for score in scores:
    lsts=[]
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_protein_prediction_metric_bootstrap_train_test_val.csv',''))
        feature_means_series = (pd.
                                read_csv(data_dir+file,index_col=0).
                                rename(columns={'bootstrap' : 'Bootstrap',
                                                'model' : 'Model'})
                               )
        feature_means_series['Feature'] = feature        
        lsts.append(feature_means_series)
        
    feature_mccv_scores_df = pd.concat(lsts)
    feature_mccv_scores_dfs[score] = feature_mccv_scores_df
    
    feature_mccv_score_means_df = (feature_mccv_scores_df.
                                   groupby(['Model','Feature'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_'+score}))

    display(feature_mccv_score_means_df.sort_values('mean_'+score).tail())

    feature_mccv_score_means_dfs.append(feature_mccv_score_means_df)

feature_mccv_score_means_df = pd.concat([
    feature_mccv_score_means_dfs[i][['mean_'+score]] for 
    i,score in enumerate(scores)],axis=1)

feature_mccv_score_means_df['Feature'] = feature_mccv_score_means_dfs[0]['Feature']
feature_mccv_score_means_df['Model'] = feature_mccv_score_means_dfs[0]['Model']

feature_mccv_score_means_df.head()

In [ ]:
n=50
lsts=[]
feature_mccv_scores_df = {}
feature_mccv_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_prediction_metric_bootstrap_train_test_val'+
                           '_patient_level_data.csv','').
                   replace('CVP_','CVP/')
                  )
        dat = pd.read_csv(data_dir+file,index_col=0)
        vals = []
        for b in range(n):
            x = (dat.
                 sample(n=dat.shape[0],replace=True,random_state=b)
                )
            vals.append([feature,b,x.model.unique()[0],scorer(x.y_true,x.y_proba)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Feature','Bootstrap',
                                                              'Model',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_scores_df[score] = \
    (pd.concat(feature_scores_bootstraps)
    )
    (pd.concat(feature_scores_bootstraps).
     groupby(['Feature','Model'])[score].
     describe(percentiles=[0.025,0.975]).
     loc[:,['2.5%','mean','97.5%']].
     sort_values('mean',ascending=False)
    ).to_csv('../../data/'+type_+score+'_CIs.csv')

    feature_mccv_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Feature','Model'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}))

    display(feature_mccv_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_score_means_dfs.append(feature_mccv_score_means_df)
feature_mccv_score_means_df = (reduce(lambda  left,right: pd.merge(left,right,
                                                                  on=['Feature','Model'],
                                            how='outer'), feature_mccv_score_means_dfs))
feature_mccv_score_means_df.head()

##### importance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' not in x) & 
                                             ('importance' in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
files[:5]

In [ ]:
lsts=[]
for file in files:
    feature = (file.
               replace(type_,'').
               replace('_prediction_metric_bootstrap_train_test_val'+
                       '_feature_importances.csv','').
               replace('CVP_','CVP/')
              )
    feature_logit_df = (pd.read_csv(data_dir+file,index_col=0).
                        rename(columns={'bootstrap' : 'Bootstrap',
                                        'model' : 'Model'}))
    lsts.append(feature_logit_df)

In [ ]:
feature_mccv_importance_odds_df = pd.concat(lsts)
feature_mccv_importance_odds_df['odds'] = np.exp(feature_mccv_importance_odds_df['Importance'])
feature_mccv_odds_df = (feature_mccv_importance_odds_df.
                        groupby(['Feature','Model'])['odds'].
                        describe(percentiles=[0.025,0.975]).
                        loc[:,['2.5%','mean','97.5%']].
                        rename(columns={'2.5%' : 'odds_lwr',
                                        'mean' : 'odds_mean',
                                        '97.5%' : 'odds_upr'}).
                        reset_index())

In [ ]:
feature_mccv_odds_df.query('odds_lwr>1 | odds_upr<1')

In [ ]:
feature_mccv_odds_df.query('Feature=="CVP/PCWP"')

##### permuted performance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' not in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
files[:5]

feature_mccv_permuted_scores_dfs = {}
for score in scores:
    lsts=[]
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_protein_prediction_metric_permute_train_test_val.csv',''))
        feature_means_series = (pd.read_csv(data_dir+file,index_col=0).
                                rename(columns={'bootstrap' : 'Bootstrap','model' : 'Model'}
                                      )
                               )
        feature_means_series['Feature'] = feature
        lsts.append(feature_means_series)
        
    feature_mccv_permuted_scores_df = pd.concat(lsts)
    feature_mccv_permuted_scores_dfs[score] = feature_mccv_permuted_scores_df
    
    feature_mccv_permuted_score_means_df = (feature_mccv_permuted_scores_df.
                                            groupby(['Model','Feature'])[score].
                                            mean().
                                            reset_index().
                                            rename(columns={score : 'mean_permuted_'+score}))

    display(feature_mccv_permuted_score_means_df.sort_values('mean_permuted_'+score).tail())

In [ ]:
n=50
lsts=[]
feature_mccv_permuted_scores_df = {}
feature_mccv_permuted_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_prediction_metric_permute_train_test_val_patient_level_data.csv',''))
        dat = pd.read_csv(data_dir+file,index_col=0)
        vals = []
        for b in range(n):
            x = (dat.
                 sample(n=dat.shape[0],replace=True,random_state=b)
                )
            vals.append([feature,b,x.model.unique()[0],scorer(x.y_true,x.y_proba)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Feature','Bootstrap',
                                                              'Model',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_permuted_scores_df[score] = \
    (pd.concat(feature_scores_bootstraps)
    )
    feature_mccv_permuted_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Feature','Model'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}))

    display(feature_mccv_permuted_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_permuted_score_means_dfs.append(feature_mccv_permuted_score_means_df)
feature_mccv_permuted_score_means_df = (reduce(lambda  left,right: pd.merge(left,right,
                                                                  on=['Feature','Model'],
                                            how='outer'), feature_mccv_permuted_score_means_dfs))
feature_mccv_permuted_score_means_df.head()

##### permuted importance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' not in x) & 
                                             ('importance' in x) & 
                                             ('bootstrap' not in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
files[:5]

In [ ]:
lsts=[]
for file in files:
    feature = (file.
               replace(type_,'').
               replace('_prediction_metric_bootstrap_train_test_val'+
                       '_feature_importances.csv',''))
    feature_logit_df = (pd.read_csv(data_dir+file,index_col=0).
                        rename(columns={'bootstrap' : 'Bootstrap','model' : 'Model'})
                       )
    lsts.append(feature_logit_df)

In [ ]:
feature_mccv_permuted_importance_odds_df = pd.concat(lsts)

feature_mccv_permuted_importance_odds_df['odds'] = \
np.exp(feature_mccv_permuted_importance_odds_df['Importance'])

feature_mccv_permuted_odds_df = (feature_mccv_permuted_importance_odds_df.
                                 groupby(['Feature','Model'])['odds'].
                                 describe(percentiles=[0.025,0.975]).
                                 loc[:,['2.5%','mean','97.5%']].
                                 rename(columns={'2.5%' : 'permuted_odds_lwr',
                                                 'mean' : 'permuted_odds_mean',
                                                  '97.5%' : 'permuted_odds_upr'}).
                                 reset_index())

In [ ]:
feature_mccv_permuted_odds_df.query('permuted_odds_lwr>1 | permuted_odds_upr<1')

##### significant performance

In [ ]:
score = 'roc_auc'
features = feature_mccv_permuted_scores_df[score].Feature.unique()
ms = feature_mccv_permuted_scores_df[score].Model.unique()


pvals = []
for f in features:
    for m in ms:
        bdist = feature_mccv_scores_df[score].query('Model==@m & Feature==@f')[score].values
        pdist = feature_mccv_permuted_scores_df[score].query('Model==@m & Feature==@f')[score].values
        t,pval = ks_2samp(pdist,bdist)
        pvals.append([f,m,t,pval])

In [ ]:
feature_mccv_performance_significance = pd.DataFrame(pvals,
                                                     columns=
                                                     ['Feature',
                                                      'Model',
                                                      'Performance_Statistic',
                                                      'Performance_P_value']
                                                    )

feature_mccv_performance_significance['Performance_bonferroni'] = \
multipletests(feature_mccv_performance_significance.Performance_P_value.values,
              method='bonferroni')[1]

In [ ]:
feature_mccv_performance_significance.head()

##### significant importance

In [ ]:
score = 'roc_auc'
features = feature_mccv_permuted_scores_df[score].Feature.unique()
ms = feature_mccv_permuted_scores_df[score].Model.unique()

pvals = []
for f in features:
    for m in ms:
        f = f.replace('CVP_','CVP/')
        bdist = feature_mccv_importance_odds_df.query('Model==@m & Feature==@f')['Importance'].values
        pdist = feature_mccv_permuted_importance_odds_df.query('Model==@m & Feature==@f')['Importance'].values
        t,pval = ks_2samp(pdist,bdist)
        pvals.append([f,m,t,pval])

In [ ]:
feature_mccv_importance_significance = pd.DataFrame(pvals,columns=['Feature','Model','Importance_Statistic','Importance_P_value'])

feature_mccv_importance_significance['Importance_bonferroni'] = multipletests(feature_mccv_importance_significance.Importance_P_value.values,method='bonferroni')[1]

In [ ]:
feature_mccv_importance_significance

##### performance and importance correlation

In [ ]:
score = 'roc_auc'
display(feature_mccv_scores_df[score].
        set_index(['Feature','Bootstrap','Model'])[[score]].
        head())

display(feature_mccv_importance_odds_df.
        set_index(['Feature','Bootstrap','Model'])[['odds']].
        head())


In [ ]:
score = 'roc_auc'
performances_and_importances_df = (feature_mccv_scores_df[score].
                                   set_index(['Feature','Bootstrap','Model'])[[score]].
                                   join(
                                       feature_mccv_importance_odds_df.
                                       set_index(['Feature','Bootstrap','Model'])[['odds']])).reset_index()
performances_and_importances_df.head()

In [ ]:
corr_df = (performances_and_importances_df.
           dropna().
 groupby(['Feature','Model']).
 apply(lambda x : pearsonr(x.roc_auc,x.odds)[0])
).reset_index().rename(columns={0 : 'Performance_Importance_Correlation'}).set_index(['Feature','Model'])

corr_pvalue_df = (performances_and_importances_df.
                  dropna().
                  groupby(['Feature','Model']).
                  apply(lambda x : pearsonr(x.roc_auc,x.odds)[1]).
                  reset_index().rename(
                      columns={0 : 'Performance_Importance_Correlation_P_value'}).
                  set_index(['Feature','Model'])
                 )

corr_pvalue_df['Performance_Importance_Correlation_bonferroni'] = multipletests(corr_pvalue_df.Performance_Importance_Correlation_P_value,method='bonferroni')[1]

In [ ]:
performances_and_importances_corr_df = corr_df.join(corr_pvalue_df).reset_index()

In [ ]:
performances_and_importances_corr_df.sort_values('Performance_Importance_Correlation')

##### join mean performance, performance significance, importance significance, feature odds, and odds/performance correlation

In [ ]:
mccv_performance_importance_correlation_significance_df = (
    feature_mccv_score_means_df.set_index(['Feature','Model']).
    join(
        feature_mccv_odds_df.set_index(['Feature','Model'])
    ).
    join(
        feature_mccv_permuted_odds_df.set_index(['Feature','Model'])
    ).
    join(
        feature_mccv_performance_significance.set_index(['Feature','Model'])
    ).
    join( feature_mccv_importance_significance.set_index(['Feature','Model'])
        ).
    join(
        performances_and_importances_corr_df.set_index(['Feature','Model'])
    ).
    reset_index()
)
mccv_performance_importance_correlation_significance_df.columns = [x.lower() for x in mccv_performance_importance_correlation_significance_df.columns]
mccv_performance_importance_correlation_significance_df.head()

In [ ]:
clinical_mccv_performance_significance_and_feature_odds_df = \
mccv_performance_importance_correlation_significance_df.copy()

In [ ]:
clinical_mccv_performance_significance_and_feature_odds_df

##### outputting

In [ ]:
(clinical_mccv_performance_significance_and_feature_odds_df.
 to_csv('../../data/clinical_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv'))

#### protein predictions

In [ ]:
type_='protein_raw_01_within_notwithcohorts_features_pgd_prediction_'
proteins_no_immunoglobulins = pickle.load(open('../../data/proteins_no_immunoglobulins.pkl','rb'))
scorers = { 'roc_auc' : roc_auc_score}

##### performances

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
print(len(files))
files[:5]

feature_mccv_scores_dfs = {}
feature_mccv_score_means_dfs = []
for score in scores:
    lsts=[]
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_protein_prediction_metric_bootstrap_train_test_val.csv',''))
        feature_means_series = (pd.
                                read_csv(data_dir+file,index_col=0).
                                rename(columns={'bootstrap' : 'Bootstrap',
                                                'model' : 'Model'})
                               )
        feature_means_series['Feature'] = feature
        
        lsts.append(feature_means_series)
        
    feature_mccv_scores_df = pd.concat(lsts)
    feature_mccv_scores_dfs[score] = feature_mccv_scores_df

    feature_mccv_score_means_df = (feature_mccv_scores_df.
                                   groupby(['Model','Feature'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_'+score}))

    display(feature_mccv_score_means_df.sort_values('mean_'+score).tail())

    feature_mccv_score_means_dfs.append(feature_mccv_score_means_df)

feature_mccv_score_means_df = pd.concat([feature_mccv_score_means_dfs[i][['mean_'+score]] 
                                         for i,score in enumerate(scores)],axis=1)

feature_mccv_score_means_df['Feature'] = feature_mccv_score_means_dfs[0]['Feature']
feature_mccv_score_means_df['Model'] = feature_mccv_score_means_dfs[0]['Model']

feature_mccv_score_means_df.head()

In [ ]:
n=50
lsts=[]
feature_mccv_scores_df = {}
feature_mccv_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',''))
        if feature not in proteins_no_immunoglobulins:
            continue
        else:
            dat = pd.read_csv(data_dir+file,index_col=0)
            vals = []
            for b in range(n):
                x = (dat.
                     sample(n=dat.shape[0],replace=True,random_state=b)
                    )
                vals.append([feature,b,x.model.unique()[0],scorer(x.y_true,x.y_proba)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Feature','Bootstrap',
                                                              'Model',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_scores_df[score] = \
    (pd.concat(feature_scores_bootstraps)
    )

    feature_mccv_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Feature','Model'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}))
    (pd.concat(feature_scores_bootstraps).
     groupby(['Feature','Model'])[score].
     describe(percentiles=[0.025,0.975]).
     loc[:,['2.5%','mean','97.5%']].
     sort_values('mean',ascending=False)
    ).to_csv('../../data/'+type_+score+'_CIs.csv')

    display(feature_mccv_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_score_means_dfs.append(feature_mccv_score_means_df)
feature_mccv_score_means_df = (reduce(lambda  left,right: pd.merge(left,right,
                                                                  on=['Feature','Model'],
                                            how='outer'), feature_mccv_score_means_dfs))
print(feature_mccv_score_means_df.shape)
feature_mccv_score_means_df.head()

##### importance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' not in x) & 
                                             ('importance' in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
files[:5]

In [ ]:
lsts=[]
for file in files:
    feature = (file.
               replace(type_,'').
               replace('_prediction_metric_bootstrap_train_test_val'+
                       '_feature_importances.csv',''))
    if feature not in proteins_no_immunoglobulins:
        continue
    else:
        feature_logit_df = (pd.
                        read_csv(data_dir+file,index_col=0).
                        rename(columns={'bootstrap' : 'Bootstrap','model' : 'Model'}).
                        dropna())
        lsts.append(feature_logit_df)

In [ ]:
feature_mccv_importance_odds_df = pd.concat(lsts)
feature_mccv_importance_odds_df['odds'] = np.exp(feature_mccv_importance_odds_df['Importance'])
feature_mccv_odds_df = feature_mccv_importance_odds_df.groupby(['Feature','Model'])['odds'].describe(percentiles=[0.025,0.975]).loc[:,['2.5%','mean','97.5%']].rename(columns={'2.5%' : 'odds_lwr','mean' : 'odds_mean','97.5%' : 'odds_upr'}).reset_index()

In [ ]:
print(feature_mccv_odds_df.query('odds_lwr>1 | odds_upr<1').shape)
feature_mccv_odds_df.query('odds_lwr>1 | odds_upr<1').head()

##### permuted performances

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' not in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
files[:5]

feature_mccv_permuted_scores_dfs = {}
feature_mccv_permuted_score_means_dfs = []
for score in scores:
    lsts=[]
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_protein_prediction_metric_permute_train_test_val.csv',''))
        feature_means_series = (pd.read_csv(data_dir+file,index_col=0).
                                rename(columns={'bootstrap' : 'Bootstrap',
                                                'model' : 'Model'}))
        feature_means_series['Feature'] = feature
        
        lsts.append(feature_means_series)
        
    feature_mccv_permuted_scores_df = pd.concat(lsts)
    feature_mccv_permuted_scores_dfs[score] = feature_mccv_permuted_scores_df

    feature_mccv_permuted_score_means_df = (feature_mccv_permuted_scores_df.
                                   groupby(['Model','Feature'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_'+score}))

    display(feature_mccv_permuted_score_means_df.sort_values('mean_'+score).tail())

    feature_mccv_permuted_score_means_dfs.append(feature_mccv_permuted_score_means_df)

feature_mccv_permuted_score_means_df = pd.concat([
    feature_mccv_permuted_score_means_dfs[i][['mean_'+score]] for 
    i,score in enumerate(scores)],axis=1)

feature_mccv_permuted_score_means_df['Feature'] = feature_mccv_permuted_score_means_dfs[0]['Feature']
feature_mccv_permuted_score_means_df['Model'] = feature_mccv_permuted_score_means_dfs[0]['Model']

feature_mccv_permuted_score_means_df.head()

In [ ]:
n=50
lsts=[]
feature_mccv_permuted_scores_df = {}
feature_mccv_permuted_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_prediction_metric_permute_train_test_val_patient_level_data.csv',''))
        if feature not in proteins_no_immunoglobulins:
            continue
        else:
            dat = pd.read_csv(data_dir+file,index_col=0)
            vals = []
            for b in range(n):
                x = (dat.
                     sample(n=dat.shape[0],replace=True,random_state=b)
                    )
                vals.append([feature,b,x.model.unique()[0],scorer(x.y_true,x.y_proba)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Feature','Bootstrap',
                                                              'Model',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_permuted_scores_df[score] = \
    (pd.concat(feature_scores_bootstraps)
    )
    feature_mccv_permuted_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Feature','Model'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}))

    display(feature_mccv_permuted_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_permuted_score_means_dfs.append(feature_mccv_permuted_score_means_df)
feature_mccv_permuted_score_means_df = (reduce(lambda  left,right: pd.merge(left,right,
                                                                  on=['Feature','Model'],
                                            how='outer'), feature_mccv_permuted_score_means_dfs))
feature_mccv_permuted_score_means_df.head()

##### permuted importance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' not in x) & 
                                             ('importance' in x) & 
                                             ('bootstrap' not in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]

In [ ]:
len(files)
files[:5]

In [ ]:
lsts=[]
for file in files:
    feature = (file.
               replace(type_,'').
               replace('_prediction_metric_permute_train_test_val'+
                       '_feature_importances.csv',''))
    if feature not in proteins_no_immunoglobulins:
        continue
    else:
        feature_logit_df = (pd.
                    read_csv(data_dir+file,index_col=0).
                    rename(columns={'bootstrap' : 'Bootstrap','model' : 'Model'}).
                    dropna())
    lsts.append(feature_logit_df)

In [ ]:
feature_mccv_permuted_importance_odds_df = pd.concat(lsts)

feature_mccv_permuted_importance_odds_df['odds'] = np.exp(feature_mccv_permuted_importance_odds_df['Importance'])

feature_mccv_permuted_odds_df = (feature_mccv_permuted_importance_odds_df.
                                 groupby(['Feature','Model'])['odds'].
                                 describe(percentiles=[0.025,0.975]).
                                 loc[:,['2.5%','mean','97.5%']].
                                 rename(columns={'2.5%' : 'permuted_odds_lwr','mean' : 'permuted_odds_mean','97.5%' : 'permuted_odds_upr'}).
                                 reset_index())

In [ ]:
feature_mccv_permuted_odds_df.query('permuted_odds_lwr>1 | permuted_odds_upr<1')

##### significant performance

In [ ]:
score = 'roc_auc'
features = feature_mccv_permuted_scores_df[score].Feature.unique()
ms = feature_mccv_permuted_scores_df[score].Model.unique()


pvals = []
for f in features:
    for m in ms:
        bdist = feature_mccv_scores_df[score].query('Model==@m & Feature==@f')[score].values
        pdist = feature_mccv_permuted_scores_df[score].query('Model==@m & Feature==@f')[score].values
        t,pval = ks_2samp(pdist,bdist)
        pvals.append([f,m,t,pval])

In [ ]:
feature_mccv_performance_significance = pd.DataFrame(pvals,
                                                     columns=
                                                     ['Feature',
                                                      'Model',
                                                      'Performance_Statistic',
                                                      'Performance_P_value']
                                                    )

feature_mccv_performance_significance['Performance_bonferroni'] = \
multipletests(feature_mccv_performance_significance.Performance_P_value.values,
              method='bonferroni')[1]

In [ ]:
feature_mccv_performance_significance.head()

##### significant importance

In [ ]:
score = 'roc_auc'
features = feature_mccv_importance_odds_df.Feature.unique()
ms = feature_mccv_permuted_importance_odds_df.Model.unique()

pvals = []
for f in features:
    for m in ms:
        f = f.replace('CVP_','CVP/')
        bdist = feature_mccv_importance_odds_df.query('Model==@m & Feature==@f')['Importance'].values
        pdist = feature_mccv_permuted_importance_odds_df.query('Model==@m & Feature==@f')['Importance'].values
        t,pval = ks_2samp(pdist,bdist)
        pvals.append([f,m,t,pval])

In [ ]:
feature_mccv_importance_significance = pd.DataFrame(pvals,columns=['Feature','Model','Importance_Statistic','Importance_P_value'])

feature_mccv_importance_significance['Importance_bonferroni'] = multipletests(feature_mccv_importance_significance.Importance_P_value.values,method='bonferroni')[1]

In [ ]:
feature_mccv_importance_significance.head()

##### performance and importance correlation

In [ ]:
score = 'roc_auc'
display(feature_mccv_scores_df[score].
        set_index(['Feature','Bootstrap','Model'])[[score]].
        head())

display(feature_mccv_importance_odds_df.
        set_index(['Feature','Bootstrap','Model'])[['odds']].
        head())


In [ ]:
score = 'roc_auc'
performances_and_importances_df = (feature_mccv_scores_df[score].
                                   set_index(['Feature','Bootstrap','Model'])[[score]].
                                   join(
                                       feature_mccv_importance_odds_df.
                                       set_index(['Feature','Bootstrap','Model'])[['odds']])).reset_index()
performances_and_importances_df.head()

In [ ]:
corr_df = (performances_and_importances_df.
           dropna().
 groupby(['Feature','Model']).
 apply(lambda x : pearsonr(x.roc_auc,x.odds)[0])
).reset_index().rename(columns={0 : 'Performance_Importance_Correlation'}).set_index(['Feature','Model'])

corr_pvalue_df = (performances_and_importances_df.
                  dropna().
                  groupby(['Feature','Model']).
                  apply(lambda x : pearsonr(x.roc_auc,x.odds)[1]).
                  reset_index().rename(
                      columns={0 : 'Performance_Importance_Correlation_P_value'}).
                  set_index(['Feature','Model'])
                 )

corr_pvalue_df['Performance_Importance_Correlation_bonferroni'] = multipletests(corr_pvalue_df.Performance_Importance_Correlation_P_value,method='bonferroni')[1]

In [ ]:
performances_and_importances_corr_df = corr_df.join(corr_pvalue_df).reset_index()

In [ ]:
performances_and_importances_corr_df.sort_values('Performance_Importance_Correlation')

##### join mean performance, performance significance, importance significance, feature odds, and odds/performance correlation

In [ ]:
mccv_performance_importance_correlation_significance_df = (
    feature_mccv_score_means_df.set_index(['Feature','Model']).
    join(
        feature_mccv_odds_df.set_index(['Feature','Model'])
    ).
    join(
        feature_mccv_permuted_odds_df.set_index(['Feature','Model'])
    ).
    join(
        feature_mccv_performance_significance.set_index(['Feature','Model'])
    ).
    join( feature_mccv_importance_significance.set_index(['Feature','Model'])
        ).
    join(
        performances_and_importances_corr_df.set_index(['Feature','Model'])
    ).
    reset_index()
)
mccv_performance_importance_correlation_significance_df.columns = [x.lower() for x in mccv_performance_importance_correlation_significance_df.columns]
mccv_performance_importance_correlation_significance_df.head()

##### outputting

In [ ]:
mccv_performance_importance_correlation_significance_df.sort_values('mean_validation_roc_auc')

In [ ]:
protein_mccv_performance_importance_correlation_significance_df = \
(mccv_performance_importance_correlation_significance_df.copy())

In [ ]:
idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

In [ ]:
protein_mccv_performance_significance_and_feature_odds_df = \
(protein_mccv_performance_importance_correlation_significance_df.
 set_index('feature').
 join(
     idmap_sub.set_index('Protein')
 ).reset_index()
)
protein_mccv_performance_significance_and_feature_odds_df

In [ ]:
(protein_mccv_performance_significance_and_feature_odds_df.
 to_csv('../../data/protein_raw_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv'))

#### plot protein and clinical markers

##### integrate

In [ ]:
clinical_mccv_performance_significance_and_feature_odds_df = \
pd.read_csv('../../data/clinical_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',index_col=0)
protein_mccv_performance_significance_and_feature_odds_df = \
pd.read_csv('../../data/protein_raw_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',index_col=0)

In [ ]:
print(clinical_mccv_performance_significance_and_feature_odds_df.shape)
display(clinical_mccv_performance_significance_and_feature_odds_df.head())
print(protein_mccv_performance_significance_and_feature_odds_df.shape)
display(protein_mccv_performance_significance_and_feature_odds_df.head())

In [ ]:
pcis = pd.read_csv('../../data/protein_raw_01_within_notwithcohorts_features_pgd_prediction_roc_auc_CIs.csv',index_col=0)
ccis = pd.read_csv('../../data/clinical_01_within_notwithcohorts_features_pgd_prediction_roc_auc_CIs.csv',index_col=0)

In [ ]:
protein_mccv_performance_significance_and_feature_odds_df = \
protein_mccv_performance_significance_and_feature_odds_df.set_index('feature').join(pcis).reset_index()
clinical_mccv_performance_significance_and_feature_odds_df = \
clinical_mccv_performance_significance_and_feature_odds_df.set_index('feature').join(ccis).reset_index()

In [ ]:
fs = clinical_mccv_performance_significance_and_feature_odds_df.feature.str.replace('_Y','')
fs = fs.str.replace('_',' ')
clinical_mccv_performance_significance_and_feature_odds_df['original_feature'] = \
clinical_mccv_performance_significance_and_feature_odds_df.feature.values
clinical_mccv_performance_significance_and_feature_odds_df.feature = fs
clinical_mccv_performance_significance_and_feature_odds_df.columns

In [ ]:
display(protein_mccv_performance_significance_and_feature_odds_df.head())
protein_mccv_performance_significance_and_feature_odds_df['original_feature'] = \
protein_mccv_performance_significance_and_feature_odds_df.feature.values
protein_mccv_performance_significance_and_feature_odds_df = \
(protein_mccv_performance_significance_and_feature_odds_df.
 drop('feature',axis=1).
 rename(columns={'Gene_name' : 'feature'}))
print(protein_mccv_performance_significance_and_feature_odds_df.shape)
print(protein_mccv_performance_significance_and_feature_odds_df.columns)
display(protein_mccv_performance_significance_and_feature_odds_df.head())

In [ ]:
clinical_mccv_performance_significance_and_feature_odds_df['Marker'] = 'Clinical'
protein_mccv_performance_significance_and_feature_odds_df['Marker'] = 'Protein'
data = pd.concat([
    clinical_mccv_performance_significance_and_feature_odds_df.sort_values('odds_mean'),
    protein_mccv_performance_significance_and_feature_odds_df.sort_values('odds_mean')],
    sort=False).set_index('feature')
display(data.shape)
data.tail()

In [ ]:
data.shape[0]-181

##### filter

In [ ]:
query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'importance_bonferroni<0.001 & (importance_bonferroni>=importance_p_value)'

In [ ]:
(data.
 query(query).
 loc[:,['mean_validation_roc_auc',
        'odds_lwr','odds_mean','odds_upr']].round(4).
 rename(columns={'mean_validation_roc_auc' : 'AUROC',
                 'odds_lwr' : 'Odds lower bound',
                 'odds_mean' : 'Odds average',
                 'odds_upr' : 'Odds upper bound'}).
 sort_values('AUROC',ascending=False).
 to_csv('../../data/individual_clinical_and_protein_01_within_notwithcohorts_marker_performance_statistics.csv')
)


In [ ]:
display(data.query(query))
data.query(query).shape

##### plot

In [ ]:
data.index = [x.split(';')[0][:len(x.split(';')[0])-1]+' family' if len(x.split(';'))>2 else x for x in data.index]

In [ ]:
stat = 'odds_mean'
score='-log10(importance_bonferroni)'
plot_data = data.copy()
plot_data.to_csv(dropbox_data+'all_individual_marker_01_within_notwithcohorts_prediction_results.csv')

sig_markers = plot_data.query(query).index.values
plot_data.loc[:,'Significance'] = 'Not-Significant'
plot_data.loc[plot_data.index.isin(sig_markers),'Significance'] = 'Significant'
plot_data['Marker_Color'] = plot_data['Marker'].map({'Clinical' : 'red','Protein' : 'blue'})
plot_data['-log10(importance_bonferroni)'] = -np.log10(plot_data['importance_bonferroni'])
plot_data.to_csv(dropbox_data+'individual_marker_01_within_notwithcohorts_prediction_results.csv')

plot_data

In [ ]:
print(plot_data.index.nunique())
plot_data.index.values

In [ ]:
display((plot_data.
 query(query).
 sort_values('mean_validation_roc_auc',ascending=False).
 loc[:,['mean_validation_roc_auc','importance_bonferroni',
       'odds_lwr','odds_mean','odds_upr']]
).round(4))
(plot_data.
 query(query).
 sort_values('mean_validation_roc_auc',ascending=False).
 loc[:,['mean_validation_roc_auc','importance_bonferroni',
       'odds_lwr','odds_mean','odds_upr']]
).round(4).to_csv(dropbox_data+'raw_01_within_notwithcohorts_significant_individual_markers.csv')

In [ ]:
plot_data['odds_mean'] = np.log(plot_data['odds_mean'])

In [ ]:
fig,ax = plt.subplots(dpi=dpi,figsize=(5,5))

palette = 'RdBu_r'

plot = plt.scatter(plot_data['odds_mean'].values,
          plot_data['mean_validation_roc_auc'].values,
          c=plot_data['-log10(importance_bonferroni)'].values,
          cmap=palette)

plt.clf()
plt.colorbar(plot)

ax = sns.scatterplot('odds_mean',
                     'mean_validation_roc_auc',
                     data=plot_data,
                     hue='-log10(importance_bonferroni)',
                     style='Marker',
                     palette=palette,
                     edgecolor='k'
                    )
ax.set_ylim(0,1)
ax.set_xlabel(r'$\beta$ coefficient',size=20)
ax.set_ylabel('AUROC',size=20)
ax.legend_.remove()


fig.tight_layout()

fig.savefig(dropbox_figures+'individual_clinical_and_protein_predictive_feature_odds_v_auroc_colored_by_significance.png')

In [ ]:
print(plot_data.query(query).query('Marker=="Protein"').shape)
print(plot_data.query(query).query('Marker=="Clinical"').shape)

In [ ]:
plot_data['odds_mean'] = np.exp(plot_data['odds_mean'])

display((plot_data.
 query(query).
 sort_values('mean_validation_roc_auc',ascending=False).
 loc[:,['2.5%','mean_validation_roc_auc','97.5%','importance_bonferroni',
       'odds_lwr','odds_mean','odds_upr']]
).round(4))
tmp = (plot_data.
 query(query))
tmp['importance_bonferroni'] = \
[np.format_float_scientific(x,
                            unique=False,
                            precision=4)
      for x in tmp['importance_bonferroni']]
(tmp.
 sort_values('mean_validation_roc_auc',ascending=False).
 loc[:,['2.5%','mean_validation_roc_auc','97.5%','importance_bonferroni',
       'odds_lwr','odds_mean','odds_upr']]
).round(4).to_csv(dropbox_data+'raw_01_within_notwithcohorts_significant_individual_markers.csv')

significant auroc/importance difference between protein and clinical markers

In [ ]:
a = plot_data[plot_data.Marker=='Protein']['mean_validation_roc_auc'].values
b = plot_data[plot_data.Marker=='Clinical']['mean_validation_roc_auc'].values
print(ttest_ind(a,b))
print((np.mean(a), np.std(a)))
print((np.mean(b), np.std(b)))

In [ ]:
a = plot_data[plot_data.Marker=='Protein']['odds_mean'].values
b = plot_data[plot_data.Marker=='Clinical']['odds_mean'].values
print(ttest_ind(a,b))
print(mean_and_std(a))
print(mean_and_std(b))

#### AUROC<0.5 checking

In [ ]:
from functools import reduce
data_dir='../../data/integrated_pgd_predictions/'
scores = ['roc_auc']
scorers = { 'roc_auc' : roc_auc_score}
type_='clinical_01_within_notwithcohorts_features_pgd_prediction_'
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]
clinical_features = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_prediction_metric_bootstrap_train_test_val'+
                           '_patient_level_data.csv','').
                   replace('CVP_','CVP/')
                  )
        dat = pd.read_csv(data_dir+file,index_col=0)
        dat['Feature'] = feature
        clinical_features.append(dat)
cpreds = pd.concat(clinical_features)
type_='protein_raw_01_within_notwithcohorts_features_pgd_prediction_'
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' in x) &
                                             ('protein_prediction_metric' not in x) & 
                                             ('clinical_prediction_metric' not in x) &
                                             ('prediction_metric' in x)
                                            )
        ]
protein_features = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_prediction_metric_bootstrap_train_test_val'+
                           '_patient_level_data.csv','').
                   replace('CVP_','CVP/')
                  )
        dat = pd.read_csv(data_dir+file,index_col=0)
        dat['Feature'] = feature
        protein_features.append(dat)
ppreds = pd.concat(protein_features)
cppreds = pd.concat([ppreds,cpreds]).sort_values(['bootstrap','Feature']).set_index('Feature')

In [ ]:
clinical_mccv_performance_significance_and_feature_odds_df = \
pd.read_csv('../../data/clinical_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',index_col=0)
protein_mccv_performance_significance_and_feature_odds_df = \
pd.read_csv('../../data/protein_raw_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',index_col=0)
data = pd.concat([
    clinical_mccv_performance_significance_and_feature_odds_df.sort_values('odds_mean'),
    protein_mccv_performance_significance_and_feature_odds_df.sort_values('odds_mean')],
    sort=False).set_index('feature')

In [ ]:
data.sort_values('mean_validation_roc_auc')

In [ ]:
tmp = (cppreds.
 groupby(['Feature','y_true'])['y_proba'].
 mean().
 reset_index().
 pivot_table(index=['Feature'],columns='y_true',values='y_proba').
 rename(columns={0 : 'control',1 : 'case'}).
       join(data[['mean_validation_roc_auc','odds_mean']])
      )
tmp['AUROC>0.5'] = (tmp['mean_validation_roc_auc']>.5)
fig,ax=plt.subplots(dpi=200)
sns.regplot('control','mean_validation_roc_auc',data=tmp,label='non-PGD',ax=ax)
sns.regplot('case','mean_validation_roc_auc',data=tmp,label='PGD',ax=ax)
ax.legend()
ax.set_xlabel('Average validation probabilities')
ax.set_ylabel('Panel AUROC')
fig.tight_layout()
fig.savefig('../../docs/imgs/AUROC>0.5_figure_1.png')
fig,ax=plt.subplots(dpi=200)
plot = plt.scatter(tmp['control'],tmp['case'],c=tmp['mean_validation_roc_auc'], 
                   cmap = 'viridis',linewidth=.2,edgecolor='black')
plt.colorbar(plot)
ax.set_xlabel('Average non-PGD patient validation probabilities')
ax.set_ylabel('Average PGD patient probabilities')
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

ax.tick_params(axis='both', which='major', labelsize=12)
fig.tight_layout()
fig.savefig('../../docs/imgs/AUROC>0.5_figure_2.png')

fig,ax=plt.subplots(dpi=200)
sns.stripplot('variable','value',hue='AUROC>0.5',
             data=tmp.reset_index().melt(id_vars=['Feature','mean_validation_roc_auc','odds_mean','AUROC>0.5']),
             linewidth=.2,edgecolor='black',size=5)
ax.set_ylabel('Average validation probability')
ax.set_xlabel('Patient type')
co = tmp[tmp.mean_validation_roc_auc>.5].control.values
ca = tmp[tmp.mean_validation_roc_auc>.5].case.values
print(np.mean(co))
print(np.mean(ca))
mannwhitneyu(co,ca)
fig.tight_layout()
fig.savefig('../../docs/imgs/AUROC>0.5_figure_3.png')

fig,ax=plt.subplots(dpi=200)
from pylab import *
plot = plt.scatter(tmp['control'],tmp['case'],c=tmp['AUROC>0.5'], 
                   cmap = cm.get_cmap('PiYG', 2),linewidth=.2,edgecolor='black')
plt.colorbar(plot)
ax.set_xlabel('Average non-PGD patient validation probabilities')
ax.set_ylabel('Average PGD patient probabilities')
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

ax.tick_params(axis='both', which='major', labelsize=12)
fig.tight_layout()
fig.savefig('../../docs/imgs/AUROC>0.5_figure_4.png')

a = cppreds.groupby(['Feature','bootstrap'])['y_true'].sum().values
b = cppreds.query('y_true==1').groupby(['Feature','bootstrap'])['y_proba'].mean().values
fig,ax=plt.subplots(dpi=200)
sns.regplot(a,b,ax=ax)
ax.set_xlabel('Number of PGD patients in validation')
ax.set_ylabel('Average PGD patient validation probabilities')
fig.tight_layout()
fig.savefig('../../docs/imgs/AUROC>0.5_figure_5.png')
tmp['diff'] = tmp['case'] - tmp['control']
a = tmp['diff'].values
b = tmp['mean_validation_roc_auc'].values
fig,ax=plt.subplots(dpi=200)
sns.regplot(a,b,ax=ax)
ax.set_xlabel('Difference in the means of case and control validation probabilities')
ax.set_ylabel('AUROC')
fig.tight_layout()
fig.savefig('../../docs/imgs/AUROC>0.5_figure_6.png')

### Individual marker comparison with and without cohort covariates

In [ ]:
m = pd.read_csv(
    dropbox_data+'all_individual_marker_01_within_prediction_results.csv'
)
m_wcohortcovs = pd.read_csv(
    dropbox_data+'all_individual_marker_01_within_notwithcohorts_prediction_results.csv'
)

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot(m['mean_validation_roc_auc'],
                m_wcohortcovs['mean_validation_roc_auc'],
                edgecolor='black',color='lightgray',lw=.3)
ax.set_ylabel('With covariate adjustment',size=16)
ax.set_xlabel('Without covariate adjustment',size=16)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

ax.tick_params(axis='both', which='major', labelsize=12)

fig.savefig(dropbox_figures+'effect_of_covariate_adjustment_on_individual_markers.png')

In [ ]:
from scipy.stats import pearsonr
pearsonr(m['mean_validation_roc_auc'], m_wcohortcovs['mean_validation_roc_auc'])

### Two marker panel prediction

#### load data

In [ ]:
dir_ = '../../data/'
cohort = 'integrated'

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
proteins_no_immunoglobulins = pickle.load(open('../../data/proteins_no_immunoglobulins.pkl','rb'))
X_all_proteins = X_all_proteins.loc[:,proteins_no_immunoglobulins]

X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_covariates.csv',index_col=0)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)

query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'importance_bonferroni<0.001 & (importance_bonferroni>=importance_p_value)'
predictive_proteins =  \
(pd.
 read_csv('../../data/protein_raw_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',
          index_col=0).
 query(query).
 feature.
 unique()
)
predictive_clinicals =  \
(pd.
 read_csv('../../data/clinical_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',
          index_col=0).
 query(query).
 feature.
 unique()
)

umarkers = np.union1d(predictive_proteins,predictive_clinicals)
len(umarkers)

In [ ]:
def pperf_dat_processing(dat='',set_=0,n=50,scorer=roc_auc_score):
        lsts = []
        for b in range(n):
            lsts.append(
                (dat.
                 sample(n=dat.shape[0],replace=True,random_state=b).
                 groupby('cohort').
                 apply(
                     lambda x : scorer(x.y_true,x.y_proba)
                 )
                )
            )

        vals = []
        for b in range(n):
            x = (dat.
                 sample(n=dat.shape[0],replace=True,random_state=b)
                )
            vals.append(scorer(x.y_true,x.y_proba))

        tmp = \
        pd.concat([
            (pd.concat(lsts,1).
             T.
             describe(percentiles=[0.025,0.975]).
             loc[['2.5%','mean','97.5%']].
             T.
             reset_index()
            ),
            (pd.
             DataFrame(vals,
                       columns=['Integrated']).
             describe(percentiles=[0.025,0.975]).
             loc[['2.5%','mean','97.5%']].
             T.
             reset_index().
             rename(columns={ 'index' : 'cohort'})
            )
        ])
        tmp['set'] = set_
        return tmp

def pperf_processing(pperf_df='',n_jobs=4,params={}):
    tmps = Parallel(n_jobs=n_jobs)(
        delayed(pperf_dat_processing)(
            dat,set_,**params) for 
        set_,dat in pperf_df.groupby('set'))
    return pd.concat(tmps,sort=True)

In [ ]:
basename = '../../data/integrated_pgd_predictions/'+\
'raw_01_within_notwithcohorts_clinicalclinical_proteinclinical_proteinprotein_and_clinical_and_protein_features_small_combos_pgd_prediction_'
feature_set = pickle.load(open(basename+'feature_set_dictionary.pkl','rb'))
print(len(feature_set.items()))
sets_to_use = [k for 
 k,v in feature_set.items() if len(np.setdiff1d(v,umarkers))==0]
print(len(sets_to_use))
features_not_to_see = [x for x in X_all_clinical.columns if 'Cohort_' in x]
features_not_to_see

In [ ]:
len(umarkers)

In [ ]:
all_pperf_df = pd.read_csv(basename+'agg_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
all_pperf_processed_df = pperf_processing(all_pperf_df,
                                          params={'scorer' : roc_auc_score})
all_pperf_processed_df.to_csv(basename+'agg_processed_roc_auc_patient_level_data.csv')
pperf_df = all_pperf_processed_df.query('cohort=="Integrated"').drop('cohort',1)
display(pperf_df.shape)
display(pperf_df.head())

all_perm_pperf_df = pd.read_csv(basename+'agg_permuted_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
all_perm_pperf_processed_df = pperf_processing(all_perm_pperf_df,
                                               params={'scorer' : roc_auc_score})
all_perm_pperf_processed_df.to_csv(basename+'agg_permuted_processed_roc_auc_patient_level_data.csv')
perm_pperf_df = all_perm_pperf_processed_df.query('cohort=="Integrated"').drop('cohort',1)
display(perm_pperf_df.shape)
display(perm_pperf_df.head())


In [ ]:
all_pperf_df = pd.read_csv(basename+'agg_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
perf_df = pd.read_csv(basename+'agg_performance.csv',index_col=0).query('set in @sets_to_use')
display(perf_df.head())
all_perm_pperf_df = pd.read_csv(basename+'agg_permuted_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
perm_perf_df = pd.read_csv(basename+'agg_permuted_performance.csv',index_col=0).query('set in @sets_to_use')
display(perm_perf_df.head())
all_pperf_processed_df = pd.read_csv(basename+'agg_processed_roc_auc_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
pperf_df = all_pperf_processed_df.query('cohort=="Integrated"').drop('cohort',1)
display(pperf_df.head())
fimps_df = (pd.
            read_csv(basename+'agg_feature_importances.csv',
                     index_col=0).
            query('Feature!="Intercept"').
            query('Feature not in @features_not_to_see').
            query('set in @sets_to_use')
           )

fimps_df['Marker'] = 'N/A'
fimps_df['Marker'][fimps_df.Feature.isin(X_all_proteins.columns)] = 'Protein'
fimps_df['Marker'][fimps_df.Feature.isin(X_all_clinical.columns)] = 'Clinical'

display(fimps_df.head())
perm_fimps_df = (pd.
                 read_csv(basename+'agg_permuted_feature_importances.csv',
                          index_col=0).
                 query('Feature!="Intercept"').
            query('Feature not in @features_not_to_see').
                 query('set in @sets_to_use')
                )
display(perm_fimps_df.head())
all_perm_pperf_processed_df = pd.read_csv(basename+'agg_permuted_processed_roc_auc_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
perm_pperf_df = all_perm_pperf_processed_df.query('cohort=="Integrated"').drop('cohort',1)
display(perm_pperf_df.head())

p_c_color_dict = {'Clinical-Clinical' : '#CC3311', #brown 
            'Clinical-Protein' : '#0077BB', #blue
                  'Protein-Clinical' : '#0077BB', #blue
            'Protein-Protein' : '#CCBB44' #yellow
                 }

In [ ]:
tmp = (all_pperf_df.
 groupby(['set','y_true'])['y_proba'].
 mean().
 reset_index().
 pivot_table(index=['set'],columns='y_true',values='y_proba').
 rename(columns={0 : 'control',1 : 'case'})
).join(pperf_df.set_index('set')
      )
fig,ax=plt.subplots(dpi=100)
sns.regplot('control','mean',data=tmp,label='non-PGD',ax=ax)
sns.regplot('case','mean',data=tmp,label='PGD',ax=ax)
ax.legend()
ax.set_xlabel('Average validation probabilities')
ax.set_ylabel('Panel AUROC')
fig,ax=plt.subplots(dpi=100)
plot = plt.scatter(tmp['control'],tmp['case'],c=tmp['mean'], cmap = 'viridis')
plt.colorbar(plot)
ax.set_xlabel('Average non-PGD patient validation probabilities')
ax.set_ylabel('Average PGD patient probabilities')
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

ax.tick_params(axis='both', which='major', labelsize=12)

#### Attribute significance

In [ ]:
import scipy.stats as sm
allpvalues=[]
sets  = [x for x in feature_set.keys() if x in sets_to_use]
print(len(sets))
for set_ in sets:
    fs = feature_set[str(set_)]

    best_params = (all_pperf_df.
                   query('set==@set_')['y_proba'].
                   values)
    perm_params = (all_perm_pperf_df.
                   query('set==@set_')['y_proba'].
                   values)

    try:
        stat,pval = sm.ks_2samp(best_params,perm_params)
        allpvalues.append(pval)
    except:
        pass

import scipy.stats as sm
allpvalues=[]
sets  = [x for x in feature_set.keys()]
for set_ in sets:
    fs = feature_set[set_]

    best_params = (fimps_df.
                   query('Feature!="Intercept"').
                   query('set==@set_')['mean'].
                   values)
    perm_params = (perm_fimps_df.
                   query('Feature!="Intercept"').
                   query('set==@set_')['mean'].
                   values)

    X_cohort = X_all_proteins.join(X_all_clinical)[fs]
    Y_cohort = Y.copy()
    best_ps = predict_probability(X_cohort.values,best_params)
    perm_ps = predict_probability(X_cohort.values,perm_params)

    stat,pval = sm.ks_2samp(best_ps,perm_ps)
    allpvalues.append(pval)

In [ ]:
from statsmodels.stats.multitest import multipletests
bonfs = multipletests(allpvalues,method='bonferroni')[1]

In [ ]:
set_sig_df = pd.DataFrame([
    sets,
    allpvalues,
    bonfs],
    index=['set','pvalue','bonferroni']).T
print(set_sig_df.shape)
set_sig_df['set'] = set_sig_df['set'].astype(int)
set_sig_df.head()

In [ ]:
perf_sig_df = pperf_df.set_index('set').join(set_sig_df.set_index('set')).reset_index()
perf_sig_df.head()

In [ ]:
alpha=0.05
bonf_thresh = (alpha / len(perf_sig_df['set'].values))
print(bonf_thresh)
print(perf_sig_df.shape)
print(perf_sig_df.query('bonferroni>=pvalue & bonferroni<@bonf_thresh').shape)
not_sig_set = perf_sig_df.query('bonferroni<=@bonf_thresh').set.values
sig_sets = perf_sig_df.query('bonferroni>=pvalue & bonferroni<@bonf_thresh').set.values

In [ ]:
perf_sig_df.to_csv(basename+'_01_within_notwithcohorts_set_significant_performance.csv')

In [ ]:
fimps_sigs_df = fimps_df[fimps_df['set'].isin(sig_sets)]
fimps_sigs_df = fimps_df.copy()

#### Marker pairwise prediction heatmap

In [ ]:
fimps_spread_1 = \
(fimps_sigs_df.
 query('Feature!="Intercept"').
 loc[:,['set','Feature']].
 rename(columns={'Feature' : 'Feature1'}).
 drop_duplicates().
 groupby('set').
 nth(0).
 join(
     fimps_sigs_df.
     query('Feature!="Intercept"').
     loc[:,['set','Feature']].
     drop_duplicates().
     rename(columns={'Feature' : 'Feature2'}).
     groupby('set').
     nth(1)
 )
)
print(fimps_spread_1.shape)
display(fimps_spread_1.head())

In [ ]:
fimps_spread_2 = \
(fimps_spread_1.
 loc[:,['Feature2','Feature1']].
 rename(columns={'Feature1' :'Feature2','Feature2' : 'Feature1'}
       )
)
fimps_spread = pd.concat([fimps_spread_1,fimps_spread_2])
print(fimps_spread.shape)
display(fimps_spread.head())

In [ ]:
f1="H0YAC1"
f2="Prior_Inotrope_Y"
display(fimps_spread.query('Feature1==@f1 & Feature2==@f2'))
fimps_spread.query('Feature1==@f2 & Feature2==@f1')

In [ ]:
perf_fimps_join = \
(pperf_df.
 reset_index().
 loc[:,['set','mean']].
 rename(columns={'mean' : 'mean_auroc'}).
 drop_duplicates().
 set_index('set').
 join(
     fimps_spread
 ).
 reset_index().
 pivot_table(index='Feature1',columns='Feature2',values='mean_auroc')
)
print(perf_fimps_join.shape)
display(perf_fimps_join.head())

In [ ]:
col_sorted_features = perf_fimps_join.mean(0).sort_values(ascending=True).index.values
row_sorted_features = perf_fimps_join.mean(1).sort_values(ascending=True).index.values

In [ ]:
plot_data = perf_fimps_join.loc[col_sorted_features,row_sorted_features]
display(plot_data.head())
plot_data

In [ ]:
minpt = perf_df['mean'].min()
maxpt = perf_df['mean'].max()
midpt = perf_df['mean'].max() - ((perf_df['mean'].max() - perf_df['mean'].min())/2)

In [ ]:
mask = np.zeros_like(plot_data, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

In [ ]:
fig,ax=plt.subplots(dpi=200,figsize=(30,30))
sns.heatmap(
    plot_data,
    mask=mask,
    square=True,
    vmin=minpt,
    vmax=maxpt,
    center=midpt, 
    cmap='RdBu_r',
    cbar_kws={"orientation": "horizontal"},
    ax=ax)
ax.set_ylabel('')
ax.set_xlabel('')
ax.set_xticklabels('')
ax.set_yticklabels('')
ax.tick_params(axis='both', which='both', length=0)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=70)
fig.savefig(dropbox_figures+'two_marker_panel_predictions_marker_combo_performance_hearmap.png')

#### Marker marker type and performance of clinical-clinical, protein-clinical, and protein-protein markers

In [ ]:
pairwise_sets = (fimps_sigs_df.groupby('set')['Feature'].count()>2).index.values
len(pairwise_sets)

In [ ]:
tmp = fimps_sigs_df[fimps_sigs_df['set'].isin(pairwise_sets)]
cs = tmp.query('Marker=="Clinical"').Feature.unique()
ps = tmp.query('Marker=="Protein"').Feature.unique()
c_c_scores=[]
for c1 in cs:
    for c2 in cs:
        if c1!=c2:
            setsc1 = tmp.query('Feature==@c1').set.unique()
            setsc2 = tmp.query('Feature==@c2').set.unique()
            set_ = np.intersect1d(setsc1,setsc2)
            if len(set_)>0:
                c_c_scores.append(pperf_df.query('set in @set_')['mean'].tolist()[0])
p_c_scores=[]
for p in ps:
    for c in cs:
        if p!=c:
            setsp = tmp.query('Feature==@p').set.unique()
            setsc = tmp.query('Feature==@c').set.unique()
            set_ = np.intersect1d(setsp,setsc)
            if len(set_)>0:
                p_c_scores.append(pperf_df.query('set in @set_')['mean'].tolist()[0])
p_p_scores=[]
for p1 in ps:
    for p2 in ps:
        if p1!=p2:
            setsp1 = tmp.query('Feature==@p1').set.unique()
            setsp2 = tmp.query('Feature==@p2').set.unique()
            set_ = np.intersect1d(setsp1,setsp2)
            if len(set_)>0:
                p_p_scores.append(pperf_df.query('set in @set_')['mean'].tolist()[0])

p_c_dict = {
    #'Clinical-Clinical' : c_c_scores, 
            'Clinical-Protein' : p_c_scores, 
            'Protein-Protein' : p_p_scores}

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
tmp = \
(pd.
 DataFrame(
     dict([ (k,pd.Series(v)) for k,v in p_c_dict.items() ])
 )
)
sns.boxplot('variable','value',data=tmp.melt(),
              ax=ax,palette=p_c_color_dict,fliersize=0)
sns.stripplot('variable','value',
              data=tmp.melt(),ax=ax,
              size=4,palette=p_c_color_dict,
              edgecolor='black',linewidth=.5,
             jitter=True)
ax.set_xlabel('')
ax.set_ylim(0.4,0.8)
ax.set_ylabel('AUROC',size=16)
ax.set_xticklabels([x.get_text()+'\npanels' for x in ax.get_xticklabels()],rotation=20,size=16)
fig.tight_layout()
fig.savefig(dropbox_figures+'two_marker_panel_predictions_marker_type_combo_performances.png')

a = tmp['Clinical-Clinical'].dropna().values
b = tmp['Clinical-Protein'].dropna().values
print(ttest_ind(a,b))

a = tmp['Clinical-Clinical'].dropna().values
b = tmp['Protein-Protein'].dropna().values
print(ttest_ind(a,b))

In [ ]:


a = tmp['Clinical-Protein'].dropna().values
b = tmp['Protein-Protein'].dropna().values
print(ttest_ind(a,b))
print(np.mean(a), np.std(a))
print(np.mean(b), np.std(b))

#### Marker marker type and performance of combos with particular protein characteristics

In [ ]:
sets = pperf_df.sort_values('mean',ascending=False).set.unique()
anchor_proteins = (fimps_sigs_df.
                   query('set in @sets').
                   query('Marker=="Protein"').
                   query('Feature!="Intercept"').
                   sort_values('Feature',ascending=False).
                   Feature.unique()
                  )
scores_w_p = {}
for p in anchor_proteins:
    sets = fimps_sigs_df.query('Feature==@p').set.unique()
    scores = []
    for s in sets:
        scores.append(pperf_df.query('set==@s')['mean'].tolist())
    scores_w_p[p] = list(itertools.chain(*scores))
    
fig,ax=plt.subplots(dpi=dpi,figsize=(15,3))
tmp = \
(pd.
 DataFrame.
 from_dict(scores_w_p, orient='index').
 T
)

protein_order = tmp.mean(0).sort_values(ascending=True).index.values
map_ = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']]
gene_order=[]
for po in protein_order:
    gene_order.append(map_[map_.Protein==po].Gene_name.values[0])

sns.boxplot('variable','value',
              data=tmp.melt(),order=protein_order,
              ax=ax,color='lightgray',fliersize=0)
sns.stripplot('variable','value',
              data=tmp.melt(),order=protein_order,
              ax=ax,
              color='black',size=2,
              edgecolor='black',linewidth=.5,
             jitter=True)
ax.set_xlabel('')
ax.set_ylim(0.45,0.75)
ax.set_ylabel('AUROC',size=16)
ax.set_xticklabels(gene_order)
ax.set_xticklabels([x.get_text() if len(x.get_text().split(';'))==1 
                    else (
                        ''+x.get_text().split(';')[0][:len(x.get_text().split(';')[0])-1]+
                        '\nfamily' 
                        if len(x.get_text().split(';'))>3 
                        else ';\n'.join(x.get_text().split(';')))
                    for x in ax.get_xticklabels()],
                   rotation=20,size=16)
fig.tight_layout()
fig.savefig(dropbox_figures+'two_marker_panel_predictions_protein_combo_performance.png')

In [ ]:
c1 = 'H0YAC1'
for c2 in tmp.columns:
    print('\t'+c2)
    a = tmp[c1].dropna().values
    b = tmp[c2].dropna().values
    print('\t',ttest_ind(a,b))

#### Marker marker type and performance of combos with particular clinical characteristic

In [ ]:
fimps_sigs_df['Marker'] = 'N/A'
fimps_sigs_df['Marker'][fimps_sigs_df.Feature.isin(X_all_proteins.columns)] = 'Protein'
fimps_sigs_df['Marker'][fimps_sigs_df.Feature.isin(X_all_clinical.columns)] = 'Clinical'
cs = fimps_sigs_df.query('Marker=="Clinical"').Feature.unique()
scores_w_c = {}
for c in cs:
    sets = fimps_sigs_df.query('Feature==@c').set.unique()
    scores = []
    for s in sets:
        scores.append(pperf_df.query('set==@s')['mean'].tolist())
    scores_w_c[c] = list(itertools.chain(*scores))
    
fig,ax=plt.subplots(dpi=dpi,figsize=(2,3))
tmp = \
(pd.
 DataFrame(
     dict([ (k,pd.Series(v)) for k,v in scores_w_c.items() ])
 )
)
fs = tmp.columns.str.replace('_Y','')
fs = fs.str.replace('_',' ')
tmp.columns = fs

x  = 'variable'
y = 'value'

sns.boxplot(x,y,data=tmp.melt(),
              ax=ax,color='lightgray',fliersize=0)
sns.stripplot(x,y,
              data=tmp.melt(),ax=ax,
              color='black',size=5,
              edgecolor='black',linewidth=.5,
             jitter=True)
arr = tmp.melt()['value'].values
arr = np.round(np.arange(0.60,0.74,.02),2)
ax.set_yticklabels(arr,size=10)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticklabels('')
#ax.set_ylim(0.45,0.75)
#ax.set_xlabel('AUROC',size=16)
#ax.set_xticklabels([x.get_text() for x in ax.get_xticklabels()],rotation=20,size=16)
#ax.set_xticklabels([x.get_text() if x.get_text()!='Mechanical Support' else 'Mechanical\nSupport' for x in ax.get_xticklabels()])
#ax.set_xticklabels([x.get_text() if x.get_text()!='Beta Blocker' else 'Beta\nBlocker' for x in ax.get_xticklabels()])
#ax.set_xticklabels([x.get_text() if x.get_text()!='Prior Inotrope' else 'Inotrope\ntherapy' for x in ax.get_xticklabels()])
#ax.set_xticklabels([x.get_text() if x.get_text()!='Antiarrhythmic Use' else 'Antiarrhythmic\nuse' for x in ax.get_xticklabels()])
#ax.set_xticklabels([x.get_text() if x.get_text()!='Blood Type O' else 'Blood\nType O' for x in ax.get_xticklabels()])
#ax.set_xticklabels([x.get_text() if x.get_text()!='Ischemic Time' else 'Ischemic\ntime' for x in ax.get_xticklabels()])
fig.tight_layout()
fig.savefig(dropbox_figures+'two_marker_panel_predictions_clinical_combo_performances.png')

In [ ]:
for c1 in tmp.columns.values:
    print(c1)
    for c2 in tmp.columns.values:
        if c1!=c2:
            print('\t'+c2)
            a = tmp[c1].dropna().values
            b = tmp[c2].dropna().values
            print('\t',ttest_ind(a,b))

#### Combo performances within and between cohorts

In [ ]:
basename = '../../data/integrated_pgd_predictions/'+\
'raw_01_within_notwithcohorts_clinicalclinical_proteinclinical_proteinprotein_and_clinical_and_protein_features_small_combos_pgd_prediction_'
feature_set = pickle.load(open(basename+'feature_set_dictionary.pkl','rb'))
sets_to_use = [k for 
 k,v in feature_set.items() if len(np.setdiff1d(v,umarkers))==0]
all_pperf_df = pd.read_csv(basename+'agg_patient_level_data.csv',index_col=0).query('set in @sets_to_use')
print(all_pperf_df.shape)

In [ ]:
def get_scores(all_pperf_df,set_,n=50,scorer=roc_auc_score):
    fs = feature_set[set_]
    
    dat=all_pperf_df.query('set==@set_')
    
    lsts = []
    for b in range(n):
        lsts.append(
            (dat.
             sample(n=dat.shape[0],replace=True,random_state=b).
             groupby('cohort').
             apply(
                 lambda x : scorer(x.y_true,x.y_proba)
             )
            )
        )
    cohort_meaen_series = pd.concat(lsts,1).T.mean()

    vals = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        vals.append(scorer(x.y_true,x.y_proba))
    
    cedar,cumc,paris,all_ = (pd.
     concat([pd.concat(lsts,1).
             T.
             mean(),
             pd.Series(np.mean(vals),
                       index=['Integrated'])
            ]
           ).
     values)
    return [all_,cumc,cedar,paris,set_,fs,
                   (cumc+cedar+paris)/3,(all_+cumc+cedar+paris)/4,
           np.var([cumc,cedar,paris]),np.var([all_,cumc,cedar,paris]),
           np.std([cumc,cedar,paris]),np.std([all_,cumc,cedar,paris])]

In [ ]:
import scipy.stats as sm

sets = all_pperf_df['set'].astype(str).unique()
params={ 'scorer' : roc_auc_score }

scores = Parallel(n_jobs=4,backend='threading')(delayed(get_scores)(all_pperf_df,set_,**params) for set_ in sets)


In [ ]:
m = (pd.DataFrame(np.array(scores),
              index=sets,
              columns=['all','cumc','cedar','paris','set','markers',
                       'avg_cohort_score','avg_score',
                      'var_cohort_score','var_score',
                      'std_cohort_score','std_score']
             ))
m.head()

In [ ]:
p1 = ['Clinical' if x[0] in X_all_clinical.columns else 'Protein' for x in m.markers]
p2 = ['Clinical' if np.any(X_all_clinical.columns.isin(x[1:2])) else 'Protein' for x in m.markers]
m['Marker type'] = [a_+'-'+b_ for a_,b_ in zip(p1,p2)]

In [ ]:
m['Marker type'][m['Marker type'].isin(['Protein-Clinical'])] = 'Clinical-Protein'

In [ ]:
m.sort_values('all',ascending=False).to_csv('../../data/marker_combos_01_within_notwithcohorts_inter_intra_cohort_mean_roc_auc.csv')

In [ ]:
m = pd.read_csv('../../data/marker_combos_01_within_notwithcohorts_inter_intra_cohort_mean_roc_auc.csv',index_col=0)
m.markers = [x.strip('][').split(', ') for x in m.markers]

In [ ]:
top_sets = m.sort_values(['all']).tail(3).set.values
(fimps_df.query('set in @top_sets').
 pivot_table(index='set',columns='Feature',values='mean').
 loc[top_sets])

In [ ]:
top10_sets = m.sort_values(['all']).tail(10).set.values
pperf_df.query('set in @top10_sets').sort_values('mean',ascending=False)

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
tmp = (m[[len(x)==2 for x in m.markers]].
 set_index('markers').
 sort_values('avg_score').
 loc[:,['all','avg_score','Marker type']])
sns.scatterplot('all','avg_score',hue='Marker type',data=tmp,ax=ax,
                linewidth=.5,s=50,palette=p_c_color_dict,
                edgecolor='black')
ax.legend(frameon=False)
#ax.set_xlim(0.45,0.75)
#ax.set_ylim(0.45,0.75)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

ax.set_ylabel('Cohort average AUROC',size=18)
ax.set_xlabel('Integrated cohort AUROC',size=18)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.tight_layout()
fig.savefig(dropbox_figures+'two_marker_panel_predictions_avg_cohort_vs_integrated_performance.png')

#### Variation of panel performance across cohorts

In [ ]:
m.sort_values('all').tail(50).sort_values('std_score').head(5)

In [ ]:
m['1/coefficient_of_variation'] = 1/((m['std_score'] / m['avg_score']).values)
m['1/var-mean'] = 1/((m['var_score'] / m['avg_score']).values)
m['1/cohort_coefficient_of_variation'] = \
1/((m['std_cohort_score'] / m['avg_cohort_score']).values)

In [ ]:
m.sort_values('1/coefficient_of_variation').tail(5)

In [ ]:
m.sort_values('all').tail(5)

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot('avg_cohort_score','all',
                hue='Marker type',data=m,ax=ax,
                linewidth=.5,s=50,palette=p_c_color_dict,
                edgecolor='black')
ax.legend().remove()
ax.set_ylabel('AUROC',size=18)
ax.set_xlabel('Cohort Mean',size=18)
ax.tick_params(axis='both', which='major', labelsize=12)
    
fig.tight_layout()

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot('var_score','all',
                hue='Marker type',data=m,ax=ax,
                linewidth=.5,s=50,palette=p_c_color_dict,
                edgecolor='black')
ax.legend().remove()
ax.set_ylabel('Performance',size=18)
ax.set_xlabel('Variation',size=18)
ax.tick_params(axis='both', which='major', labelsize=12)
    
fig.tight_layout()
fig.savefig(dropbox_figures+'Variation_vs_Performance_for_two_marker_panels.png')

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot('1/var-mean','all',
                hue='Marker type',data=m,ax=ax,
                linewidth=.5,s=50,palette=p_c_color_dict,
                edgecolor='black')
ax.legend().remove()
ax.set_ylabel('AUROC',size=18)
ax.set_xlabel('Variation/Mean',size=18)
ax.tick_params(axis='both', which='major', labelsize=12)
    
fig.tight_layout()

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot('1/cohort_coefficient_of_variation','all',
                hue='Marker type',data=m,ax=ax,
                linewidth=.5,s=50,palette=p_c_color_dict,
                edgecolor='black')
ax.legend().remove()
ax.set_ylabel('AUROC',size=18)
ax.set_xlabel(r'$Cohort Coefficient\ of\ Variation^{-1}$',size=18)
ax.tick_params(axis='both', which='major', labelsize=12)
    
fig.tight_layout()

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot('1/coefficient_of_variation','all',
                hue='Marker type',data=m,ax=ax,
                linewidth=.5,s=50,palette=p_c_color_dict,
                edgecolor='black')
ax.legend().remove()
ax.set_ylabel('AUROC',size=18)
ax.set_xlabel(r'$Coefficient\ of\ Variation^{-1}$',size=18)
ax.tick_params(axis='both', which='major', labelsize=12)
    
fig.tight_layout()
fig.savefig(dropbox_figures+'Inv_CV_vs_AUROC_for_two_marker_panels.png')

#### correlation between aurocs w and wo cohort covariates

In [ ]:
m = pd.read_csv('../../data/marker_combos_01_within_inter_intra_cohort_mean_roc_auc.csv',index_col=0)
m.markers = [x.strip('][').split(', ') for x in m.markers]
m_wcohortcovs = pd.read_csv('../../data/marker_combos_01_within_wcohortcovs_inter_intra_cohort_mean_roc_auc.csv',index_col=0)
m_wcohortcovs.markers = [x.strip('][').split(', ') for x in m_wcohortcovs.markers]

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot(m['all'],m_wcohortcovs['all'],edgecolor='black',color='lightgray',lw=.3)
ax.set_ylabel('With covariate adjustment',size=16)
ax.set_xlabel('Without covariate adjustment',size=16)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)


ax.tick_params(axis='both', which='major', labelsize=12)

fig.savefig(dropbox_figures+'effect_of_covariate_adjustment.png')

output table with two marker panels better than individual markers

In [ ]:
m = pd.read_csv('../../data/marker_combos_01_within_notwithcohorts_inter_intra_cohort_mean_roc_auc.csv',index_col=0)
m_individual = pd.read_csv(
    dropbox_data+'all_individual_marker_01_within_notwithcohorts_prediction_results.csv',
    index_col=0
)
umarkers = m_individual.original_feature.unique()
markers = [x.strip('\'[]\'').split('\', \'') for x in m.markers]
rmarkers = []
for x in markers:
    lst=[]
    for i in x:
        lst.append(m_individual[m_individual.original_feature.isin([i])].index.values[0])
    rmarkers.append(' and '.join(lst))
m['names'] = rmarkers
m_pperf_df = m.set_index('set').join(pperf_df.set_index('set'))
tmp = (m_pperf_df.loc[:,['names','2.5%','mean','97.5%']].
 sort_values('mean',ascending=False)
)
tmp = tmp[tmp['2.5%']>0.6655].reset_index(drop=True)
tmp.to_csv(dropbox_data+'table_of_two_marker_panels_significantly_better_than_individual_markers.csv')
tmp

### Differential expression & GSEA 

using src/python/bootstrap_conditional_protein_logit.py to create protein/gene statistics

using 20190104_GSEA.ipynb for creating files and running GSEA

In [ ]:
import gseapy as gp
gp.__version__

In [ ]:
uniprot = pd.read_csv('../../data/uniprot-all_20171124.tab.gz',sep='\t')

In [ ]:
characterized_prots = uniprot.query('Organism=="Homo sapiens (Human)"').Entry.values

In [ ]:
idmap = uniprot[['Entry','Gene names  (primary )']].rename(columns={'Entry' : 'Protein',"Gene names  (primary )" : 'Gene_name'})
idmap_sub = idmap[idmap.Protein.isin(characterized_prots)]
idmap_sub.to_csv('../../data/gene_list.txt',sep='\n',header=None,index=None)

In [ ]:
stat = 'mean'
cohort='integrated'
dir_ = "../../data/bootstrap_conditional_protein_logit/"

#### Generate results

In [ ]:
cohort='integrated'
print(cohort)
logit = pd.read_csv(dir_+cohort+
            "/logit_bootstrap_pgd_~_protein_+_cohort_-_paris_lwr_mean_median_upr.csv")
print(logit.shape)

#Joining genes
tmp = logit.set_index('variable').join(idmap_sub.set_index('Protein'))
leftover_inds = tmp.Gene_name.isnull()
leftover_prots = tmp.index[leftover_inds].values
leftover_prots_split = [k.split('-')[0] for k in leftover_prots]

tmp_df = pd.DataFrame({'Protein' : leftover_prots,
                       'Split' : leftover_prots_split,
                       'cohort_identified_in' : cohort})

tmp_df_join = tmp_df.set_index('Split').join(idmap_sub.set_index('Protein'))

join_genes = tmp_df_join.Gene_name.values
join_prots = tmp_df_join.Protein.values

tmp.at[join_prots,'Gene_name'] = join_genes

#single_gene_map = \
#(tmp.
# Gene_name.
# str.
# split('; ').
# apply(pd.Series).
# rename_axis('Protein').
# reset_index().
# melt(id_vars=['Protein'],value_name='Gene_name').
# dropna().
# drop('variable',axis=1).
# set_index('Protein')
#)

#tmp_single_genes = \
#(tmp.
# dropna().
# drop('Gene_name',axis=1).
# join(single_gene_map).
# drop_duplicates().
# sort_values('mean')
#)

null_prots = tmp_df_join[tmp_df_join.Gene_name.isnull()].index.values
df = (tmp[~tmp.index.isin(null_prots)].
      reset_index(drop=True).
      set_index('Gene_name')
     )
df_sig = df[(df.lwr>1) | (df.upr<1)]
#

logit_rnk = (df[[stat]].
         sort_values(stat,ascending=False).
         reset_index().
         rename(columns={stat : 'Statistic','Gene_name' : 'Gene'}))

df = (logit_rnk.
  groupby('Gene').
  agg('mean').
  reset_index().
  sort_values('Statistic',ascending=False))
print(df.shape)
(df.
 to_csv('../../data/'+cohort+
        '_bootstrap_conditional_protein_logit_'+stat+
        '_gene_name_statistic.rnk',
        sep='\t',header=None,index=None))

df_no_IGs = df[~(df.
 Gene.
 str.
startswith('IG'))]
(df_no_IGs.
 to_csv('../../data/'+cohort+
        '_bootstrap_conditional_protein_logit_'+stat+
        '_gene_name_statistic_no_IGs.rnk',
        sep='\t',header=None,index=None))

logit_rnk_sig = (df_sig[[stat]].
         sort_values(stat,ascending=False).
         reset_index().
         rename(columns={stat : 'Statistic','Gene_name' : 'Gene'}))

df_sig = (logit_rnk_sig.
  groupby('Gene').
  agg('mean').
  reset_index().
  sort_values('Statistic',ascending=False))
print(df_sig.shape)
(df_sig.
 to_csv('../../data/'+cohort+
        '_bootstrap_conditional_protein_logit_'+stat+
        '_gene_name_significant_statistic.rnk',
        sep='\t',header=None,index=None))
df_sig.Gene = df_sig.Gene.apply(lambda x : x.split(';')[0])

In [ ]:
tmp = df.copy()
tmp['Significant'] = (tmp.Gene.isin(df_sig.Gene)).values
(tmp.
 to_csv('../../data/Gene_GSEA_Statistic.csv',sep=',',index=False))
(df.
 Gene.
 to_csv('../../data/gene_list.txt',sep='\t',index=False))
(df_no_IGs.
 Gene.
 to_csv('../../data/gene_list_no_IGs.txt',sep='\t',index=False))

In [ ]:
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
pre_ress = {}
cohort='integrated'
rnk = pd.read_table('../../data/'+cohort+
                    '_bootstrap_conditional_protein_logit_'+stat+
                    '_gene_name_statistic.rnk', header=None)
rnk.iloc[:,0] = rnk.iloc[:,0].apply(lambda x : x.split(';')[0])
display(rnk.head())
for g in gs:
    print('\t'+g)
    pre_res = gp.prerank(rnk=rnk, gene_sets=g,
                     processes=4,
                     permutation_num=10000, 
                     outdir='../../data/'+cohort+
                         '_bootstrap_conditional_protein_logit_'+stat+
                         '_prerank_report_'+g,format='png')

In [ ]:
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
pre_ress = {}
cohort='integrated'
rnk = pd.read_table('../../data/'+cohort+
                    '_bootstrap_conditional_protein_logit_'+stat+
                    '_gene_name_statistic_no_IGs.rnk', header=None)
rnk.iloc[:,0] = rnk.iloc[:,0].apply(lambda x : x.split(';')[0])
display(rnk.head())
for g in gs:
    print('\t'+g)
    pre_res = gp.prerank(rnk=rnk, gene_sets=g,
                     processes=4,
                     permutation_num=10000, 
                     outdir='../../data/'+cohort+
                         '_bootstrap_conditional_protein_logit_'+stat+
                         '_prerank_report_'+g+'_no_IGs',format='png')

#### Enriched and Depleted pathways

https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=7068ee737a4b433316e95d85e9326697

In [ ]:
stat = 'mean'
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
cohort='integrated'
datas=[]
for path in gs:
    data = (pd.read_csv('../../data/'+cohort+
                        '_bootstrap_conditional_protein_logit_'+stat+
                       '_prerank_report_'+path+
                       '/gseapy.prerank.gene_sets.report.csv').
            sort_values(['fdr','nes'],
                        ascending=[True,False]))
    data['Category'] = path
    datas.append(data)
pd.concat(datas).to_csv('../../data/'+cohort+
                        '_bootstrap_conditional_protein_logit_'+stat+
                       '_prerank_report_all_categories.csv')
pd.concat(datas).shape

In [ ]:
col_map = { 'nes' : 'Normalized Enrichment Score', 'pval' : 'P-value', 'fdr' : 'False Discovery Rate',"Category" : 'Category'}
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
datas = []
for path in gs:
    cohort="integrated"
    data_ = (pd.read_csv('../../data/'+cohort+
                          '_bootstrap_conditional_protein_logit_'+stat+
                          '_prerank_report_all_categories.csv',index_col=0).
            query('Category==@path').
             sort_values('fdr',ascending=False).
             set_index('Term'))
    data_ = data_.query('fdr < 0.2 & nes > 0 & fdr>pval').rename(columns=col_map)
    datas.append(data_)
enriched = pd.concat(datas).copy()
tmp = enriched[[k for k in col_map.values()]].sort_values('False Discovery Rate',ascending=True).round(4)
tmp.to_csv(dropbox_data+'enriched_PGD_pathways_functions.csv')
enriched.to_csv(dropbox_data+'enriched_PGD_pathways_functions_wgenes.csv')
display(tmp)
print(tmp.shape[0])

In [ ]:
enriched

In [ ]:
enriched_genes = np.unique(np.concatenate([k for k in pd.concat(datas).genes.str.split(';')]))
cohort = 'integrated'
stat= 'mean'
rnk = pd.read_csv('../../data/'+cohort+
                            '_bootstrap_conditional_protein_logit_'+stat+
                            '_gene_name_statistic.rnk', sep='\t',header=None)
rnk['In_Enriched_Pathway'] = rnk.loc[:,0].apply(lambda x : x.split(';')[0]).isin(enriched_genes)
rnk = rnk.rename(columns={0: 'Gene',1 : 'Statistic'})
#rnk

In [ ]:
col_map = { 'nes' : 'Normalized Enrichment Score', 'pval' : 'P-value', 'fdr' : 'False Discovery Rate',"Category" : 'Category'}
datas = []
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
for path in gs:
    cohort="integrated"
    data_ = (pd.read_csv('../../data/'+cohort+
                          '_bootstrap_conditional_protein_logit_'+stat+
                          '_prerank_report_all_categories.csv',index_col=0).
            query('Category==@path').
             sort_values('fdr',ascending=False).
             set_index('Term'))
    data_ = data_.query('fdr < 0.2 & nes < 0 & fdr>pval').rename(columns=col_map)
    datas.append(data_)
depleted = pd.concat(datas).copy()
tmp = depleted[[k for k in col_map.values()]].sort_values('False Discovery Rate',ascending=True).round(4)
tmp.to_csv(dropbox_data+'depleted_PGD_pathways_functions.csv')
depleted.to_csv(dropbox_data+'depleted_PGD_pathways_functions_wgenes.csv')
display(tmp)
print(tmp.shape[0])

In [ ]:
depleted

In [ ]:
depleted.genes.apply(lambda x : 'ADIPOQ' in x)

#### Enriched and Depleted pathways - No IGs

https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=7068ee737a4b433316e95d85e9326697

In [ ]:
stat = 'mean'
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
cohort='integrated'
datas=[]
for path in gs:
    data = (pd.read_csv('../../data/'+cohort+
                        '_bootstrap_conditional_protein_logit_'+stat+
                       '_prerank_report_'+path+'_no_IGs'+
                       '/gseapy.prerank.gene_sets.report.csv').
            sort_values(['fdr','nes'],
                        ascending=[True,False]))
    data['Category'] = path
    datas.append(data)
pd.concat(datas).to_csv('../../data/'+cohort+
                        '_bootstrap_conditional_protein_logit_'+stat+
                       '_prerank_report_all_categories_no_IGs.csv')
pd.concat(datas).shape

In [ ]:
col_map = { 'nes' : 'Normalized Enrichment Score', 'pval' : 'P-value', 'fdr' : 'False Discovery Rate',"Category" : 'Category'}
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
datas = []
for path in gs:
    cohort="integrated"
    data_ = (pd.read_csv('../../data/'+cohort+
                          '_bootstrap_conditional_protein_logit_'+stat+
                          '_prerank_report_all_categories_no_IGs.csv',index_col=0).
            query('Category==@path').
             sort_values('fdr',ascending=False).
             set_index('Term'))
    data_ = data_.query('fdr < 0.2 & nes > 0 & fdr>pval').rename(columns=col_map)
    datas.append(data_)
enriched = pd.concat(datas).copy()
tmp = enriched[[k for k in col_map.values()]].sort_values('False Discovery Rate',ascending=True).round(4)
tmp.to_csv(dropbox_data+'enriched_PGD_pathways_functions_no_IGs.csv')
enriched.to_csv(dropbox_data+'enriched_PGD_pathways_functions_wgenes_no_IGs.csv')
display(tmp)
print(tmp.shape[0])

In [ ]:
enriched

In [ ]:
enriched_genes = np.unique(np.concatenate([k for k in pd.concat(datas).genes.str.split(';')]))
cohort = 'integrated'
stat= 'mean'
rnk = pd.read_csv('../../data/'+cohort+
                            '_bootstrap_conditional_protein_logit_'+stat+
                            '_gene_name_statistic_no_IGs.rnk', sep='\t',header=None)
rnk['In_Enriched_Pathway'] = rnk.loc[:,0].apply(lambda x : x.split(';')[0]).isin(enriched_genes)
rnk = rnk.rename(columns={0: 'Gene',1 : 'Statistic'})
#rnk

In [ ]:
col_map = { 'nes' : 'Normalized Enrichment Score', 'pval' : 'P-value', 'fdr' : 'False Discovery Rate',"Category" : 'Category'}
datas = []
gs = ['GO_Biological_Process_2017b','GO_Molecular_Function_2017b',
      'GO_Cellular_Component_2017b','Reactome_2016','WikiPathways_2019_Human',
      'KEGG_2019_Human']
for path in gs:
    cohort="integrated"
    data_ = (pd.read_csv('../../data/'+cohort+
                          '_bootstrap_conditional_protein_logit_'+stat+
                          '_prerank_report_all_categories_no_IGs.csv',index_col=0).
            query('Category==@path').
             sort_values('fdr',ascending=False).
             set_index('Term'))
    data_ = data_.query('fdr < 0.2 & nes < 0 & fdr>pval').rename(columns=col_map)
    datas.append(data_)
depleted = pd.concat(datas).copy()
tmp = depleted[[k for k in col_map.values()]].sort_values('False Discovery Rate',ascending=True).round(4)
tmp.to_csv(dropbox_data+'depleted_PGD_pathways_functions_no_IGs.csv')
depleted.to_csv(dropbox_data+'depleted_PGD_pathways_functions_wgenes_no_IGs.csv')
display(tmp)
print(tmp.shape[0])

In [ ]:
depleted

In [ ]:
depleted.genes.apply(lambda x : 'ADIPOQ' in x)

#### Protein associations

In [ ]:
top_depleted_genes = np.unique(np.concatenate([k for k in pd.concat(datas).genes.str.split(';')]))
rnk['In_Top_Depleted_Pathway'] = rnk.loc[:,'Gene'].apply(lambda x : x.split(';')[0]).isin(top_depleted_genes)
rnk

In [ ]:
rnk.query('In_Enriched_Pathway==False & In_Top_Depleted_Pathway==True')

In [ ]:
logit = pd.read_csv(dir_+
                    "/bootstrap_conditional_protein_logit/"+
                    cohort+
                    "/logit_bootstrap_pgd_~_protein_+_cohort_-_paris_lwr_mean_median_upr.csv")

In [ ]:
tmp = logit.set_index('variable').join(idmap_sub.set_index('Protein'))
leftover_inds = tmp.Gene_name.isnull()
leftover_prots = tmp.index[leftover_inds].values
leftover_prots_split = [k.split('-')[0] for k in leftover_prots]

tmp_df = pd.DataFrame({'Protein' : leftover_prots,
                       'Split' : leftover_prots_split,
                       'cohort_identified_in' : cohort})

tmp_df_join = tmp_df.set_index('Split').join(idmap_sub.set_index('Protein'))

join_genes = tmp_df_join.Gene_name.values
join_prots = tmp_df_join.Protein.values

tmp.at[join_prots,'Gene_name'] = join_genes

null_prots = tmp_df_join[tmp_df_join.Gene_name.isnull()].index.values
df = tmp[~tmp.index.isin(null_prots)].reset_index(drop=True).set_index('Gene_name')

In [ ]:
genes_in_gene_set = []
genes = df[~df.index.str.startswith('IG')].reset_index().Gene_name.apply(lambda x : x.split(';')[0]).values
term_genes = np.union1d(depleted.genes,enriched.genes)
terms = np.union1d(enriched.index,depleted.index)
gene_sets = [x.split(';') for x in term_genes]
for i,set_ in enumerate(gene_sets):
    t = list(np.intersect1d(genes,set_))
    t.sort()
    genes_in_gene_set.append(t)
sig_genes = np.concatenate(genes_in_gene_set)
sig_genes

In [ ]:
stat = 'mean'

In [ ]:
fig,ax = plt.subplots(dpi=dpi,figsize=(5,5))
data = (df[~df.index.str.startswith('IG')].
        query('lwr>1 | upr<1').
        sort_values(stat,ascending=False))
display(data.head())
data.index = [x.split(';')[0][:len(x.split(';')[0])-1]+' family' if len(x.split(';'))>2 else x for x in data.index]
data.index = [x+'*' if (x in sig_genes) else x for x in data.index]
ax.errorbar(y=data.index,
            x=data[stat],
            xerr=(data[stat] - data['lwr'],
                 data['upr'] - data[stat]),
           fmt='o',markersize=3,linewidth=1)
ax.plot([1,1],[0,len(data.index.unique())-1],'r--',linewidth=0.5)
ax.set_xlabel('Odds',fontsize=16)
fig.tight_layout()
fig.savefig(dropbox_figures+'significant_proteins.pdf')

#### Centered Bar plot of GSEA Enriched/Depleted pathways

In [ ]:
enriched = pd.read_csv(dropbox_data+'enriched_PGD_pathways_functions_wgenes.csv')
depleted = pd.read_csv(dropbox_data+'depleted_PGD_pathways_functions_wgenes.csv')

In [ ]:
enriched.Term = enriched.Term.str.split('[(_]').apply(pd.Series).iloc[:,0].copy()
depleted.Term = depleted.Term.str.split('[(_]').apply(pd.Series).iloc[:,0].copy()

In [ ]:
data = pd.concat([enriched,depleted]).sort_values('Normalized Enrichment Score',ascending=False)
data.Term = data.Term.str.replace('WP545','').str.replace(' WP15','')
data.head()

In [ ]:
from matplotlib import ticker

fig,ax=plt.subplots(dpi=dpi,figsize=(10,5))
xlab = 'Normalized Enrichment Score'
sns.barplot(xlab,'Term',data=data,ax=ax,color='darkgray',linewidth=.2)
ax.set_ylabel('')
ax.set_xlabel(xlab,size=16)
ax.set_xscale('symlog')

formatter = ticker.ScalarFormatter(useOffset=True,)
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(ticker.FixedLocator([-40,-10,0,1,2,3]))
ax.grid(b=True, which='major', color='gray', linewidth=0.1)
ax.tick_params(labelsize=18)

fig.tight_layout()
fig.savefig(dropbox_figures+'GSEA_Enriched_Depleted_BarPlot.png')

### assigning proteins to their GSEA category

In [ ]:
uniprot = pd.read_csv('../../data/uniprot-all_20171124.tab.gz',sep='\t')
raw_samples = (pd.read_csv('../../data/integrated_sample_data_mean_std_scaled.csv',
                          index_col=0))
df = raw_samples.join(uniprot.loc[:,['Entry','Gene ontology (biological process)']].set_index('Entry')).dropna()
df.loc[:,'Gene ontology (biological process)'] = df.loc[:,'Gene ontology (biological process)'].apply(lambda x : x.split(';'))
df.index.name = 'Protein'
df=df.reset_index()
lens = [len(item) for item in df['Gene ontology (biological process)']]
df_out = pd.DataFrame( {"Protein" : np.repeat(df['Protein'].values,lens), 
               "GO_Biological_Process" : np.hstack(df['Gene ontology (biological process)'])
              })
df_out
df_join=(df.
         set_index('Protein').
         join(df_out.
              set_index('Protein')
             ).
         set_index('GO_Biological_Process').
         drop('Gene ontology (biological process)',axis=1)
        )
X = df_join.groupby(df_join.index).agg(np.mean).T
X

In [ ]:
riched = (pd.
          read_csv('../../data/integrated_bootstrap_conditional_protein_logit'+
                   '_mean_prerank_report_all_categories.csv',index_col=0)
         )
display(riched.head())


In [ ]:
def tall_func(riched):
    df = riched.copy().drop('genes',axis=1)
    df.ledge_genes = df.ledge_genes.apply(lambda x : x.split(';'))
    return (df.
            ledge_genes.
            apply(pd.Series).
            merge(df, 
                  right_index = True, 
                  left_index = True).
            drop(['ledge_genes'],axis=1).
            melt(id_vars=np.setdiff1d(riched.columns,['genes','ledge_genes']),
                 value_name='ledge_gene').
            drop('variable',axis=1).
            dropna().
            reset_index(drop=True).
            sort_values(['Category','Term','ledge_gene'])
    )

In [ ]:
riched_tall = tall_func(riched)
display(riched_tall.head())
riched_tall.to_csv('../../data/integrated_bootstrap_conditional_protein_logit'+
                   '_mean_prerank_report_all_categories_tall.csv')

In [ ]:
riched_tall.ledge_gene.unique()

### GSEA category protein predictions

In [ ]:
data_dir = '../../data/integrated_pgd_predictions/gsea_categories/'
type_='ledge_protein_features_pgd_prediction_'
proteins_immunoglobulins = pickle.load(open('../../data/proteins_immunoglobulins.pkl','rb'))
scorers = { 'roc_auc' : roc_auc_score}

##### performances

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' in x) &
                                             ('proteins_prediction_metric' in x) &
                                             ('slash' not in x)
                                            )
        ]

In [ ]:
print(len(files))
files[:5]

In [ ]:
n=50
lsts=[]
feature_mccv_scores_df = {}
feature_mccv_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_proteins_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',
                           '')
                  )
        dat = pd.read_csv(data_dir+file,index_col=0)
        vals = []
        for b in range(n):
            x = (dat.
                 sample(n=dat.shape[0],replace=True,random_state=b)
                )
            vals.append([feature,b,x.model.unique()[0],scorer(x.y_true,x.y_proba)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Pathway','Bootstrap',
                                                              'Model',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_scores_df[score] = \
    (pd.concat(feature_scores_bootstraps)
    )

    feature_mccv_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Pathway','Model'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}))
    (pd.concat(feature_scores_bootstraps).
     groupby(['Pathway','Model'])[score].
     describe(percentiles=[0.025,0.975]).
     loc[:,['2.5%','mean','97.5%']].
     sort_values('mean',ascending=False)
    ).to_csv(data_dir+type_+score+'_CIs.csv')

    display(feature_mccv_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_score_means_dfs.append(feature_mccv_score_means_df)
feature_mccv_score_means_df = (reduce(lambda  left,right: pd.merge(left,right,
                                                                  on=['Pathway','Model'],
                                            how='outer'), feature_mccv_score_means_dfs))
print(feature_mccv_score_means_df.shape)
feature_mccv_score_means_df.head()

##### importance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' not in x) & 
                                             ('importance' in x) & 
                                             ('bootstrap' in x) &
                                             ('proteins_prediction_metric' in x) &
                                             ('slash' not in x)
                                            )
        ]

In [ ]:
files[:5]

In [ ]:
lsts=[]
for file in files:
    feature = (file.
               replace(type_,'').
               replace('_proteins_prediction_metric_bootstrap_train_test_val'+
                       '_feature_importances.csv',''))
    feature_logit_df = (pd.
                    read_csv(data_dir+file,index_col=0).
                    rename(columns={'bootstrap' : 'Bootstrap','model' : 'Model'}).
                    dropna())
    feature_logit_df['Pathway'] = feature
    lsts.append(feature_logit_df)

In [ ]:
feature_mccv_importance_odds_df = pd.concat(lsts)
feature_mccv_importance_odds_df['odds'] = np.exp(feature_mccv_importance_odds_df['Importance'])
feature_mccv_odds_df = (feature_mccv_importance_odds_df.
                        groupby(['Gene_name','Model','Pathway'])['odds'].
                        describe(percentiles=[0.025,0.975]).
                        loc[:,['2.5%','mean','97.5%']].
                        rename(columns={'2.5%' : 'odds_lwr',
                                        'mean' : 'odds_mean',
                                        '97.5%' : 'odds_upr'}
                              ).
                        reset_index().
                        reset_index(drop=True)
                       )

In [ ]:
print(feature_mccv_odds_df.query('odds_lwr>1 | odds_upr<1').shape)
feature_mccv_odds_df.query('odds_lwr>1 | odds_upr<1').head()

In [ ]:
(feature_mccv_importance_odds_df.
 query('Gene_name=="TPM4"').
 pivot_table(index='Pathway',columns='Bootstrap',values='Importance')
)

In [ ]:
feature_mccv_odds_df.pivot_table(index='Gene_name',columns='Pathway',values='odds_mean')

##### permuted performances

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' in x) & 
                                             ('importance' not in x) & 
                                             ('bootstrap' not in x) &
                                             ('proteins_prediction_metric' in x) &
                                             ('slash' not in x)
                                            )
        ]

In [ ]:
print(len(files))
files[:5]

In [ ]:
n=50
lsts=[]
feature_mccv_permuted_scores_df = {}
feature_mccv_permuted_score_means_dfs = []
for score,scorer in scorers.items():
    feature_scores_bootstraps = []
    for file in files:
        feature = (file.
                   replace(type_,'').
                   replace('_proteins_prediction_metric_permute_train_test_val_patient_level_data.csv',
                           '')
                  )
        dat = pd.read_csv(data_dir+file,index_col=0)
        vals = []
        for b in range(n):
            x = (dat.
                 sample(n=dat.shape[0],replace=True,random_state=b)
                )
            vals.append([feature,b,x.model.unique()[0],scorer(x.y_true,x.y_proba)])
        feature_scores_bootstrap = pd.DataFrame(vals,columns=['Pathway','Bootstrap',
                                                              'Model',score])
        feature_scores_bootstraps.append(feature_scores_bootstrap)
    
    feature_mccv_permuted_scores_df[score] = \
    (pd.concat(feature_scores_bootstraps)
    )

    feature_mccv_permuted_score_means_df = (pd.concat(feature_scores_bootstraps).
                                   groupby(['Pathway','Model'])[score].
                                   mean().
                                   reset_index().
                                   rename(columns={score : 'mean_validation_'+score}))
    (pd.concat(feature_scores_bootstraps).
     groupby(['Pathway','Model'])[score].
     describe(percentiles=[0.025,0.975]).
     loc[:,['2.5%','mean','97.5%']].
     sort_values('mean',ascending=False)
    ).to_csv(data_dir+type_+score+'_CIs.csv')

    display(feature_mccv_permuted_score_means_df.sort_values('mean_validation_'+score).tail())

    feature_mccv_permuted_score_means_dfs.append(feature_mccv_permuted_score_means_df)
feature_mccv_permuted_score_means_df = (reduce(lambda  left,right: pd.merge(left,right,
                                                                  on=['Pathway','Model'],
                                            how='outer'), feature_mccv_permuted_score_means_dfs))
print(feature_mccv_permuted_score_means_df.shape)
feature_mccv_permuted_score_means_df.head()

##### permuted importance

In [ ]:
files = [x for x in os.listdir(data_dir) if ( ('pkl' not in x) & 
                                             (type_ in x) & 
                                             ('patient' not in x) & 
                                             ('importance' in x) & 
                                             ('bootstrap' not in x) &
                                             ('proteins_prediction_metric' in x) &
                                             ('slash' not in x)
                                            )
        ]

In [ ]:
files[:5]

In [ ]:
lsts=[]
for file in files:
    feature = (file.
               replace(type_,'').
               replace('_proteins_prediction_metric_permute_train_test_val'+
                       '_feature_importances.csv',''))
    feature_logit_df = (pd.
                    read_csv(data_dir+file,index_col=0).
                    rename(columns={'bootstrap' : 'Bootstrap','model' : 'Model'}).
                    dropna())
    feature_logit_df['Pathway'] = feature
    lsts.append(feature_logit_df)

In [ ]:
feature_mccv_permuted_importance_odds_df = pd.concat(lsts)
feature_mccv_permuted_importance_odds_df['odds'] = np.exp(feature_mccv_permuted_importance_odds_df['Importance'])
feature_mccv_permuted_odds_df = (feature_mccv_permuted_importance_odds_df.
                        groupby(['Gene_name','Model','Pathway'])['odds'].
                        describe(percentiles=[0.025,0.975]).
                        loc[:,['2.5%','mean','97.5%']].
                        rename(columns={'2.5%' : 'permuted_odds_lwr',
                                        'mean' : 'permuted_odds_mean',
                                        '97.5%' : 'permuted_odds_upr'}
                              ).
                        reset_index().
                        reset_index(drop=True)
                       )

In [ ]:
print(feature_mccv_permuted_odds_df.query('permuted_odds_lwr>1 | permuted_odds_upr<1').shape)
feature_mccv_permuted_odds_df.query('permuted_odds_lwr>1 | permuted_odds_upr<1').head()

##### significant performance

In [ ]:
score = 'roc_auc'
pathways = feature_mccv_permuted_scores_df[score].Pathway.unique()
ms = feature_mccv_permuted_scores_df[score].Model.unique()


pvals = []
for p in pathways:
    for m in ms:
        bdist = feature_mccv_scores_df[score].query('Model==@m & Pathway==@p')[score].values
        pdist = feature_mccv_permuted_scores_df[score].query('Model==@m & Pathway==@p')[score].values
        t,pval = ks_2samp(pdist,bdist)
        pvals.append([p,m,t,pval])

In [ ]:
feature_mccv_performance_significance = pd.DataFrame(pvals,
                                                     columns=
                                                     ['Pathway',
                                                      'Model',
                                                      'Performance_Statistic',
                                                      'Performance_P_value']
                                                    )

feature_mccv_performance_significance['Performance_bonferroni'] = \
multipletests(feature_mccv_performance_significance.Performance_P_value.values,
              method='bonferroni')[1]

In [ ]:
feature_mccv_performance_significance.head()

##### significant importance

In [ ]:
score = 'roc_auc'
pathways = feature_mccv_importance_odds_df.Pathway.unique()
print(len(pathways))
genes = feature_mccv_permuted_importance_odds_df.Gene_name.unique()

pvals = []
for m in ms:
    for p in pathways:
        print(p)
        genes = (feature_mccv_permuted_importance_odds_df.
                 query('Pathway==@p').
                 Gene_name.unique()
                )
        print(len(genes))
        for g in genes:
            bdist = feature_mccv_importance_odds_df.query('Model==@m & Gene_name==@g & Pathway==@p')['Importance'].values
            pdist = feature_mccv_permuted_importance_odds_df.query('Model==@m & Gene_name==@g & Pathway==@p')['Importance'].values
            if len(bdist)>0:
                t,pval = ks_2samp(pdist,bdist)
                pvals.append([p,g,m,t,pval])

In [ ]:
feature_mccv_importance_significance = pd.DataFrame(pvals,columns=['Pathway','Gene_name','Model','Importance_Statistic','Importance_P_value'])

feature_mccv_importance_significance['Importance_bonferroni'] = multipletests(feature_mccv_importance_significance.Importance_P_value.values,method='bonferroni')[1]

In [ ]:
feature_mccv_importance_significance.head()

##### joining

In [ ]:
pathways = (feature_mccv_score_means_df.
 set_index(['Pathway','Model']).
 join(feature_mccv_performance_significance.
     set_index(['Pathway','Model']))
)
display(pathways.head())
genes = (feature_mccv_odds_df.
 set_index(['Pathway','Model','Gene_name']).
 join(feature_mccv_permuted_odds_df.
      set_index(['Pathway','Model','Gene_name'])
     ).
 join(feature_mccv_importance_significance.
     set_index(['Pathway','Model','Gene_name']))
)
display(genes.head())
joined = genes.join(pathways)
print(joined.shape)
joined.head()

##### outputting

In [ ]:
joined.reset_index().to_csv('../../data/gsea_categories_proteins_performance_significance_odds.csv')

#### joining with gsea statistics

In [ ]:
gsea=(pd.
      read_csv('../../data/integrated_bootstrap_conditional_'+
               'protein_logit_mean_prerank_report_all_categories.csv',index_col=0)
     )
print(gsea.shape)
display(gsea.head())
gsea_predictions = (pd.
                   read_csv('../../data/gsea_categories_proteins_'+
                            'performance_significance_odds.csv',index_col=0))
print(gsea_predictions.shape)
display(gsea_predictions.head())

In [ ]:
gsea_statistics_predictions = (gsea.
set_index(['Term']).
join(gsea_predictions.
    set_index(['Pathway'])
    )
)
gsea_statistics_predictions

#### Analysis and plots

##### Intersection of individually predictive proteins and predictive GSEA proteins

In [ ]:
ind_protein_preds = pd.read_csv('../../data/protein_raw_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',index_col=0)

In [ ]:
query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'importance_bonferroni<0.001 & (importance_bonferroni>=importance_p_value)'
ind_predictive_genes = (ind_protein_preds.
 query(query).
 Gene_name.
 unique()
)
print(len(ind_predictive_genes))
ind_predictive_genes

In [ ]:
gsea_statistics_predictions.Gene_name.nunique()

In [ ]:
query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'Importance_bonferroni<0.001 & (Importance_bonferroni>=Importance_P_value)'
(gsea_statistics_predictions.
query(query).
 rename_axis('Pathway').
 reset_index().
 loc[:,['Pathway','Gene_name',
        'mean_validation_roc_auc','Performance_bonferroni',
        'odds_mean','Importance_bonferroni']]
).to_csv(dropbox_data+'Significantly_predictive_proteins_within_pathways.csv')
gsea_ind_predictive_genes = \
(gsea_statistics_predictions.
query(query).
 Gene_name.
 unique()
)
print(len(gsea_ind_predictive_genes))
gsea_ind_predictive_genes

In [ ]:
gsea_and_ind_predictive_proteins = np.intersect1d(gsea_ind_predictive_genes,
                                                  ind_predictive_genes)
print(gsea_and_ind_predictive_proteins)
print(len(gsea_and_ind_predictive_proteins))

In [ ]:
np.setdiff1d(gsea_ind_predictive_genes,ind_predictive_genes)

In [ ]:
np.setdiff1d(ind_predictive_genes,gsea_ind_predictive_genes)

##### Odds distribution of proteins in different categories

In [ ]:
edpathways = (gsea.
 query('fdr<.2 & fdr>pval').
 Term.
 unique()
)
edpathways

In [ ]:
data = (gsea_statistics_predictions.
 query('Gene_name in @gsea_and_ind_predictive_proteins').
 rename_axis('Pathway').
 reset_index().
 loc[:,['Gene_name','Pathway','odds_mean','Category']].
 drop_duplicates()
)
display(data.head())
data = data[data.Pathway.isin(edpathways)]

In [ ]:
data['odds_mean'] = np.log(data['odds_mean'])

In [ ]:
display(data.sort_values('odds_mean',ascending=False).head(20))
print(data.sort_values('odds_mean',ascending=False).head(20).Pathway.values)
display(data.sort_values('odds_mean',ascending=False).tail(20))

In [ ]:
fig,ax = plt.subplots(dpi=dpi,figsize=(25,14))
sns.stripplot(x='odds_mean',y='Gene_name',hue='Pathway',
              data=data,ax=ax,
             jitter=True, linewidth=2,size=22,
             dodge=True,palette='bright')

yticks = [x for x in ax.get_yticklabels()]
for i,ytick in enumerate(yticks):
    ax.axhline(i,c='gray',alpha=.5,ls='--')
    
plt.legend(prop={'size': 16},frameon=False)
ax.axvline(0,c='red',ls='--')
ax.tick_params(labelsize=20)
ax.set_ylabel('')
ax.set_xlabel(r'Within-Pathway PGD Prediction $\beta$ Coefficient',size=24)
fig.savefig(dropbox_figures+'gsea_pathway_proteins_significant_predictions.png')

##### Heatmap of significantly predictive Genes by Pathway colored by odds*

In [ ]:
data = (gsea_statistics_predictions.
 query('Gene_name in @gsea_ind_predictive_genes').
 rename_axis('Pathway').
 reset_index().
 loc[:,['Gene_name','Pathway','odds_mean','Category']].
 drop_duplicates().
        pivot_table(index='Pathway',columns='Gene_name',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
fig,ax=plt.subplots(dpi=dpi,figsize=(15,30))
sns.heatmap(data,ax=ax,cmap='seismic',linewidth=.1,linecolor='black',center=0)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,ha="center")
fig.savefig(
    dropbox_figures+'Pathway_by_significantly_predicting_gsea_proteins_heatmap.png', 
    bbox_inches='tight')

##### Heatmap of significantly predictive Genes by Pathway colored by odds*

In [ ]:
data = (gsea_statistics_predictions.
 query('Gene_name in @gsea_and_ind_predictive_proteins').
 rename_axis('Pathway').
 reset_index().
 loc[:,['Gene_name','Pathway','odds_mean','Category']].
 drop_duplicates().
        pivot_table(index='Pathway',columns='Gene_name',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
fig,ax=plt.subplots(dpi=dpi,figsize=(15,30))
sns.heatmap(data,ax=ax,cmap='seismic',linewidth=.1,linecolor='black',center=0)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,ha="center")
fig.savefig(
    dropbox_figures+'Pathway_by_significantly_predicting_proteins_heatmap.png', 
    bbox_inches='tight')

In [ ]:
paths = gsea.query('fdr<0.2 & fdr>pval').sort_values('nes',ascending=False).Term.unique()
display((gsea_statistics_predictions.
        query('Gene_name in @gsea_ind_predictive_genes').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @paths').
        loc[:,['Gene_name','Pathway','nes','Category']].
        drop_duplicates()))
data = (gsea_statistics_predictions.
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Pathway',columns='Gene_name',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
paths = gsea.sort_values('nes',ascending=False).Term.unique()
sig_paths = gsea.query('fdr<0.2 & fdr>pval').sort_values('nes',ascending=False).Term.unique()
gsea_sig_data_sig_paths = (gsea_statistics_predictions.
        query('Gene_name in @gsea_ind_predictive_genes').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @sig_paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
gsea_sig_and_sig_data_sig_paths = (gsea_statistics_predictions.
        query('Gene_name in @gsea_and_ind_predictive_proteins').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @sig_paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
gsea_sig_and_sig_data_paths = (gsea_statistics_predictions.
        query('Gene_name in @gsea_and_ind_predictive_proteins').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
gsea_sig_data_paths = (gsea_statistics_predictions.
        query('Gene_name in @gsea_ind_predictive_genes').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
sig_data_paths = (gsea_statistics_predictions.
        query('Gene_name in @ind_predictive_genes').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
sig_data_sig_paths = (gsea_statistics_predictions.
        query('Gene_name in @ind_predictive_genes').
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @sig_paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
data_sig_paths = (gsea_statistics_predictions.
        rename_axis('Pathway').
        reset_index().
        query('Pathway in @sig_paths').
        loc[:,['Gene_name','Pathway','odds_mean','Category']].
        drop_duplicates().
        pivot_table(index='Gene_name',columns='Pathway',values='odds_mean').
        applymap(lambda x : np.log(x))
)

In [ ]:
import scipy.cluster.hierarchy as h

In [ ]:
cols = pd.Series(gsea_sig_and_sig_data_sig_paths.columns.values).str.split('[(_]').apply(pd.Series).iloc[:,0].copy()


In [ ]:
cols = cols.str.replace(' WP545','').str.replace(' WP15','')

In [ ]:
g = sns.clustermap(gsea_sig_and_sig_data_sig_paths.fillna(0).T,
                   cmap='RdBu_r',linewidth=.1,linecolor='black',center=0,
                   row_cluster=False,
                   figsize=(10,10),
                  cbar_kws={
                      "ticks": np.arange(-1,3,.5)})
g.fig.dpi=dpi
g.ax_heatmap.set_xlabel('')
g.ax_heatmap.set_ylabel('')
g.ax_heatmap.set_yticks(g.ax_heatmap.get_yticks())
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xticklabels(),fontsize=20,rotation=0)
g.ax_heatmap.set_yticklabels(cols,
                             rotation=0,ha='left',fontsize=20)
plt.savefig(
    dropbox_figures+'Significant_pathway_by_significantly_predicting_proteins_heatmap.png', 
    bbox_inches='tight')

##### Fisher test of enrichment of significantly predictive proteins in enriched/depleted GSEA pathways

In [ ]:
ind_protein_preds = pd.read_csv('../../data/protein_raw_01_within_notwithcohorts_mccv_performance_significance_and_feature_odds_df.csv',index_col=0)

In [ ]:
ind_all_genes = (ind_protein_preds.
 Gene_name.
 unique()
)
print(len(ind_all_genes))
query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'importance_bonferroni<0.001 & (importance_bonferroni>=importance_p_value)'
ind_predictive_genes = (ind_protein_preds.
 query(query).
 Gene_name.
 dropna().
 unique()
)
print(len(ind_predictive_genes))
ind_predictive_genes

In [ ]:
gsea_ind_all_genes = \
(gsea_statistics_predictions.
 Gene_name.
 dropna().
 unique()
)
print(len(gsea_ind_all_genes))
query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'Importance_bonferroni<0.001 & (Importance_bonferroni>=Importance_P_value)'
gsea_ind_predictive_genes = \
(gsea_statistics_predictions.
query(query).
 Gene_name.
 unique()
)
print(len(gsea_ind_predictive_genes))
gsea_ind_predictive_genes

In [ ]:
gsea_and_ind_predictive_proteins = np.intersect1d(gsea_ind_predictive_genes,ind_predictive_genes)
print(gsea_and_ind_predictive_proteins)
print(len(gsea_and_ind_predictive_proteins))

In [ ]:
gsea_ind_not_predictive_genes = np.setdiff1d(gsea_ind_all_genes,gsea_ind_predictive_genes)
ind_not_predictive_genes = np.setdiff1d(ind_all_genes,ind_predictive_genes)
print(len(gsea_ind_not_predictive_genes))
print(len(ind_not_predictive_genes))

In [ ]:
a = len(np.intersect1d(ind_predictive_genes,gsea_ind_predictive_genes))
print(a)
b = len(np.setdiff1d(ind_predictive_genes,gsea_ind_not_predictive_genes))
print(b)
c = len(np.setdiff1d(ind_not_predictive_genes,gsea_ind_predictive_genes))
print(c)
d = len(np.union1d(ind_not_predictive_genes,gsea_ind_not_predictive_genes))
print(d)

In [ ]:
import scipy.stats as stats

In [ ]:
oddsratio, pvalue = stats.fisher_exact([[a,b],[c,d]])
print(oddsratio)
print(pvalue)

### Two marker panel bootstrap validation performance

In [ ]:
std_name='01_within_notwithcohorts'
basename = '../../data/integrated_pgd_predictions/'+\
'raw_'+std_name+'_clinicalclinical_proteinclinical_proteinprotein_and_clinical_and_protein_features_small_combos_pgd_prediction_'

perf_df = pd.read_csv(basename+'agg_performance.csv',index_col=0).query('set in @sets_to_use')
fimps_df = (pd.
            read_csv(basename+'agg_feature_importances.csv',
                     index_col=0).
            query('Feature!="Intercept"').
            query('Feature not in @features_not_to_see').
            query('set in @sets_to_use')
           )

query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'importance_bonferroni<0.001 & (importance_bonferroni>=importance_p_value)'
predictive_proteins =  \
(pd.
 read_csv('../../data/protein_raw_'+std_name+'_mccv_performance_significance_and_feature_odds_df.csv',
          index_col=0).
 query(query).
 feature.
 unique()
)
predictive_clinicals =  \
(pd.
 read_csv('../../data/clinical_'+std_name+'_mccv_performance_significance_and_feature_odds_df.csv',
          index_col=0).
 query(query).
 feature.
 unique()
)

umarkers = np.union1d(predictive_proteins,predictive_clinicals)
feature_set = pickle.load(open(basename+'feature_set_dictionary.pkl','rb'))
sets_to_use = [k for 
 k,v in feature_set.items() if len(np.intersect1d(umarkers,v))==len(v)]
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_minus_paris_covariates.csv',index_col=0)
features_not_to_see = [x for x in X_all_clinical.columns if 'Cohort_' in x]

def get_validation_scores(set_):
    
    try:
        dat = fimps_df.query('set==@set_')
        lst = []
        fs = dat.Feature.tolist()
        lst = [fs]
        vals = perf_df.query('set==@set_').values[0]
        lst.extend(vals)
        return lst
    except:
        return []

m = []
params = {}
arrs = Parallel(backend='threading')(delayed(get_validation_scores)(set_,**params) for 
                                     set_ in sets_to_use)
tmp = pd.DataFrame(arrs,columns=['Features','set','2.5%','mean','97.5%'])

m.extend([tmp])
tmp.set = tmp.set.astype(int)
display(tmp.sort_values('mean',ascending=False).head())
(pd.concat(m).
 sort_values('mean',ascending=False).
 to_csv('../../data/marker_combo_'+std_name+'_bootstrap_validation_performance.csv')
)

   ### Performance of two marker equations of our data

In [ ]:
std_name='01_within_notwithcohorts'
basename = '../../data/integrated_pgd_predictions/'+\
'raw_'+std_name+'_clinicalclinical_proteinclinical_proteinprotein_and_clinical_and_protein_features_small_combos_pgd_prediction_'


query='mean_validation_roc_auc>0.5 &'+ \
           ' (odds_lwr>1 | odds_upr<1) & '+ \
           '(permuted_odds_lwr<1 & permuted_odds_upr>1) &'+ \
           'importance_bonferroni<0.001 & (importance_bonferroni>=importance_p_value)'
predictive_proteins =  \
(pd.
 read_csv('../../data/protein_raw_'+std_name+'_mccv_performance_significance_and_feature_odds_df.csv',
          index_col=0).
 query(query).
 feature.
 unique()
)
predictive_clinicals =  \
(pd.
 read_csv('../../data/clinical_'+std_name+'_mccv_performance_significance_and_feature_odds_df.csv',
          index_col=0).
 query(query).
 feature.
 unique()
)

umarkers = np.union1d(predictive_proteins,predictive_clinicals)

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_minus_paris_covariates.csv',index_col=0)
X_all = X_all_proteins.join(X_all_clinical)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)
y_true = Y.values.reshape(1,-1)[0]

cumc = pd.read_csv('../../data/df_samples_cumc_allsets.csv',index_col=0).columns.tolist()
cedar = pd.read_csv('../../data/df_samples_cedar_allsets.csv',index_col=0).columns.tolist()
paris = pd.read_csv('../../data/df_samples_paris_allsets.csv',index_col=0).columns.tolist()

y_true = Y.values.reshape(1,-1)[0]
y_true_cumc = Y.loc[cumc].values.reshape(1,-1)[0]
y_true_cedar = Y.loc[cedar].values.reshape(1,-1)[0]
y_true_paris = Y.loc[paris].values.reshape(1,-1)[0]

feature_set = pickle.load(open(basename+'feature_set_dictionary.pkl','rb'))
sets_to_use = [k for 
 k,v in feature_set.items() if len(np.intersect1d(umarkers,v))==len(v)]
features_not_to_see = [x for x in X_all_clinical.columns if 'Cohort_' in x]

fimps_df = (pd.
            read_csv(basename+'agg_feature_importances.csv',
                     index_col=0).
            query('Feature!="Intercept"').
            query('Feature not in @features_not_to_see').
            query('set in @sets_to_use')
           )

def predict_probability(data, weights):
    """probability predicted by the logistic regression"""
    score = np.dot(data, weights)
    predictions = 1 / (1 + np.exp(-score))
    return predictions

def get_equation_scores(set_,func,name='roc_auc'):
    
    dat = fimps_df.query('set==@set_')
    fs = dat.Feature.tolist()
    equation = dat['mean'].values
    X = X_all[fs]
    X_cumc = X.loc[cumc]
    X_cedar = X.loc[cedar]
    X_paris = X.loc[paris]
    ps = predict_probability(X,equation)
    ps_cumc = predict_probability(X_cumc,equation)
    ps_cedar = predict_probability(X_cedar,equation)
    ps_paris = predict_probability(X_paris,equation)
    return [fs,set_,name,
            np.round(func(y_true,ps),4),
            np.round(func(y_true_cumc,ps_cumc),4),
            np.round(func(y_true_cedar,ps_cedar),4),
            np.round(func(y_true_paris,ps_paris),4)]
    
m = []
params={ 'func' : roc_auc_score }
arrs = Parallel(backend='threading')(delayed(get_equation_scores)(set_,**params) for 
                                     set_ in sets_to_use)
tmp = (pd.DataFrame(arrs,columns=['Features','set','score',
                                 'integrated','cumc','cedar','paris']))
m.extend([tmp])
display(tmp.sort_values('integrated',ascending=False).head())
(pd.concat(m).
 sort_values('integrated',ascending=False).
 to_csv('../../data/marker_combo_'+std_name+'_equation_performance_on_our_data.csv')
)

### Inotrope and KLKB1 Panel Prediction

#### panel prediction

In [ ]:
basename = '../../data/integrated_pgd_predictions/'+\
'raw_01_within_notwithcohorts_clinicalclinical_proteinclinical_proteinprotein_and_clinical_and_protein_features_small_combos_pgd_prediction_'
feature_set = pickle.load(open(basename+'feature_set_dictionary.pkl','rb'))
all_pperf_df = pd.read_csv(basename+'agg_patient_level_data.csv',index_col=0).query('set in @sets_to_use')

In [ ]:
k_sets = [k for k,v in feature_set.items() if 'H0YAC1' in v]
i_sets = [k for k,v in feature_set.items() if 'Prior_Inotrope_Y' in v]
ki_set = np.intersect1d(k_sets,i_sets)[0]
ki_pperf = all_pperf_df.query('set==@ki_set')

In [ ]:
def get_pperf_CI_scores(dat,n=50,scorer=roc_auc_score,seed=seed):
    
    lsts = []
    for b in range(n):
        lsts.append(
            (dat.
             sample(n=dat.shape[0],replace=True,random_state=b).
             groupby('cohort').
             apply(
                 lambda x : scorer(x.y_true,x.y_proba)
             )
            )
        )
    cohort_df = (pd.concat(lsts,1).
                           T.
                           describe(percentiles=[0.025,0.975]).
                           loc[['2.5%','mean','97.5%']].
                T)

    vals = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        vals.append(scorer(x.y_true,x.y_proba))
    
    integrated_df = (pd.
     DataFrame(vals,
               columns=['Integrated']).
     describe(percentiles=[0.025,0.975]).
     loc[['2.5%','mean','97.5%']].T)
    return pd.concat([cohort_df,integrated_df])

In [ ]:
def compare_cohort_scores(dat1,dat2,f1,f2,n=50,scorer=roc_auc_score,stat_scorer=ks_2samp,seed=seed):
    
    lsts = []
    for b in range(n):
        lsts.append(
            (dat1.
             sample(n=dat1.shape[0],replace=True,random_state=b).
             groupby('cohort').
             apply(
                 lambda x : scorer(x.y_true,x.y_proba)
             )
            )
        )
    dat1_df = pd.concat(lsts,1).T

    lsts = []
    for b in range(n):
        lsts.append(
            (dat2.
             sample(n=dat2.shape[0],replace=True,random_state=b).
             groupby('cohort').
             apply(
                 lambda x : scorer(x.y_true,x.y_proba)
             )
            )
        )
    dat2_df = pd.concat(lsts,1).T

    lst = []
    for c1 in dat1_df:
        for c2 in dat2_df:
            if c1==c2:
                stat,p = stat_scorer(dat1_df.loc[:,c1],dat2_df.loc[:,c2])
                lst.append([c1,f1,f2,stat,p])
    return pd.DataFrame(lst,columns=['Cohort','Panel1','Panel2',
                                     'KS_Statistic','KS_pvalue'])


In [ ]:
def compare_integrated_scores(dat1,dat2,f1,f2,n=50,scorer=roc_auc_score,stat_scorer=ks_2samp,seed=seed):
    
    lsts = []
    for b in range(n):
        x = (dat1.
         sample(n=dat1.shape[0],replace=True,random_state=b)
        )
        lsts.append(roc_auc_score(x.y_true,x.y_proba))
    dat1_df = pd.DataFrame(lsts,columns=['Integrated'])

    lsts = []
    for b in range(n):
        x = (dat2.
         sample(n=dat2.shape[0],replace=True,random_state=b)
        )
        lsts.append(roc_auc_score(x.y_true,x.y_proba))
    dat2_df = pd.DataFrame(lsts,columns=['Integrated'])

    lst = []
    for c1 in dat1_df:
        for c2 in dat2_df:
            if c1==c2:
                stat,p = stat_scorer(dat1_df.loc[:,c1],dat2_df.loc[:,c2])
                lst.append([c1,f1,f2,stat,p])
    return pd.DataFrame(lst,columns=['Cohort','Panel1','Panel2',
                                     'KS_Statistic','KS_pvalue'])


In [ ]:
def get_pperf_scores(dat,n=50,scorer=roc_auc_score,seed=seed):
    
    lsts = []
    for b in range(n):
        lsts.append(
            (dat.
             sample(n=dat.shape[0],replace=True,random_state=b).
             groupby('cohort').
             apply(
                 lambda x : scorer(x.y_true,x.y_proba)
             )
            )
        )
    cohort_meaen_series = pd.concat(lsts,1).T.mean()

    vals = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        vals.append(scorer(x.y_true,x.y_proba))
    
    cedar,cumc,paris,all_ = (pd.
     concat([pd.concat(lsts,1).
             T.
             mean(),
             pd.Series(np.mean(vals),
                       index=['Integrated'])
            ]
           ).
     values)
    return [all_,cumc,cedar,paris,
            (cumc+cedar+paris)/3,(all_+cumc+cedar+paris)/4,
           np.var([cumc,cedar,paris]),np.var([all_,cumc,cedar,paris])]

In [ ]:
k_pperf = \
(pd.
 read_csv('../../data/integrated_pgd_predictions/'+
          'protein_raw_01_within_notwithcohorts_features_pgd_prediction_'+
          'H0YAC1_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',
         index_col=0)
)
i_pperf = \
(pd.
 read_csv('../../data/integrated_pgd_predictions/'+
          'clinical_01_within_notwithcohorts_features_pgd_prediction_'+
          'Prior_Inotrope_Y_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',
         index_col=0)
)

In [ ]:
(k_pperf[['bootstrap','y_proba','cohort']].
 pivot_table(index='bootstrap',columns='cohort',values='y_proba')).hist()

In [ ]:
val_boot_cohort_N = \
(pd.DataFrame(
    (k_pperf[['bootstrap','cohort']].
     groupby('bootstrap')['cohort'].
     value_counts()
    )).
 rename(columns = {'cohort' : 'N'}).
 reset_index().
 pivot_table(index='bootstrap',columns='cohort',values='N')
)
val_boot_cohort_N.hist()

#### #3, avg within bootstrap then avg across bootstrap validation probabilities

In [ ]:
fig,ax=plt.subplots(dpi=200)
scorer = roc_auc_score
scores=[]
m=[]
for i,grp in ki_pperf.groupby('bootstrap'):
    vals=grp.y_proba.values
    norm_vals = (vals - min(vals)) / (max(vals) - min(vals))
    m.append(vals)
    scores.append(scorer(grp.y_true.values,vals))
    sns.kdeplot(vals,color='blue',alpha=.1,ax=ax)
sns.kdeplot([np.median(j) for j in m],color='red',ax=ax)
plt.axvline(np.mean([np.median(j) for j in m]),c='purple',lw=3)
print(np.mean(scores))
print(np.median(m))
print(np.var(m))

#### #3, normalize probabilities and avg within bootstrap then avg across bootstrap validation probabilities

In [ ]:
fig,ax=plt.subplots(dpi=200)
scorer = roc_auc_score
scores=[]
m=[]
v=[]
for i,grp in ki_pperf.groupby('bootstrap'):
    vals=grp.y_proba.values
    norm_vals = (vals - min(vals)) / (max(vals) - min(vals))
    m.append(norm_vals)
    scores.append(scorer(grp.y_true.values,norm_vals))
    sns.kdeplot(norm_vals,color='blue',alpha=.1,ax=ax)
sns.kdeplot([np.median(j) for j in m],color='red',ax=ax)
plt.axvline(np.mean([np.median(j) for j in m]),c='purple',lw=3)
print(np.mean(scores))
print(np.median(m))
print(np.var(m))

#### #1, bootstrap all 13*200 validation probabilities 50 times

In [ ]:
fig,ax=plt.subplots(dpi=200)
dat = ki_pperf.copy()
scorer = roc_auc_score
n=50
m = []
scores=[]
vars_=[]
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True)
        )
    vals = x.y_proba.values
    norm_vals = (vals - min(vals)) / (max(vals) - min(vals))
    scores.append(scorer(x.y_true,vals))
    sns.kdeplot(vals,color='blue',alpha=.1,ax=ax)
    m.append(vals)
sns.kdeplot([np.median(j) for j in m],color='red',ax=ax)
print(np.mean(scores))
print(np.median(m))
print(np.var(m))

#### #1, normalize the bootstrap validation probabilities 50 times 

In [ ]:
fig,ax=plt.subplots(dpi=200)
dat = ki_pperf.copy()
n=50
m = []
scores=[]
vars_=[]
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True)
        )
    vals = x.y_proba.values
    norm_vals = (vals - min(vals)) / (max(vals) - min(vals))
    scores.append(scorer(x.y_true,norm_vals))
    sns.kdeplot(norm_vals,color='blue',alpha=.1,ax=ax)
    m.append(norm_vals)
sns.kdeplot([np.median(j) for j in m],color='red',ax=ax)
print(np.mean(scores))
print(np.median(m))
print(np.var(m))

#### Curves and stats

In [ ]:
print('roc_auc')
func = roc_auc_score
print('k')
display(pd.DataFrame(get_pperf_scores(k_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(k_pperf,scorer=func))
print('i')
display(pd.DataFrame(get_pperf_scores(i_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(i_pperf,scorer=func))
print('ki')
display(pd.DataFrame(get_pperf_scores(ki_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(ki_pperf,scorer=func))

display(compare_integrated_scores(k_pperf,i_pperf,f1='KLKB1',f2='Inotrope',scorer=func))
display(compare_integrated_scores(i_pperf,ki_pperf,f1='Inotrope',f2='KLKB1+Inotrope',scorer=func))
display(compare_integrated_scores(k_pperf,ki_pperf,f1='KLKB1',f2='KLKB1+Inotrope',scorer=func))

print('auprc')
func = average_precision_score
print('k')
display(pd.DataFrame(get_pperf_scores(k_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(k_pperf,scorer=func))
print('i')
display(pd.DataFrame(get_pperf_scores(i_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(i_pperf,scorer=func))
print('ki')
display(pd.DataFrame(get_pperf_scores(ki_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(ki_pperf,scorer=func))

display(compare_integrated_scores(k_pperf,i_pperf,f1='KLKB1',f2='Inotrope',scorer=func))
display(compare_integrated_scores(i_pperf,ki_pperf,f1='Inotrope',f2='KLKB1+Inotrope',scorer=func))
display(compare_integrated_scores(k_pperf,ki_pperf,f1='KLKB1',f2='KLKB1+Inotrope',scorer=func))


In [ ]:
def get_pperf_roc_curve_stats(dat,n=50):
    
    tups = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        f,t,th = roc_curve(x.y_true,x.y_proba)

        tups.append(
            pd.DataFrame({ 'fpr' : f,
                          'tpr' : t,
                          't' : th
                         }
                        )
        )

    tmp = pd.concat(tups).groupby('t').mean()
    fpr = tmp['fpr'].values
    tpr = tmp['tpr'].values
    return fpr,tpr
    
def get_pperf_precision_recall_curve_stats(dat,n=50):
    
    tups = []
    for b in range(n):
        x = (dat.
             sample(n=dat.shape[0],replace=True,random_state=b)
            )
        r,p,th = precision_recall_curve(x.y_true,x.y_proba)
        r = list(r)
        p = list(p)
        r.pop()
        p.pop()
        tups.append(
            pd.DataFrame({ 'precision' : p,
                          'recall' : r,
                          't' : th
                         }
                        )
        )

    tmp = pd.concat(tups).groupby('t').mean()
    p = tmp['precision'].tolist()
    r = tmp['recall'].tolist()
    p[0] = 1
    r[0] = 0
    return p,r

def plt_atts_roc(ax,fig):
    ax.set_xlim(-0.01,1.01)
    ax.set_ylim(-0.01,1.01)

    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)

    ax.set_ylabel('Sensitivity',size=18)
    ax.set_xlabel('1 - Specificity',size=18)
    ax.tick_params(axis='both', which='major', labelsize=14)

    fig.tight_layout()
    
    return fig

def plt_atts_pr(ax,fig):
    ax.set_xlim(-0.01,1.01)
    ax.set_ylim(-0.01,1.01)

    lims = [
        [np.min(ax.get_xlim()), np.max(ax.get_ylim())],  
        [np.max(ax.get_xlim()), np.min(ax.get_ylim())]
    ]

    # now plot both limits against eachother
    ax.plot(lims[0], lims[1], 'r--', alpha=0.75, zorder=0)

    ax.set_ylabel('Precision',size=18)
    ax.set_xlabel('Recall',size=18)
    ax.tick_params(axis='both', which='major', labelsize=14)

    fig.tight_layout()
    
    return fig

In [ ]:
type_='PGD_Prediction_Panel'
func=get_pperf_roc_curve_stats
fig,ax = plt.subplots(dpi=dpi)

fpr,tpr = func(ki_pperf.copy())

c='purple'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)

fpr,tpr = func(k_pperf.copy())

c='red'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='^',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)

fpr,tpr = func(i_pperf.copy())

c='steelblue'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='s',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)
fig.savefig(dropbox_figures+'All_Cohorts_Best_Clinical_Protein_'+type_+'.png')

cohorts=['Columbia','Cedar','Paris']
for cohort in cohorts:
    fig,ax = plt.subplots(dpi=dpi)

    fpr,tpr = func(ki_pperf.query('cohort==@cohort').copy())

    c='purple'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.001)
    fog = plt_atts_roc(ax,fig)

    fpr,tpr = func(k_pperf.query('cohort==@cohort').copy())

    c='red'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='^',mec=c,ms=1,lw=0.001)
    fig = plt_atts_roc(ax,fig)

    fpr,tpr = func(i_pperf.query('cohort==@cohort').copy())

    c='steelblue'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='s',mec=c,ms=1,lw=0.001)
    fig = plt_atts_roc(ax,fig)
    fig.savefig(dropbox_figures+cohort+'_Best_Clinical_Protein_'+type_+'.png')


In [ ]:
type_='PGD_Precision_Recall_Panel'
func=get_pperf_precision_recall_curve_stats
fig,ax = plt.subplots(dpi=dpi)

fpr,tpr = func(ki_pperf.copy())

c='purple'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)

fpr,tpr = func(k_pperf.copy())

c='red'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='^',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)

fpr,tpr = func(i_pperf.copy())

c='steelblue'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='s',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)
fig.savefig(dropbox_figures+'All_Cohorts_Best_Clinical_Protein_'+type_+'.png')

cohorts=['Columbia','Cedar','Paris']
for cohort in cohorts:
    fig,ax = plt.subplots(dpi=dpi)

    fpr,tpr = func(ki_pperf.query('cohort==@cohort').copy())

    c='purple'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.001)
    fog = plt_atts_pr(ax,fig)

    fpr,tpr = func(k_pperf.query('cohort==@cohort').copy())

    c='red'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='^',mec=c,ms=1,lw=0.001)
    fig = plt_atts_pr(ax,fig)

    fpr,tpr = func(i_pperf.query('cohort==@cohort').copy())

    c='steelblue'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='s',mec=c,ms=1,lw=0.001)
    fig = plt_atts_pr(ax,fig)
    fig.savefig(dropbox_figures+cohort+'_Best_Clinical_Protein_'+type_+'.png')


### Composite measures vs top panel

In [ ]:
cvppcwp_pperf = \
(pd.
 read_csv('../../data/integrated_pgd_predictions/'+
          'clinical_01_within_notwithcohorts_features_pgd_prediction_'+
          'CVP_PCWP_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',
         index_col=0)
)
meld_pperf = \
(pd.
 read_csv('../../data/integrated_pgd_predictions/'+
          'clinical_01_within_notwithcohorts_features_pgd_prediction_'+
          'MELD_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',
         index_col=0)
)
radial_pperf = \
(pd.
 read_csv('../../data/integrated_pgd_predictions/'+
          'clinical_01_within_notwithcohorts_features_pgd_prediction_'+
          'Radial_Score_prediction_metric_bootstrap_train_test_val_patient_level_data.csv',
         index_col=0)
)

print('roc_auc')
func = roc_auc_score
print('cvppcwp')
display(pd.DataFrame(get_pperf_scores(cvppcwp_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(cvppcwp_pperf,scorer=func))
print('meld')
display(pd.DataFrame(get_pperf_scores(meld_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(meld_pperf,scorer=func))
print('radial')
display(pd.DataFrame(get_pperf_scores(radial_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(radial_pperf,scorer=func))
print('ki')
display(pd.DataFrame(get_pperf_scores(ki_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(ki_pperf,scorer=func))

display(compare_integrated_scores(cvppcwp_pperf,radial_pperf,f1='CVP/PCWP',f2='Radial Score',scorer=func))
display(compare_integrated_scores(cvppcwp_pperf,meld_pperf,f1='CVP/PCWP',f2='MELD',scorer=func))
display(compare_integrated_scores(meld_pperf,radial_pperf,f1='MELD',f2='Radial Score',scorer=func))
display(compare_integrated_scores(radial_pperf,ki_pperf,f1='Radial Score',f2='KLKB1+Inotrope',scorer=func))
display(compare_integrated_scores(cvppcwp_pperf,ki_pperf,f1='CVP/PCWP',f2='KLKB1+Inotrope',scorer=func))
display(compare_integrated_scores(meld_pperf,ki_pperf,f1='MELD',f2='KLKB1+Inotrope',scorer=func))

print('auprc')
func=average_precision_score
print('cvppcwp')
display(pd.DataFrame(get_pperf_scores(cvppcwp_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(cvppcwp_pperf,scorer=func))
print('meld')
display(pd.DataFrame(get_pperf_scores(meld_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(meld_pperf,scorer=func))
print('radial')
display(pd.DataFrame(get_pperf_scores(radial_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(radial_pperf,scorer=func))
print('ki')
display(pd.DataFrame(get_pperf_scores(ki_pperf,scorer=func),index=['Integrated','Columbia','Cedar',
                                                       'Paris','avg_cohort_score',
                                                       'avg_score','var_cohort_score',
                                                       'var_score']).T)
display(get_pperf_CI_scores(ki_pperf,scorer=func))

display(compare_integrated_scores(cvppcwp_pperf,radial_pperf,f1='CVP/PCWP',f2='Radial Score',scorer=func))
display(compare_integrated_scores(cvppcwp_pperf,meld_pperf,f1='CVP/PCWP',f2='MELD',scorer=func))
display(compare_integrated_scores(meld_pperf,radial_pperf,f1='MELD',f2='Radial Score',scorer=func))
display(compare_integrated_scores(radial_pperf,ki_pperf,f1='Radial Score',f2='KLKB1+Inotrope',scorer=func))
display(compare_integrated_scores(cvppcwp_pperf,ki_pperf,f1='CVP/PCWP',f2='KLKB1+Inotrope',scorer=func))
display(compare_integrated_scores(meld_pperf,ki_pperf,f1='MELD',f2='KLKB1+Inotrope',scorer=func))



In [ ]:
type_='PGD_Prediction_Composite_Panel'
func=get_pperf_roc_curve_stats
fig,ax = plt.subplots(dpi=dpi)

fpr,tpr = func(cvppcwp_pperf.copy())

c='brown'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='X',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)

fpr,tpr = func(radial_pperf.copy())

c='blue'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='D',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)

fpr,tpr = func(meld_pperf.copy())

c='orange'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='p',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)

fpr,tpr = func(ki_pperf.copy())

c='purple'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.00001)
fig = plt_atts_roc(ax,fig)
fig.savefig(dropbox_figures+'All_Cohorts_Best_Clinical_Protein_'+type_+'.png')

cohorts=['Columbia','Cedar','Paris']
for cohort in cohorts:
    fig,ax = plt.subplots(dpi=dpi)

    fpr,tpr = func(cvppcwp_pperf.query('cohort==@cohort').copy())

    c='brown'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='X',mec=c,ms=1,lw=0.001)
    fog = plt_atts_roc(ax,fig)

    fpr,tpr = func(radial_pperf.query('cohort==@cohort').copy())

    c='blue'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='D',mec=c,ms=1,lw=0.001)
    fig = plt_atts_roc(ax,fig)

    fpr,tpr = func(meld_pperf.query('cohort==@cohort').copy())

    c='orange'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='p',mec=c,ms=1,lw=0.001)
    fig = plt_atts_roc(ax,fig)

    fpr,tpr = func(ki_pperf.query('cohort==@cohort').copy())
    
    c='purple'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.001)
    fig = plt_atts_roc(ax,fig)

    fig.savefig(dropbox_figures+cohort+'_Best_Clinical_Protein_'+type_+'.png')


In [ ]:
type_='PGD_Precision_Recall_Composite_Panel'
func=get_pperf_precision_recall_curve_stats
fig,ax = plt.subplots(dpi=dpi)

fpr,tpr = func(cvppcwp_pperf.copy())

c='brown'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='X',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)

fpr,tpr = func(radial_pperf.copy())

c='blue'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='D',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)

fpr,tpr = func(meld_pperf.copy())

c='orange'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='p',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)

fpr,tpr = func(ki_pperf.copy())

c='purple'
ax.plot(fpr,tpr,c=c)
ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.00001)
fig = plt_atts_pr(ax,fig)
fig.savefig(dropbox_figures+'All_Cohorts_Best_Clinical_Protein_'+type_+'.png')

cohorts=['Columbia','Cedar','Paris']
for cohort in cohorts:
    fig,ax = plt.subplots(dpi=dpi)

    fpr,tpr = func(cvppcwp_pperf.query('cohort==@cohort').copy())

    c='brown'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='X',mec=c,ms=1,lw=0.001)
    fog = plt_atts_pr(ax,fig)

    fpr,tpr = func(radial_pperf.query('cohort==@cohort').copy())

    c='blue'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='D',mec=c,ms=1,lw=0.001)
    fig = plt_atts_pr(ax,fig)

    fpr,tpr = func(meld_pperf.query('cohort==@cohort').copy())

    c='orange'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='p',mec=c,ms=1,lw=0.001)
    fig = plt_atts_pr(ax,fig)

    fpr,tpr = func(ki_pperf.query('cohort==@cohort').copy())
    
    c='purple'
    ax.plot(fpr,tpr,c=c)
    ax.plot(fpr,tpr,'.',c=c,marker='o',mec=c,ms=1,lw=0.001)
    fig = plt_atts_pr(ax,fig)

    fig.savefig(dropbox_figures+cohort+'_Best_Clinical_Protein_'+type_+'.png')


### Inotrope therapy prediction in each cohort

In [ ]:
%run /Users/nickgiangreco/Research/Projects/exosome_pgf/src/python/prediction_functions.py


metric = 'roc_auc'
cv_split = 10
n_jobs = 20
nboot=200
test_size = 0.15
treat='PGD'
i=0
classification_metrics = ['roc_auc']

dir_="../../data/"
cohort = 'integrated'

def get_performance(lst):
    perf = (pd.
            concat(lst,keys=range(len(lst))).
            reset_index(level=1,drop=True).
            rename_axis('bootstrap').
            reset_index()
           )
    return perf

def model_feature_importances(boot_mods):
    dfs = []
    X = params['X'].copy()
    X.loc[:,'Intercept'] = 0
    for i in range(len(boot_mods)):
        for j in boot_mods[i].keys():
            mod = boot_mods[i][j]
            coef = []
            try:
                coef.extend([i for i in mod.feature_importances_])
            except:
                coef.extend([i for i in mod.coef_[0]])
            coef.extend(mod.intercept_)
            fs = []
            fs.extend(X.columns.values)
            df = pd.DataFrame({
                'Feature' : fs,
                'Gene_name' : (X.T.
                               join(idmap_sub.
                                    set_index('Protein'),how='left').
                               Gene_name.values),
                'Importance' : coef,
                'Model' : j,
                'Bootstrap' : i
            })
            dfs.append(df)
    return pd.concat(dfs,sort=True)

def patient_predictions(lst):
        dat = \
        (pd.
         concat(
             lst
         ).
         reset_index().
         rename(columns={0 : 'Sample'}).
         set_index('Sample').
         join(all_cov_df,how='left').
         reset_index().
         melt(id_vars=['Sample','bootstrap','model','y_true','y_pred','y_proba'],
              var_name='cohort',value_name='mem')
        )
        dat.cohort = dat.cohort.str.split('_').apply(lambda x : x[1])
        dat = dat[dat.mem==1].drop('mem',1).reset_index(drop=True)
        return dat

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_covariates.csv',index_col=0)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)

cov_df = X_all_clinical.loc[:,['Cohort_Columbia','Cohort_Cedar']].copy().astype(int)
all_cov_df = cov_df.copy()
all_cov_df.loc[:,'Cohort_Paris'] = (
    (all_cov_df['Cohort_Columbia'] + 
     all_cov_df['Cohort_Cedar'])==0).astype(int)

idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

features = ['Prior_Inotrope_Y']

X_all = X_all_proteins.join(X_all_clinical)
X = X_all[features]
feature_set[str(i)] = X.columns.tolist()

In [ ]:
i=0
params = {'X' : X,'Y' : Y, 'cv_split' : cv_split, 
		  'metrics' : classification_metrics, 'n_jobs' : 1, 
		  'test_size' : test_size,
		  'retrained_models' : True, 'patient_level_predictions' : True,
         'models' : l1_logit_model.copy()}
lst = bootstrap_of_fcn(func=train_test_val_top_fold_01_within,
               params=params,n_jobs=n_jobs,nboot=nboot)
perf_all = get_performance([lst[i][0] for i in range(len(lst))])
perf_all['set'] = str(i)
fimps_all = model_feature_importances([lst[i][1] for i in range(len(lst))])
fimps_all['set'] = str(i)
ppreds_all = patient_predictions([lst[i][2] for i in range(len(lst))])
ppreds_all['set'] = str(i)

In [ ]:
ppreds_all.groupby(['cohort','y_true'])['y_proba'].mean()

In [ ]:
pperf_dat_processing(ppreds_all)

### Sex_F prediction - investigating AUROC<0.5

In [ ]:
%run /Users/nickgiangreco/Research/Projects/exosome_pgf/src/python/prediction_functions.py


metric = 'roc_auc'
cv_split = 10
n_jobs = 20
nboot=200
test_size = 0.15
treat='PGD'
i=0
classification_metrics = ['roc_auc']

dir_="../../data/"
cohort = 'integrated'

def get_performance(lst):
    perf = (pd.
            concat(lst,keys=range(len(lst))).
            reset_index(level=1,drop=True).
            rename_axis('bootstrap').
            reset_index()
           )
    return perf

def model_feature_importances(boot_mods):
    dfs = []
    X = params['X'].copy()
    X.loc[:,'Intercept'] = 0
    for i in range(len(boot_mods)):
        for j in boot_mods[i].keys():
            mod = boot_mods[i][j]
            coef = []
            try:
                coef.extend([i for i in mod.feature_importances_])
            except:
                coef.extend([i for i in mod.coef_[0]])
            coef.extend(mod.intercept_)
            fs = []
            fs.extend(X.columns.values)
            df = pd.DataFrame({
                'Feature' : fs,
                'Gene_name' : (X.T.
                               join(idmap_sub.
                                    set_index('Protein'),how='left').
                               Gene_name.values),
                'Importance' : coef,
                'Model' : j,
                'Bootstrap' : i
            })
            dfs.append(df)
    return pd.concat(dfs,sort=True)

def patient_predictions(lst):
        dat = \
        (pd.
         concat(
             lst
         ).
         reset_index().
         rename(columns={0 : 'Sample'}).
         set_index('Sample').
         join(all_cov_df,how='left').
         reset_index().
         melt(id_vars=['Sample','bootstrap','model','y_true','y_pred','y_proba'],
              var_name='cohort',value_name='mem')
        )
        dat.cohort = dat.cohort.str.split('_').apply(lambda x : x[1])
        dat = dat[dat.mem==1].drop('mem',1).reset_index(drop=True)
        return dat

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_covariates.csv',index_col=0)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)

cov_df = X_all_clinical.loc[:,['Cohort_Columbia','Cohort_Cedar']].copy().astype(int)
all_cov_df = cov_df.copy()
all_cov_df.loc[:,'Cohort_Paris'] = (
    (all_cov_df['Cohort_Columbia'] + 
     all_cov_df['Cohort_Cedar'])==0).astype(int)

idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

features = ['Sex_F']

X_all = X_all_proteins.join(X_all_clinical)
X = X_all[features]
tmp = X.join(Y)
tmp['mem'] = 1
tmp.groupby(['Sex_F',1])['mem'].sum()

In [ ]:
nonpgd = Y.index.values[Y.values.reshape(1,-1)[0]==0]
pgd = Y.index.values[Y.values.reshape(1,-1)[0]==1]
male = X.index.values[X.values.reshape(1,-1)[0]==0]
female = X.index.values[X.values.reshape(1,-1)[0]==1]
val=1
Y.at[np.intersect1d(male,nonpgd)[:9],1] = 1
#X.at[np.intersect1d(female,nonpgd)[:5],'Sex_F'] = 0
#Y.at[np.intersect1d(male,pgd)[:5],1] = 0
#X.at[np.intersect1d(female,pgd)[:5],'Sex_F'] = 0
tmp = X.join(Y)
tmp['mem'] = 1
tmp.groupby(['Sex_F',1])['mem'].sum()

In [ ]:
tmp = X.rename(columns={'Sex_F' : 'Sex_new'})
tmp['Sex_new'] = np.random.randint(0,2,X.shape[0])
X = tmp
#tmp.to_csv('../../X.csv')
Y =  pd.Series(np.random.randint(0,2,X.shape[0]),index=Y.index)
Y.name=1
tmp = X.join(Y)
tmp['mem'] = 1
tmp.groupby(['Sex_new',1])['mem'].sum()

In [ ]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
i=0
params = {'X' : X,'Y' : Y, 'cv_split' : cv_split, 
		  'metrics' : classification_metrics, 'n_jobs' : 1, 
		  'test_size' : test_size,
		  'retrained_models' : True, 'patient_level_predictions' : True,
         'models' : {'Logistic Regression' : LogisticRegression(solver='saga')}}
lst = bootstrap_of_fcn(func=train_test_val_top_fold_01_within_unveiled,
               params=params,n_jobs=n_jobs,nboot=nboot)
perf_all = get_performance([lst[i][0] for i in range(len(lst))])
perf_all['set'] = str(i)
fimps_all = model_feature_importances([lst[i][1] for i in range(len(lst))])
fimps_all['set'] = str(i)
ppreds_all = patient_predictions([lst[i][2] for i in range(len(lst))])
ppreds_all['set'] = str(i)

In [ ]:
for i,mod in enumerate([lst[i][1] for i in range(len(lst))]):
    X_train, y_train, X_test, y_test = [lst[i][3] for i in range(len(lst))][i]
    auroc = roc_auc_score(y_test.values.reshape(1,-1)[0],
                          mod['Logistic Regression'].predict_proba(X_test)[:,1])
    if auroc<.5:
        print(y_test.values.reshape(1,-1)[0])
        print(mod['Logistic Regression'].coef_[0][0])
        print(mod['Logistic Regression'].predict_proba(X_test)[:,1])
        print(auroc,'\n')

In [ ]:
def XY_data_processing(lst,cov='Sex_F',cov_name='females',noncov_name='males'):
    X_train, y_train, X_test, y_test = lst
    train_cov_breakdown = X_train.join(y_train).groupby([cov]).sum().reindex([0,1]).fillna(0)
    train_pgd_breakdown = X_train.join(y_train).groupby([1]).sum().reindex([0,1]).fillna(0)
    train_total = X_train.shape[0]
    train_cov = train_pgd_breakdown.sum().values[0]
    train_noncov = train_total - train_cov
    train_pgd = train_cov_breakdown.sum().values[0]
    train_nonpgd = train_total - train_pgd
    train_nonpgd_cov = train_pgd_breakdown.loc[0].values[0]
    train_nonpgd_noncov = train_nonpgd - train_nonpgd_cov
    train_pgd_cov = train_cov_breakdown.loc[1].values[0]
    train_pgd_noncov = train_pgd - train_pgd_cov    

    val_cov_breakdown = X_test.join(y_test).groupby([cov]).sum().reindex([0,1]).fillna(0)
    val_pgd_breakdown = X_test.join(y_test).groupby([1]).sum().reindex([0,1]).fillna(0)
    val_total = X_test.shape[0]
    val_cov = val_pgd_breakdown.sum().values[0]
    val_noncov = val_total - val_cov
    val_pgd = val_cov_breakdown.sum().values[0]
    val_nonpgd = val_total - val_pgd
    val_nonpgd_cov = val_pgd_breakdown.loc[0].values[0]
    val_nonpgd_noncov = val_nonpgd - val_nonpgd_cov
    val_pgd_cov = val_cov_breakdown.loc[1].values[0]
    val_pgd_noncov = val_pgd - val_pgd_cov
    
    res = pd.DataFrame(
        [train_total,train_nonpgd,train_pgd,
         train_cov,train_noncov,
         train_nonpgd_noncov, train_nonpgd_cov, train_pgd_noncov, train_pgd_cov,
         val_total,val_nonpgd,val_pgd,
         val_cov,val_noncov,
         val_nonpgd_noncov, val_nonpgd_cov,val_pgd_noncov, val_pgd_cov],
        index=['N_training','N_training_nonpgd','N_training_pgd',
               'N_training_'+cov_name,'N_training_'+noncov_name,
               'N_training_nonpgd_'+noncov_name,'N_training_nonpgd_'+cov_name,
                 'N_training_pgd_'+noncov_name,'N_training_pgd_'+cov_name,
                 'N_validation','N_validation_nonpgd','N_validation_pgd',
               'N_validation_'+cov_name,'N_validation_'+noncov_name,
               'N_validation_nonpgd_'+noncov_name,'N_validation_nonpgd_'+cov_name,
                 'N_validation_pgd_'+noncov_name,'N_validation_pgd_'+cov_name]
    ).T
    return res

In [ ]:
XY_data_N = \
(pd.concat([XY_data_processing(lst[i][3],cov='Sex_new') for i in range(len(lst))]).
 reset_index(drop=True).
 rename_axis('bootstrap').
 reset_index()
)

In [ ]:
n=50
dat = ppreds_all
vals = []
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True)
        )
    vals.append(['Sex_new',b,x.model.unique()[0],roc_auc_score(x.y_true,x.y_proba)])
tmp = pd.DataFrame(vals,columns=['Feature','Bootstrap','Model','roc_auc'])
tmp['roc_auc'].mean()

In [ ]:
XY_data_imp_score = \
(XY_data_N.
 join(fimps_all.
      query('Feature=="Sex_new"').
      set_index('Bootstrap')).
 join(tmp[['Bootstrap','roc_auc']].set_index('Bootstrap')).
 rename(columns={'roc_auc' : 'bootstrapped_roc_auc',
                 'Importance' : 'Beta_Coefficient'})
)
XY_data_imp_score['Importance'] = [lst[i][1]['Logistic Regression'].coef_[0][0] for i in range(len(lst))]
aurocs = []
coefs = []
for i,mod in enumerate([lst[i][1] for i in range(len(lst))]):
    X_train, y_train, X_test, y_test = [lst[i][3] for i in range(len(lst))][i]
    auroc = roc_auc_score(y_test.values.reshape(1,-1)[0],
                          mod['Logistic Regression'].predict_proba(X_test)[:,1])
    coef = mod['Logistic Regression'].coef_[0][0]
    coefs.append(coef)
    aurocs.append(auroc)
XY_data_imp_score['Importance'] = coefs
XY_data_imp_score['roc_auc'] = aurocs
print(XY_data_imp_score.shape)
XY_data_imp_score.head()

In [ ]:
(XY_data_imp_score['roc_auc']==0.5).sum()

In [ ]:
aurocs=[]
for i in range(50):
    auroc = (XY_data_imp_score.
             roc_auc.
             sample(n=XY_data_imp_score.shape[0],replace=True).
             mean()
            )
    aurocs.append(auroc)
plt.hist(aurocs)

In [ ]:
XY_data_imp_score[['roc_auc']].describe(percentiles=[0.025,0.975
                                                    ])

In [ ]:
XY_data_imp_score[['bootstrapped_roc_auc']].describe(percentiles=[0.025,0.975
                                                    ])

In [ ]:
XY_data_imp_score['N_training_pgd_females_freq'] = \
XY_data_imp_score['N_training_pgd_females'] / XY_data_imp_score['N_training_pgd']
XY_data_imp_score['N_training_nonpgd_females_freq'] = \
XY_data_imp_score['N_training_nonpgd_females'] / XY_data_imp_score['N_training_nonpgd']
XY_data_imp_score['N_validation_pgd_females_freq'] = \
XY_data_imp_score['N_validation_pgd_females'] / XY_data_imp_score['N_validation_pgd']
XY_data_imp_score['N_validation_nonpgd_females_freq'] = \
XY_data_imp_score['N_validation_nonpgd_females'] / XY_data_imp_score['N_validation_nonpgd']
XY_data_imp_score['N_training_pgd_males_freq'] = \
XY_data_imp_score['N_training_pgd_males'] / XY_data_imp_score['N_training_pgd']
XY_data_imp_score['N_training_nonpgd_males_freq'] = \
XY_data_imp_score['N_training_nonpgd_males'] / XY_data_imp_score['N_training_nonpgd']
XY_data_imp_score['N_validation_pgd_males_freq'] = \
XY_data_imp_score['N_validation_pgd_males'] / XY_data_imp_score['N_validation_pgd']

In [ ]:
from sklearn.linear_model import Ridge
mod = Ridge(normalize=False)
tmp = (XY_data_imp_score.
         filter(like='N').
       drop(['N_training','N_validation'],1).
       filter(regex='females$').
         apply(lambda x : (x - min(x)) / (max(x) - min(x)) ))
mod.fit(tmp,
        XY_data_imp_score[['roc_auc']])
pd.DataFrame({
    'term' : tmp.columns,
    'coef' : mod.coef_[0]
}).sort_values('coef')

In [ ]:
(XY_data_imp_score['N_training_pgd_females_freq'].describe(percentiles=[.025,0.975]))
display(XY_data_imp_score['N_training_nonpgd_females_freq'].describe(percentiles=[.025,0.975]))
display(XY_data_imp_score['N_validation_pgd_females_freq'].describe(percentiles=[.025,0.975]))
display(XY_data_imp_score['N_training_pgd_females_freq'].describe(percentiles=[.025,0.975]))
display(XY_data_imp_score['N_validation_nonpgd_females_freq'].describe(percentiles=[.025,0.975]))

In [ ]:
a = XY_data_imp_score['N_training_pgd_females_freq']
b = XY_data_imp_score['N_validation_pgd_females_freq']
c = XY_data_imp_score['roc_auc']
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,c,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(b,c,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c<.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c==.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c>.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c,ax=ax)

In [ ]:
a = XY_data_imp_score['N_training_pgd_males_freq']
b = XY_data_imp_score['N_validation_pgd_males_freq']
c = XY_data_imp_score['roc_auc']
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,c,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(b,c,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c<.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c==.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c>.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c,ax=ax)

In [ ]:
a = XY_data_imp_score.N_training_pgd_females_freq
b = XY_data_imp_score.N_training_nonpgd_females_freq
c = XY_data_imp_score.roc_auc
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(b,c,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c<.5,ax=ax,cmap='viridis')
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c==.5,ax=ax,cmap='viridis')
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c>.5,ax=ax,cmap='viridis')
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c,ax=ax,cmap='viridis')
ax.legend().remove()

In [ ]:
a = XY_data_imp_score.N_training_pgd_males_freq
b = XY_data_imp_score.N_training_nonpgd_males_freq
c = XY_data_imp_score.roc_auc
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(b,c,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c<.5,ax=ax,cmap='viridis')
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c==.5,ax=ax,cmap='viridis')
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c>.5,ax=ax,cmap='viridis')
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c,ax=ax,cmap='viridis')
ax.legend().remove()

In [ ]:
a = XY_data_imp_score.N_training_pgd_females_freq
b = XY_data_imp_score.N_training_nonpgd_males_freq
c = XY_data_imp_score.roc_auc
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c<.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue= c==.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c>.5,ax=ax)
fig,ax=plt.subplots(dpi=200)
sns.scatterplot(a,b,hue=c,ax=ax)
ax.legend().remove()

In [ ]:
fimps_all.query('Feature=="Sex_F"')['Importance'].value_counts()

In [ ]:
tmp = (ppreds_all.
 set_index('Sample').
 join(X_all_clinical).
       reset_index().
 loc[:,['bootstrap','Sample','y_true','y_proba','cohort','Sex_F']])
(tmp.
groupby(['bootstrap'])['Sex_F'].sum()).hist()


In [ ]:
tmp = (perf_all.
       set_index('bootstrap').
       drop('set',1).
       join(fimps_all.
            query('Feature=="Sex_F"').
            set_index('Bootstrap')).
       loc[:,['test_roc_auc','validation_roc_auc','Feature','Importance']]
      )
fig,ax=plt.subplots(dpi=200)
sns.scatterplot('Importance','validation_roc_auc',
                data=tmp,linewidth=.2,edgecolor='black',ax=ax)
tmp

In [ ]:
not_these = fimps_all.query('Importance==0 & Feature=="Sex_F"')['Bootstrap'].values

In [ ]:
n=50
dat = ppreds_all
vals = []
for b in range(n):
    x = (dat.
         query('bootstrap not in @not_these').
         sample(n=200-len(not_these),replace=True,random_state=b)
        )
    vals.append(['Sex_F',b,x.model.unique()[0],roc_auc_score(x.y_true,x.y_proba)])
tmp = pd.DataFrame(vals,columns=['Feature','Bootstrap','Model','roc_auc'])
tmp['roc_auc'].mean()

In [ ]:
n=50
dat = ppreds_all
vals = []
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True,random_state=b)
        )
    vals.append(['Sex_F',b,x.model.unique()[0],roc_auc_score(x.y_true,x.y_proba)])
tmp = pd.DataFrame(vals,columns=['Feature','Bootstrap','Model','roc_auc'])
tmp['roc_auc'].mean()

### Inotrope and LVAD relationship to PGD

In [ ]:
%run /Users/nickgiangreco/Research/Projects/exosome_pgf/src/python/prediction_functions.py


metric = 'roc_auc'
cv_split = 10
n_jobs = 20
nboot=200
test_size = 0.15
treat='PGD'
i=0
classification_metrics = ['roc_auc']

dir_="../../data/"
cohort = 'integrated'

def get_performance(lst):
    perf = (pd.
            concat(lst,keys=range(len(lst))).
            reset_index(level=1,drop=True).
            rename_axis('bootstrap').
            reset_index()
           )
    return perf

def model_feature_importances(boot_mods):
    dfs = []
    X = params['X'].copy()
    X.loc[:,'Intercept'] = 0
    for i in range(len(boot_mods)):
        for j in boot_mods[i].keys():
            mod = boot_mods[i][j]
            coef = []
            try:
                coef.extend([i for i in mod.feature_importances_])
            except:
                coef.extend([i for i in mod.coef_[0]])
            coef.extend(mod.intercept_)
            fs = []
            fs.extend(X.columns.values)
            df = pd.DataFrame({
                'Feature' : fs,
                'Gene_name' : (X.T.
                               join(idmap_sub.
                                    set_index('Protein'),how='left').
                               Gene_name.values),
                'Importance' : coef,
                'Model' : j,
                'Bootstrap' : i
            })
            dfs.append(df)
    return pd.concat(dfs,sort=True)

def patient_predictions(lst):
        dat = \
        (pd.
         concat(
             lst
         ).
         reset_index().
         rename(columns={0 : 'Sample'}).
         set_index('Sample').
         join(all_cov_df,how='left').
         reset_index().
         melt(id_vars=['Sample','bootstrap','model','y_true','y_pred','y_proba'],
              var_name='cohort',value_name='mem')
        )
        dat.cohort = dat.cohort.str.split('_').apply(lambda x : x[1])
        dat = dat[dat.mem==1].drop('mem',1).reset_index(drop=True)
        return dat

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_covariates.csv',index_col=0)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)

cov_df = X_all_clinical.loc[:,['Cohort_Columbia','Cohort_Cedar']].copy().astype(int)
all_cov_df = cov_df.copy()
all_cov_df.loc[:,'Cohort_Paris'] = (
    (all_cov_df['Cohort_Columbia'] + 
     all_cov_df['Cohort_Cedar'])==0).astype(int)

idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

features = ['Prior_Inotrope_Y']

X_all = X_all_proteins.join(X_all_clinical)
X = X_all[features]

i=0
params = {'X' : X,'Y' : Y, 'cv_split' : cv_split, 
		  'metrics' : classification_metrics, 'n_jobs' : 1, 
		  'test_size' : test_size,
		  'retrained_models' : True, 'patient_level_predictions' : True,
         'models' : l1_logit_model.copy()}
lst = bootstrap_of_fcn(func=train_test_val_top_fold_01_within,
               params=params,n_jobs=n_jobs,nboot=nboot)
perf_i = get_performance([lst[i][0] for i in range(len(lst))])
perf_i['set'] = str(i)
fimps_i = model_feature_importances([lst[i][1] for i in range(len(lst))])
fimps_i['set'] = str(i)
ppreds_i = patient_predictions([lst[i][2] for i in range(len(lst))])
ppreds_i['set'] = str(i)

features = ['Mechanical_Support_Y']
X = X_all[features]

i=1
params = {'X' : X,'Y' : Y, 'cv_split' : cv_split, 
		  'metrics' : classification_metrics, 'n_jobs' : 1, 
		  'test_size' : test_size,
		  'retrained_models' : True, 'patient_level_predictions' : True,
         'models' : l1_logit_model.copy()}
lst = bootstrap_of_fcn(func=train_test_val_top_fold_01_within,
               params=params,n_jobs=n_jobs,nboot=nboot)
perf_l = get_performance([lst[i][0] for i in range(len(lst))])
perf_l['set'] = str(i)
fimps_l = model_feature_importances([lst[i][1] for i in range(len(lst))])
fimps_l['set'] = str(i)
ppreds_l = patient_predictions([lst[i][2] for i in range(len(lst))])
ppreds_l['set'] = str(i)

features = ['Prior_Inotrope_Y','Mechanical_Support_Y']
X = X_all[features]

i=2
params = {'X' : X,'Y' : Y, 'cv_split' : cv_split, 
		  'metrics' : classification_metrics, 'n_jobs' : 1, 
		  'test_size' : test_size,
		  'retrained_models' : True, 'patient_level_predictions' : True,
         'models' : l1_logit_model.copy()}
lst = bootstrap_of_fcn(func=train_test_val_top_fold_01_within,
               params=params,n_jobs=n_jobs,nboot=nboot)
perf_il = get_performance([lst[i][0] for i in range(len(lst))])
perf_il['set'] = str(i)
fimps_il = model_feature_importances([lst[i][1] for i in range(len(lst))])
fimps_il['set'] = str(i)
ppreds_il = patient_predictions([lst[i][2] for i in range(len(lst))])
ppreds_il['set'] = str(i)

In [ ]:
print(fimps_i.
      query('Feature!="Intercept"')['Importance'].
      describe(percentiles=[0.025,0.975]).
     loc[['2.5%','mean','97.5%']])
print(fimps_l.
      query('Feature!="Intercept"')['Importance'].
      describe(percentiles=[0.025,0.975]).
     loc[['2.5%','mean','97.5%']])
print(fimps_il.
      query('Feature!="Intercept"').
      groupby('Feature')['Importance'].
      describe(percentiles=[0.025,0.975]).
     loc[:,['2.5%','mean','97.5%']])

In [ ]:
n=50
dat = ppreds_i
vals = []
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True,random_state=b)
        )
    vals.append(['Inotrope Therapy',b,x.model.unique()[0],roc_auc_score(x.y_true,x.y_proba)])
tmp = pd.DataFrame(vals,columns=['Feature','Bootstrap','Model','roc_auc'])
tmp['roc_auc'].describe(percentiles=[0.025,0.975]).loc[['2.5%','mean','97.5%']]

In [ ]:
n=50
dat = ppreds_l
vals = []
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True,random_state=b)
        )
    vals.append(['LVAD',b,x.model.unique()[0],roc_auc_score(x.y_true,x.y_proba)])
tmp = pd.DataFrame(vals,columns=['Feature','Bootstrap','Model','roc_auc'])
tmp['roc_auc'].describe(percentiles=[0.025,0.975]).loc[['2.5%','mean','97.5%']]

In [ ]:
n=50
dat = ppreds_il
vals = []
for b in range(n):
    x = (dat.
         sample(n=dat.shape[0],replace=True,random_state=b)
        )
    vals.append(['Inotrope therapy and LVAD',b,x.model.unique()[0],roc_auc_score(x.y_true,x.y_proba)])
tmp = pd.DataFrame(vals,columns=['Feature','Bootstrap','Model','roc_auc'])
tmp['roc_auc'].describe(percentiles=[0.025,0.975]).loc[['2.5%','mean','97.5%']]

In [ ]:
plot_dat = \
(pd.concat([
    fimps_i.query('Feature!="Intercept"')[['Bootstrap','Feature','Importance','set']],
    fimps_l.query('Feature!="Intercept"')[['Bootstrap','Feature','Importance','set']],
    fimps_il.query('Feature!="Intercept"')[['Bootstrap','Feature','Importance','set']]
]).
 rename(
     columns={
         'Inotrope therapy' : 'Prior_Inotrope_Y',
         'LVAD' : 'Mechanical Support_Y'
     }
 )
)
set_dict = {
    0 : 'Inotrope therapy',
    1 : 'LVAD', 
    2 : 'Inotrope therapy and LVAD'
}

plot_dat['set'] = (plot_dat['set'].
                   astype(int).
                   apply(lambda x : set_dict[x])
                  )
plot_dat.head()

In [ ]:
fig,ax=plt.subplots(dpi=200)
sns.boxplot('set','Importance',hue='Feature',
            data=plot_dat,ax=ax,fliersize=0,
           color='lightgray')
ax.legend().remove()
ax.set_xlabel('')
ax.set_xticklabels(['Inotrope therapy','LVAD','Inotrope therapy\nand\nLVAD'],size=16)
ax.set_ylabel(r'$\beta$ coefficient',size=20)
ax.axvline(0.5,color='gray')
ax.axvline(1.5,color='gray')
ax.axhline(0,c='r',linestyle='--')
fig.tight_layout()
fig.savefig(dropbox_figures+'inotrope_lvad_prediction_comparison.png')

### KLKB1 validation

In [ ]:
data = pd.read_csv('../../data/KLKB1_80_DEIDENTIFIED_patient_validation.csv')
print(data.shape)
print(data['PGD'].value_counts())
print(data.columns)
print(data.barcode_id.values)
data.barcode_id = data.barcode_id.astype(int)
display(data.head())
data['PGD'].value_counts()

In [ ]:
cats = {}
i = 0
for cat in data.LVAD.unique():
    cats[cat] = i
    i = i + 1
data['LVAD_map'] = data.LVAD.map(cats)
data['PGD_agg'] = (data['PGD']>0).astype(int)
data.head()

#### PGD vs no PGD

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [1,2,3,4,5]')[var].values
display(ttest_ind(a,b))
mannwhitneyu(a,b)

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [1,2,3,4]')[var].values
display(ttest_ind(a,b))
mannwhitneyu(a,b)

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [1,2,3]')[var].values
display(ttest_ind(a,b))
mannwhitneyu(a,b)

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [1,2]')[var].values
display(ttest_ind(a,b))
mannwhitneyu(a,b)

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [2,3,4]')[var].values
display(ttest_ind(a,b))
mannwhitneyu(a,b)

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [2,3]')[var].values
print(ttest_ind(a,b))
print((np.mean(a), np.std(a)))
print((np.mean(b), np.std(b)))
mannwhitneyu(a,b)

In [ ]:
var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [3]')[var].values
print(ttest_ind(a,b))
print((np.mean(a), np.std(a)))
print((np.mean(b), np.std(b)))
mannwhitneyu(a,b)

#### PGD type by elisa swarmplot

In [ ]:
tmp = data.query('PGD in [0,2,3]').copy()
tmp['concentration'] = \
(tmp['concentration'] - tmp['concentration'].min() ) / ( tmp['concentration'].max() - tmp['concentration'].min())
fig,ax=plt.subplots(dpi=dpi,figsize=(10,4))
sns.boxplot('PGD','concentration',data=tmp,ax=ax,color='darkgrey',fliersize=0)
sns.stripplot('PGD','concentration',hue='Inotrope',
              palette=['lightgray','black'],edgecolor='black',
              data=tmp,
              jitter=True,linewidth=1
             )
ax.set_xticklabels(['no PGD','moderate PGD','severe PGD'],)
ax.set_xlabel('')
ax.set_ylabel('Normalized\nKLKB1 ELISA Concentration',size=18)
ax.tick_params(axis='both', which='both', length=0,labelsize=14)

noi_patch = Line2D([0],[0],marker='o',
                   markerfacecolor='lightgray',markeredgecolor='black',
                   color='w',markersize=5,label='no Inotrope')
i_patch = Line2D([0],[0],marker='o',
                 markerfacecolor='black',markeredgecolor='black',
                 color='w',markersize=5,label='Inotrope')
ax.legend(handles=[noi_patch,i_patch],title='',frameon=False)
fig.savefig(dropbox_figures+'pgd_grade_by_elisa')

#### no-PGD vs. PGD by elisa swarmplot

In [ ]:
tmp = data.query('PGD in [0,2,3]').copy()
tmp['concentration'] = \
(tmp['concentration'] - tmp['concentration'].min() ) / ( tmp['concentration'].max() - tmp['concentration'].min())
log = tmp.PGD==2
tmp.at[log,'PGD'] = 3
fig,ax=plt.subplots(dpi=dpi,figsize=(6,4))
sns.boxplot('PGD','concentration',data=tmp,ax=ax,color='darkgrey',fliersize=0)
sns.stripplot('PGD','concentration',hue='Inotrope',
              palette=['lightgray','black'],edgecolor='black',
              data=tmp,
              jitter=True,linewidth=1
             )
ax.set_xticklabels(['no PGD','moderate/severe PGD'],fontsize=20)
ax.set_xlabel('')
ax.set_ylabel('Normalized\nKLKB1 ELISA\nConcentration',size=18)
ax.tick_params(axis='both', which='both', length=0,labelsize=16)

noi_patch = Line2D([0],[0],marker='o',
                   markerfacecolor='lightgray',markeredgecolor='black',
                   color='w',markersize=5,label='no Inotrope')
i_patch = Line2D([0],[0],marker='o',
                 markerfacecolor='black',markeredgecolor='black',
                 color='w',markersize=5,label='Inotrope')
ax.legend(handles=[noi_patch,i_patch],title='',frameon=False,fontsize=20,markerscale=2)

var = 'concentration'
a = data.query('PGD in [0]')[var].values
b = data.query('PGD in [2,3]')[var].values
test, pv = mannwhitneyu(a,b)

ax.set_title('Mann Whitney test p-value={}'.format(np.round(pv,4)),size=20
        )
fig.tight_layout()
fig.savefig(dropbox_figures+'nopgd_pgd_by_elisa')

#### PGD agg by concentration

In [ ]:
tmp = data.copy()
tmp.loc[tmp.loc[:,'PGD'].isin([2,3]),'PGD_agg'] = 'PGD'
tmp.loc[tmp.loc[:,'PGD'].isin([1,4,5]),'PGD_agg'] = np.nan
tmp.loc[tmp.loc[:,'PGD'].isin([0]),'PGD_agg'] = 'no PGD'
tmp['concentration'] = \
(tmp['concentration'] - tmp['concentration'].min() ) / ( tmp['concentration'].max() - tmp['concentration'].min())
fig,ax=plt.subplots(dpi=dpi)
sns.boxplot('PGD_agg','concentration',color='gray',data=tmp,ax=ax,fliersize=0)
sns.stripplot('PGD_agg','concentration',
              marker='o',color='lightgray',edgecolor='black',
              data=tmp[tmp['Inotrope']==0],
              ax=ax,jitter=True,linewidth=1
             )
sns.stripplot('PGD_agg','concentration',
              marker='^',color='black',edgecolor='black',
              data=tmp[tmp['Inotrope']==1],
              ax=ax,jitter=True,linewidth=1
             )
ax.set_xlabel('')
ax.set_xticklabels(['No PGD', 'Moderate\nand Severe PGD'])
ax.set_ylabel('Normalized\nKLKB1 ELISA Concentration',size=18)

noi_patch = Line2D([0],[0],marker='o',
                   markerfacecolor='lightgray',markeredgecolor='black',
                   color='w',markersize=5,label='no Inotrope')
i_patch = Line2D([0],[0],marker='^',
                 markerfacecolor='black',markeredgecolor='black',
                 color='w',markersize=5,label='Inotrope')
ax.legend(handles=[noi_patch,i_patch],title='',frameon=False)


ax.tick_params(axis='both', which='both', length=0,labelsize=14)
sns.despine()
fig.tight_layout()
fig.savefig(dropbox_figures+'KLKB1_to_PGD_agg_validation.png')

#### PGD 2-3 vs nonPGD

In [ ]:
fimps_df.query('Feature=="H0YAC1"')

In [ ]:
set_ = "17"
fimps_df.query('set==@set_').set_index('Feature')['mean'].sort_index(ascending=False).values

In [ ]:
best_params = fimps_df.query('set==@set_').set_index('Feature')['mean'].sort_index(ascending=False).values
best_params

In [ ]:
#http://ethen8181.github.io/machine-learning/text_classification/logistic.html
def predict_probability(data, weights):
    """probability predicted by the logistic regression"""
    score = np.dot(data, weights)
    predictions = 1 / (1 + np.exp(-score))
    return predictions

In [ ]:
data = pd.read_csv('../../data/KLKB1_80_DEIDENTIFIED_patient_validation.csv')
display(data.PGD.value_counts())
data = data.query('PGD in [0,2,3]')
data['PGD_agg'] = (data['PGD']>0).map({True : 1, False : 0})
Y_elisa = data[['PGD_agg']]
display(Y_elisa['PGD_agg'].value_counts())
elisa_X = data[['Inotrope','concentration']]
elisa_X['concentration'] = (elisa_X['concentration'] - elisa_X['concentration'].min()) / (elisa_X['concentration'].max() - elisa_X['concentration'].min())
ps = predict_probability(elisa_X.values,best_params)
fpr, tpr, thresholds = roc_curve(Y_elisa.values,ps,pos_label=1)
score = np.round(roc_auc_score(Y_elisa.values,ps),2)
fig,ax=plt.subplots(dpi=dpi)
ax.plot(fpr,tpr,c='red')
ax.plot(fpr,tpr,'.',c='red',mec='red',lw=0.001)
ax.set_ylabel('Sensitivity',size=18)
print('AUROC : {}'.format(score))
ax.set_xlabel('1 - Specificity',size=18)

ax.tick_params(axis='both', which='both', length=0,labelsize=14)
sns.despine()
fig.tight_layout()
fig.savefig(dropbox_figures+'assessment_set_equation_roc_curve_nopgd_vs_pgd23.png')
cs = []
for t in thresholds:
    tn, fp, fn, tp = confusion_matrix(Y_elisa.values,ps>=t).ravel()
    cs.append([tp,fp,fn,tn])
cs_df = (pd.
         DataFrame(cs,
                   columns=['TP','FP','FN','TN'],
                   index=thresholds
                  ).
         rename_axis('Threshold').
         reset_index().
         sort_values('Threshold',ascending=True).
         reset_index(drop=True)
)
cs_df['Sensitivity'] = cs_df['TP'] / (cs_df['TP'] + cs_df['FN'])
cs_df['Specificity'] = cs_df['TN'] / (cs_df['FP'] + cs_df['TN'])
cs_df['1-Specificity'] = 1 - cs_df['Specificity']
cs_df['Accuracy'] = ( cs_df['TP'] + cs_df['TN'] ) / ( cs_df['TP'] + cs_df['TN'] + cs_df['FP'] + cs_df['FN'])
cs_df['FPR'] = cs_df['FP'] / (cs_df['FP'] + cs_df['TN'])
cs_df['TPR'] = cs_df['TP'] / (cs_df['TP'] + cs_df['FN'])
cs_df['PPV'] = cs_df['TP'] / (cs_df['TP'] + cs_df['FP'])
cs_df['NPV'] = cs_df['TN'] / (cs_df['TN'] + cs_df['FN'])
display(cs_df)
cs_df.to_csv(dropbox_data+'assessment_set_equation_pgd0_vs_pgd23_performance_table.csv')

precision, recall, thresholds = precision_recall_curve(Y_elisa.values,ps,pos_label=1)
precision[len(precision)-1] = 0
score = np.round(average_precision_score(Y_elisa.values,ps),2)
fig,ax=plt.subplots(dpi=dpi)
ax.plot(recall,precision,c='red')
ax.plot(recall,precision,'.',c='red',mec='red',lw=0.001)
print('AUPRC : {}'.format(score))
ax.set_ylabel('Precision',size=18)
ax.set_xlabel('Recall',size=18)
ax.set_ylim(0,1)
ax.tick_params(axis='both', which='both', length=0,labelsize=14)
sns.despine()
fig.tight_layout()
fig.savefig(dropbox_figures+'assessment_set_equation_precision_recall_curve_nopgd_vs_pgd23.png')

In [ ]:
data = pd.read_csv('../../data/KLKB1_80_DEIDENTIFIED_patient_validation.csv')
display(data.PGD.value_counts())
data = data.query('PGD in [0,2,3]')
data['PGD_agg'] = (data['PGD']>0).map({True : 1, False : 0})
Y_elisa = data[['PGD_agg']]
display(Y_elisa['PGD_agg'].value_counts())
elisa_X = data[['concentration']]
elisa_X['concentration'] = (elisa_X['concentration'] - elisa_X['concentration'].min()) / (elisa_X['concentration'].max() - elisa_X['concentration'].min())
ps = predict_probability(elisa_X.values,[0.1959])
fpr, tpr, thresholds = roc_curve(Y_elisa.values,ps,pos_label=1)
score = np.round(roc_auc_score(Y_elisa.values,ps),2)
fig,ax=plt.subplots(dpi=dpi)
ax.plot(fpr,tpr,c='red')
ax.plot(fpr,tpr,'.',c='red',mec='red',lw=0.001)
ax.set_ylabel('Sensitivity',size=18)
print('AUROC : {}'.format(score))
ax.set_xlabel('1 - Specificity',size=18)

ax.tick_params(axis='both', which='both', length=0,labelsize=14)
sns.despine()
fig.tight_layout()
fig.savefig(dropbox_figures+'assessment_set_equation_roc_curve_nopgd_vs_pgd23_just_klkb1.png')
cs = []
for t in thresholds:
    tn, fp, fn, tp = confusion_matrix(Y_elisa.values,ps>=t).ravel()
    cs.append([tp,fp,fn,tn])
cs_df = (pd.
         DataFrame(cs,
                   columns=['TP','FP','FN','TN'],
                   index=thresholds
                  ).
         rename_axis('Threshold').
         reset_index().
         sort_values('Threshold',ascending=True).
         reset_index(drop=True)
)
cs_df['Sensitivity'] = cs_df['TP'] / (cs_df['TP'] + cs_df['FN'])
cs_df['Specificity'] = cs_df['TN'] / (cs_df['FP'] + cs_df['TN'])
cs_df['1-Specificity'] = 1 - cs_df['Specificity']
cs_df['Accuracy'] = ( cs_df['TP'] + cs_df['TN'] ) / ( cs_df['TP'] + cs_df['TN'] + cs_df['FP'] + cs_df['FN'])
cs_df['FPR'] = cs_df['FP'] / (cs_df['FP'] + cs_df['TN'])
cs_df['TPR'] = cs_df['TP'] / (cs_df['TP'] + cs_df['FN'])
cs_df['PPV'] = cs_df['TP'] / (cs_df['TP'] + cs_df['FP'])
cs_df['NPV'] = cs_df['TN'] / (cs_df['TN'] + cs_df['FN'])
display(cs_df)
cs_df.to_csv(dropbox_data+'assessment_set_equation_pgd0_vs_pgd23_performance_table_just_klkb1.csv')

precision, recall, thresholds = precision_recall_curve(Y_elisa.values,ps,pos_label=1)
precision[len(precision)-1] = 0
score = np.round(average_precision_score(Y_elisa.values,ps),2)
fig,ax=plt.subplots(dpi=dpi)
ax.plot(recall,precision,c='red')
ax.plot(recall,precision,'.',c='red',mec='red',lw=0.001)
print('AUPRC : {}'.format(score))
ax.set_ylabel('Precision',size=18)
ax.set_xlabel('Recall',size=18)
ax.set_ylim(0,1)
ax.tick_params(axis='both', which='both', length=0,labelsize=14)
sns.despine()
fig.tight_layout()
fig.savefig(dropbox_figures+'assessment_set_equation_precision_recall_curve_nopgd_vs_pgd23_just_klkb1.png')

In [ ]:
tmp = (pd.
       DataFrame(
           {'Probability' : ps, 'PGD_agg' : data['PGD_agg'].values, 'PGD' : data['PGD']
           }
       ).
       sort_values('Probability')
      )
tmp['n_cumsum'] = (tmp['PGD_agg'].cumsum())
tmp['perc_cumsum'] = (tmp['PGD_agg'].cumsum() / tmp['PGD_agg'].sum())*100

In [ ]:
tmp.query('PGD==2')

In [ ]:
tmp.query('PGD==3')

In [ ]:
tmp[tmp['PGD_agg']==0].reset_index(drop=True)

In [ ]:
fig,ax=plt.subplots(dpi=dpi)
sns.scatterplot('Probability',
               'perc_cumsum',
                data=tmp,
                s=0,
                color='black',
               ax=ax)
sns.lineplot('Probability',
               'perc_cumsum',
                data=tmp,
                color='lightgray',
               ax=ax)
ax2 = plt.twinx()
sns.scatterplot('Probability',
               'n_cumsum',
                data=tmp[tmp['PGD_agg']==0],
                s=10,
                marker='o',
                linewidth=.5,
                color='lightgray',
                edgecolor='black',
               ax=ax2)
sns.scatterplot('Probability',
               'n_cumsum',
                data=tmp[tmp['PGD_agg']==1],
                hue='PGD',
                palette=['darkgray','black'],
                s=40,
                marker='^',
                linewidth=.5,
                edgecolor='black',
               ax=ax2)
ax.set_ylabel('Percent of PGD patients')
ax2.set_ylabel('Number of PGD patients')
ax.set_xlabel('Probability of PGD')

ax2.set_yticks(np.arange(0,8,1))
ax2.set_yticklabels(np.arange(0,8,1))

ax.legend().remove()
ax2.legend().remove()

noi_patch = Line2D([0],[0],marker='^',
                   markerfacecolor='darkgray',markeredgecolor='black',
                   color='w',markersize=5,label='Moderate')
i_patch = Line2D([0],[0],marker='^',
                 markerfacecolor='black',markeredgecolor='black',
                 color='w',markersize=5,label='Severe')
ax.legend(handles=[noi_patch,i_patch],title='',frameon=False)

fig.tight_layout()
fig.savefig(dropbox_figures+'validation_data_calibration_curve.png')

### Clinical dignostic figures

In [ ]:
data.query('PGD in [0,2,3]').dropna(subset=['C3','C4','Total Complement'])['PGD'].value_counts()

In [ ]:
type_='grades_03'
vars_=['C3','C4','Total Complement','Complement','ESR','hsCRP']
pgds = ['PGD_agg']
for p in pgds:
    if p=='PGD_agg':
        plot = data.query('PGD in [0,3]')
    else:
        plot = data.copy()
    for var in vars_:
        if var=="Complement":
            cvars = ['C3','C4','Total Complement']
            tmp = (plot.
                   loc[:,[p,'C3','C4','Total Complement','Inotrope']].
                   dropna().
                   melt(id_vars=[p,'Inotrope'],var_name='Complement Type')
                  )
            fig,ax=plt.subplots(dpi=dpi)
            g = sns.stripplot('Complement Type','value',
                              hue=p,palette=['lightgray','black'],
                              data=tmp,ax=ax,
                              marker='o',color='darkgray',linewidth=.2,edgecolor='black',
                              dodge=True,jitter=True)
            ax.set_xticklabels(ax.get_xticklabels(),size=14)
            ax.legend(loc='best',handles=[
                    Line2D([0],[0],color='lightgray',marker='o',linewidth=0,label='no PGD'),
                    Line2D([0],[0],color='black',marker='o',linewidth=0,label='PGD')
                ])
            fig.tight_layout()
            if p=='PGD_agg':
                    ax.set_xlabel('')
            fig.savefig(dropbox_figures+
                        p+'_x_Complements_'+type_+'.png')
        else:
            for i,grp1 in plot[[p,var]].dropna().groupby(p):
                for j,grp2 in plot[[p,var]].dropna().groupby(p):
                    if i<j:
                        a = grp1[var].values
                        b = grp2[var].values
                        tmp = pd.DataFrame([grp1[var].describe(),
                                            grp2[var].describe()],
                                           index=['PGD'+str(i),'PGD'+str(j)]
                                          )
                        tmp.index.name=p
                        tmp.columns.name=var
                        test = mannwhitneyu(b,a)
                        m,sd=np.mean(b),np.std(b)
                        print(var)
                        print(m)
                        print(sd)
                        m,sd=np.mean(a),np.std(a)
                        print(m)
                        print(sd)
                fig,ax=plt.subplots(dpi=dpi)
                g = sns.boxplot(p,var,data=plot,ax=ax,color='lightgrey',fliersize=0)
                g = sns.stripplot(p,var,hue='Inotrope',palette=['darkgray','black'],
                                  data=plot,size=10,
                                  ax=ax,marker='o',linewidth=.5,edgecolor='black',
                              dodge=True,jitter=True)
                if var!='ESR':
                    ax.set_ylabel(var+'\nconcentration',size=18)
                else:
                    ax.set_ylabel(var,size=18)
                ax.set_xticklabels(ax.get_xticklabels(),size=14)
                ax.set_xlabel(p.replace('_agg',''),size=18)
                ax.legend(loc='upper center',handles=[
                    Line2D([0],[0],color='darkgray',marker='o',linewidth=0,label='no Inotrope'),
                    Line2D([0],[0],color='black',marker='o',linewidth=0,label='Inotrope')
                ],frameon=False)
                if p=='PGD_agg':
                    ax.set_xticklabels(['no PGD','PGD'],size=14)
                    ax.set_xlabel('')
                    ax.set_title('U statistic = {}; p-value = {}'.
                                 format(np.round(test[0],2),np.round(test[1],2)),size=18)
                fig.tight_layout()
                fig.savefig(dropbox_figures+p+'_x_'+var+'_'+type_+'.png')

## Supplemental Figures

### Clinical descriptions

#### PGD by all

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv("../../data/integrated_sample_groups_imputed_data_raw.csv",index_col=0).set_index('Sample')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style="ticks")
sns.set_style('whitegrid')

x='PGD'
vars_ = np.setdiff1d(data.columns.values,'PGD')
for v in vars_:
    y = v
    try:
        g = sns.catplot(y,x,hue=x,data=data)
        g.fig.dpi = 150
        g.set_axis_labels(y_var='')
        g.set_yticklabels('')
    except:
        g = sns.catplot(x,hue=y,kind='count',data=data)
        g.fig.dpi = 150
        g.set_axis_labels(y_var='Count')
    g.savefig(dropbox_figures+x+'_by_'+y+'.pdf')

#### Univariate association statistics

#### clinical

In [ ]:
uni = pd.read_csv('../../data/bootstrap_clinical_logit/integrated_logit_bootstrap_pgd_~_clinical_features.csv').reset_index(drop=True)
uni_agg = pd.read_csv('../../data/bootstrap_clinical_logit/integrated_logit_bootstrap_pgd_~_clinical_features_lwr_mean_median_upr.csv').reset_index(drop=True)
uni['odds'] = np.log(uni['odds'])
uni_agg['mean'] = np.log(uni_agg['mean'])

In [ ]:
var_ord = uni_agg.sort_values('mean',ascending=False).variable.unique()
uni_agg = uni_agg.sort_values('mean',ascending=False)
dfs = []
for var in var_ord:
    dfs.append(uni.query('variable==@var'))
data = pd.concat(dfs)

In [ ]:
fs = data.variable.str.replace('_Y','')
fs = fs.str.replace('_',' ')
data.variable = fs

fs = uni_agg.variable.str.replace('_Y','')
fs = fs.str.replace('_',' ')
uni_agg.variable = fs



In [ ]:
fig,ax = plt.subplots(dpi=400,figsize=(10,12))

sns.stripplot('odds','variable',data=data,ax=ax,alpha=0.2)

sns.stripplot('mean','variable',data=uni_agg,ax=ax,jitter=False,color="red",linewidth=.5)

ax.set_ylabel('')
ax.set_xlabel('Population risk coefficient')

fig.tight_layout()

plt.savefig(dropbox_figures+'clinical_characteristics_population_risk_stripplot.pdf',width=20,height=30)

#### protein

In [ ]:
uniprot = pd.read_csv('../../data/uniprot-all_20171124.tab.gz',sep='\t')

In [ ]:
characterized_prots = uniprot.query('Organism=="Homo sapiens (Human)"').Entry.values

In [ ]:
idmap = uniprot[['Entry','Gene names  (primary )']].rename(columns={'Entry' : 'Protein',"Gene names  (primary )" : 'Gene_name'})
idmap_sub = idmap[idmap.Protein.isin(characterized_prots)]
idmap_sub.to_csv('../../data/gene_list.txt',sep='\n',header=None,index=None)

In [ ]:
cohort = 'integrated'
logit = pd.read_csv("../../data/bootstrap_conditional_protein_logit/"+cohort+"/logit_bootstrap_pgd_~_protein_+_cohort_+_set_lwr_mean_median_upr.csv")

In [ ]:
tmp = logit.set_index('variable').join(idmap_sub.set_index('Protein'))
leftover_inds = tmp.Gene_name.isnull()
leftover_prots = tmp.index[leftover_inds].values
leftover_prots_split = [k.split('-')[0] for k in leftover_prots]

tmp_df = pd.DataFrame({'Protein' : leftover_prots,
                       'Split' : leftover_prots_split,
                       'cohort_identified_in' : cohort})

tmp_df_join = tmp_df.set_index('Split').join(idmap_sub.set_index('Protein'))

join_genes = tmp_df_join.Gene_name.values
join_prots = tmp_df_join.Protein.values

tmp.at[join_prots,'Gene_name'] = join_genes

display(len(tmp.dropna()[~tmp.dropna().Gene_name.str.startswith('IG')].query('lwr>1 | upr<1').index.values))
display(tmp.head())
tmp.dropna().reset_index(drop=True).to_csv('../../data/bootstrap_protein_univariate_features.csv')
pickle.dump(tmp.dropna()[~tmp.dropna().Gene_name.str.startswith('IG')].query('lwr>1 | upr<1').index.values,open('../../data/significant_bootstrap_protein_univariate_features.pkl','wb'))

null_prots = tmp_df_join[tmp_df_join.Gene_name.isnull()].index.values
df = tmp[~tmp.index.isin(null_prots)].reset_index(drop=True).set_index('Gene_name')

In [ ]:
stat='mean'
data = (df[~df.index.str.startswith('IG')].
        query('lwr>1 | upr<1').
        sort_values(stat,ascending=False))
data.index = [x.split(';')[0]+' family' if len(x.split(';'))>2 else x for x in data.index]

In [ ]:
data = pd.concat([data,uni_sig.set_index('variable')],sort=True)

In [ ]:
data['lwr'] = np.log(data['lwr'])
data['mean'] = np.log(data['mean'])
data['upr'] = np.log(data['upr']) 

In [ ]:
fig,ax = plt.subplots(dpi=dpi,figsize=(5,5))
display(data.shape)
ax.errorbar(y=data.index,
            x=data[stat],
            xerr=(data[stat] - data['lwr'],
                 data['upr'] - data[stat]),
           fmt='o',markersize=3,linewidth=1)
ax.plot([0,0],[0,len(data.index.unique())-1],'r--',linewidth=0.5)
ax.set_xlabel('Population risk coefficient',fontsize=16)
fig.tight_layout()
fig.savefig(dropbox_figures+'significant_proteins_and_clinical_characteristics.png')

### PCA colored by experimental batch

In [ ]:
X = pd.read_csv('../../data/integrated_X_all_but_immunoglobulin_proteins.csv',index_col=0)
display(X.head())
tmt_covs = pd.read_csv('../../data/integrated_tmt_tag_covariates.csv',index_col=0)
display(tmt_covs.head())
set_covs = pd.read_csv('../../data/integrated_set_covariates.csv',index_col=0)
display(set_covs.head())
cohort_covs = pd.read_csv('../../data/integrated_cohort_covariates.csv',index_col=0)
display(cohort_covs.head())

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
mod = PCA(n_components=2)

mod_data = pd.DataFrame(mod.fit_transform(X),columns=['PC1','PC2'],index=X.index)

In [ ]:
data = (mod_data.
        join(tmt_covs).
        melt(id_vars=['PC1','PC2'],
             var_name='Covariate').
        query('value==1')
       )
display(data.head())

fig,ax = plt.subplots(dpi=200)

sns.scatterplot('PC1','PC2',hue='Covariate',data=data,ax=ax)

ax.legend(loc='upper right',frameon=False,fontsize='small')

fig.tight_layout()

fig.savefig(dropbox_figures+'Protein_PCA_by_tmt_tag.png')

In [ ]:
data = (mod_data.
        join(set_covs).
        melt(id_vars=['PC1','PC2'],
             var_name='Covariate').
        query('value==1')
       )
display(data.head())

fig,ax = plt.subplots(dpi=200)

sns.scatterplot('PC1','PC2',hue='Covariate',data=data,ax=ax)

ax.legend(loc='upper right',frameon=False,fontsize='small')

fig.tight_layout()

fig.savefig(dropbox_figures+'Protein_PCA_by_set.png')

In [ ]:
data = (mod_data.
        join(cohort_covs).
        melt(id_vars=['PC1','PC2'],
             var_name='Covariate').
        query('value==1')
       )
display(data.head())

fig,ax = plt.subplots(dpi=200)

sns.scatterplot('PC1','PC2',hue='Covariate',data=data,ax=ax)

ax.legend(loc='upper right',frameon=False,fontsize='small')

fig.tight_layout()

fig.savefig(dropbox_figures+'Protein_PCA_by_cohort.png')

### KLKB1 tetramer prediction

In [ ]:
def performance_df_from_lst(lst):
    tmp = [lst[i][0] for i in range(len(lst))]
    data = (pd.concat(tmp,keys=range(len(tmp))).
            reset_index(level=1,drop=True).
            rename_axis('bootstrap').
            reset_index())
    return data

def feature_importances_df_from_lst(lst):
    boot_mods = [lst[i][1] for i in range(nboot)]
    dfs = []
    X = params['X'].copy()
    X.loc[:,'Intercept'] = 0
    for i in range(len(boot_mods)):
        for j in boot_mods[i].keys():
            mod = boot_mods[i][j]
            coef = []
            try:
                coef.extend([i for i in mod.feature_importances_])
            except:
                coef.extend([i for i in mod.coef_[0]])
            coef.extend(mod.intercept_)
            fs = []
            fs.extend(X.columns.values)
            df = pd.DataFrame({
                'Feature' : fs,
                'Gene_name' : (X.T.
                               join(idmap_sub.
                                    set_index('Protein'),how='left').
                               Gene_name.values),
                'Importance' : coef,
                'Model' : j,
                'Bootstrap' : i
            })
            dfs.append(df)
    return pd.concat(dfs)

In [ ]:
%run /Users/nickgiangreco/Research/Projects/exosome_pgf/src/python/prediction_functions.py

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

dir_ = '../../data/'
cohort = 'integrated'

classification_metrics = ['roc_auc','precision','recall','f1']
nboot=1000
n_jobs = 4
test_size = .15
cv_split = 10

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_minus_paris_covariates.csv',index_col=0)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)

idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['H0YAC1']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
klkb1_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
klkb1_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(klkb1_perf_df)
display(klkb1_fimp_df)

In [ ]:
i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
klkb1_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
klkb1_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(klkb1_perf_df)
display(klkb1_fimp_df)

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['P01042']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
kng1_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
kng1_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(kng1_perf_df)
display(kng1_fimp_df)

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['P00748']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
f12_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
f12_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(f12_perf_df)
display(f12_fimp_df)

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['H0YAC1','P01042']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
klkb1_kng1_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
klkb1_kng1_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(klkb1_kng1_perf_df)
display(klkb1_kng1_fimp_df)

In [ ]:
X_all['H0YAC1'][Y.values.reshape(1,-1)[0]==1].values

In [ ]:
import scipy as sc

X_all = X_all_proteins.join(X_all_clinical)

display(sc.stats.ttest_ind(X_all['H0YAC1'][Y.values.reshape(1,-1)[0]==1].values,
                           X_all['H0YAC1'][Y.values.reshape(1,-1)[0]==0].values))

display(sc.stats.ttest_ind(X_all['P00748'][Y.values.reshape(1,-1)[0]==1].values,
                           X_all['P00748'][Y.values.reshape(1,-1)[0]==0].values))

display(sc.stats.ttest_ind(X_all['P01042'][Y.values.reshape(1,-1)[0]==1].values,
                           X_all['P01042'][Y.values.reshape(1,-1)[0]==0].values))

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['H0YAC1','P00748']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
klkb1_f12_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
klkb1_f12_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(klkb1_f12_perf_df)
display(klkb1_f12_fimp_df)

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['P01042','P00748']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
f12_kng1_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
f12_kng1_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(f12_kng1_perf_df)
display(f12_kng1_fimp_df)

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['H0YAC1','P01042','P00748']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
tet_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
tet_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(tet_perf_df)
display(tet_fimp_df)

### KLKB1 inhibitor prediction

In [ ]:
def performance_df_from_lst(lst):
    tmp = [lst[i][0] for i in range(len(lst))]
    data = (pd.concat(tmp,keys=range(len(tmp))).
            reset_index(level=1,drop=True).
            rename_axis('bootstrap').
            reset_index())
    return data

def feature_importances_df_from_lst(lst):
    boot_mods = [lst[i][1] for i in range(nboot)]
    dfs = []
    X = params['X'].copy()
    X.loc[:,'Intercept'] = 0
    for i in range(len(boot_mods)):
        for j in boot_mods[i].keys():
            mod = boot_mods[i][j]
            coef = []
            try:
                coef.extend([i for i in mod.feature_importances_])
            except:
                coef.extend([i for i in mod.coef_[0]])
            coef.extend(mod.intercept_)
            fs = []
            fs.extend(X.columns.values)
            df = pd.DataFrame({
                'Feature' : fs,
                'Gene_name' : (X.T.
                               join(idmap_sub.
                                    set_index('Protein'),how='left').
                               Gene_name.values),
                'Importance' : coef,
                'Model' : j,
                'Bootstrap' : i
            })
            dfs.append(df)
    return pd.concat(dfs)

In [ ]:
%run /Users/nickgiangreco/Research/Projects/exosome_pgf/src/python/prediction_functions.py

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

dir_ = '../../data/'
cohort = 'integrated'

classification_metrics = ['roc_auc','precision','recall','f1']
nboot=1000
n_jobs = 4
test_size = .15
cv_split = 10

X_all_proteins = pd.read_csv(dir_+cohort+'_X_raw_all_proteins.csv',index_col=0)
X_all_clinical = pd.read_csv(dir_+cohort+'_X_clinical_and_cohort_minus_paris_covariates.csv',index_col=0)
Y = pd.read_csv(dir_+cohort+'_pgd_y.csv',index_col=0,header=None)

idmap_sub = pd.read_csv('../../data/protein_gene_map_full.csv')[['Protein','Gene_name']].dropna()

In [ ]:
X_all.columns.values

In [ ]:
params = {'Y' : Y, 'cv_split' : cv_split, 
          'metrics' : classification_metrics, 'n_jobs' : 1, 
          'test_size' : test_size,
          'retrained_models' : True, 'patient_level_predictions' : True}

X_all = X_all_proteins.join(X_all_clinical)
features = ['P05154']
X = X_all[features]

params.update({'X' : X,'models' : l1_logit_model.copy()})

lst = bootstrap_of_fcn(func=train_test_val_top_fold,
                       params=params,n_jobs=n_jobs,nboot=nboot)

i = 10000
fimps = feature_importances_df_from_lst(lst)
fimps['set'] = str(i)
inh_fimp_df = (fimps.
            groupby(['set','Feature'])['Importance'].
            describe(percentiles=[0.025,0.975]).
            loc[:,['2.5%','mean','97.5%']].
            sort_values('2.5%',ascending=False).
            reset_index()
          )
perf = performance_df_from_lst(lst)
perf['set'] = str(i)
inh_perf_df = (perf.
           groupby(['set'])['validation_roc_auc'].
           describe(percentiles=[0.025,0.975]).
           loc[:,['2.5%','mean','97.5%']].
           sort_values('2.5%',ascending=False).
           reset_index()
          )
display(inh_perf_df)
display(inh_fimp_df)